# 1. 패키지 임포트 및 설정

In [1]:
import pandas as pd
import numpy as np 
import re

from glob import glob
from tqdm import tqdm
import warnings

from pandas.api.types import is_numeric_dtype # column by column으로 데이터 타입 확인
from pandas.api.types import is_string_dtype

In [2]:
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder

In [3]:
warnings.filterwarnings(action='ignore') 

know_train = [pd.read_csv(path) for path in sorted(glob('./train/*.csv'))]
know_test  =  [pd.read_csv(path) for path in sorted(glob('./test/*.csv'))]

In [22]:
list(know_train[1].columns)

['idx',
 'cq1',
 'cq2',
 'cq3',
 'cq4',
 'cq5',
 'cq6',
 'cq7',
 'cq8',
 'cq9',
 'cq10',
 'cq11',
 'cq12',
 'cq13',
 'cq14',
 'cq15',
 'cq16',
 'cq17',
 'cq18',
 'cq19',
 'cq20',
 'cq21',
 'cq22',
 'cq23',
 'cq24',
 'cq25',
 'cq26',
 'cq27',
 'cq28',
 'cq29',
 'cq30',
 'cq31',
 'cq32',
 'cq33',
 'cq34',
 'cq35',
 'cq36',
 'cq37',
 'cq38',
 'cq39',
 'cq40',
 'cq41',
 'cq42',
 'cq43',
 'cq44',
 'cq45',
 'cq46',
 'cq47',
 'cq48',
 'cq49',
 'cq50_1',
 'cq50_2',
 'cq50_3',
 'cq50_4',
 'cq50_5',
 'cq50_6',
 'cq50_7',
 'cq50_8',
 'iq1',
 'iq2',
 'iq3',
 'iq4',
 'iq5',
 'iq6',
 'bq1',
 'bq2',
 'bq3',
 'bq4',
 'bq4_1a',
 'bq4_1b',
 'bq4_1c',
 'bq5',
 'bq5_1',
 'bq5_2',
 'bq6',
 'bq7',
 'bq8_1',
 'bq8_2',
 'bq8_3',
 'bq9',
 'bq10',
 'bq11',
 'bq12_1',
 'bq12_2',
 'bq12_3',
 'bq12_4',
 'bq12_5',
 'bq13',
 'bq14',
 'bq15',
 'bq16',
 'bq17',
 'bq18',
 'bq19',
 'bq20',
 'bq21',
 'bq221',
 'bq222',
 'bq223',
 'bq231',
 'bq232',
 'bq233',
 'bq234',
 'bq235',
 'bq241',
 'bq242',
 'bq243',
 'bq244',
 'b

In [23]:
(0.536-0.528)/0.528

0.015151515151515164

# 2. 연도별 전처리

## 2.0. 4개년도 공통 전처리 

### 2.0.1. 공백 및 NA값 처리

- 공백 : 0으로 대체

In [4]:
for df in know_train:
    for col in df.columns:
        df[col].replace(' ', '0', inplace=True)

In [5]:
for df in know_test:
    for col in df.columns:
        df[col].replace(' ', '0', inplace=True)

- NA값 : 0으로 대체 

In [6]:
for df in know_train:
    for col in df.columns:
        df[col].fillna('0', inplace=True)

In [7]:
for df in know_test:
    for col in df.columns:
        df[col].fillna('0', inplace=True)

In [8]:
# know_train[0].copy()
# know_train[1].copy()
# know_train[2].copy()
# know_train[3].copy()

- 자격증 칼럼 1개만 남기기

In [9]:
for df_cap  in enumerate(know_train):
    # df_cap[0] : 번호 : 0,1,2,3
    # df_cap[1] : 연도별 데이터 프레임
        df= df_cap[1]
        df = df.drop(['bq4_1b', 'bq4_1c'], axis = 1)
        know_train[df_cap[0]]  = df

In [10]:
for df_cap  in enumerate(know_test):
    # df_cap[0] : 번호 : 0,1,2,3
    # df_cap[1] : 연도별 데이터 프레임
        df= df_cap[1]
        df = df.drop(['bq4_1b', 'bq4_1c'], axis = 1)
        know_test[df_cap[0]]  = df

## 2.1. 2017년 전처리

### 2.1.1.  2017년 Train 데이터 전처리

In [11]:
train_2017 = know_train[0].copy()

#### 1) aq문항(일반업무활동능력)

##### 1-1) 데이터프레임 분할 

In [12]:
# aq문항 나누기
train_2017_a = train_2017.loc[:,'idx':'aq41_2']   
train_2017_a['knowcode'] = train_2017['knowcode']

- knowcode 는 총 538개 존재
- 9999999는 아직 배정되지 않은 직업코드(다른년도에 존재하는 직업코드)

#### 2_1_2. 결측치 확인

- null, ' ' 을 결측치로 간주
- 결측치는 우선 0으로 처리
- 최종적으로 결측치를 산업별, 업체유형별(bq1, bq2) 평균으로 처리 예정

In [13]:
# 산업별, 업체유형별 평균 처리하기 위한 컬럼 추가
train_2017_a['bq1'] = train_2017['bq1']
train_2017_a['bq2'] = train_2017['bq2']

In [14]:
# 빈칸으로 입력된 셀 값 확인.  질문 로직에 따른 미응답인지 표기오류 등으로 인한 미응답인지 확인해야함
train_2017_a.isin([' ']).sum().sum()

0

In [15]:
# 데이터 타입을 정수형으로 변환(정수형으로 변환하지 않으면 그룹핑 이후 평균값을 나타낼 때 정수형으로 변환하지 않은 컬럼은 표시되지 않음)
train_2017_a = train_2017_a.astype(int)

#### 2_1_3. 1번 문항 이상치 및 결측치 확인

In [16]:
# 1번 문항은 1~5 입력만 허용

for col in range (1, 42):
    for i in range(len(train_2017_a.index)):
        if (1 <= train_2017_a['aq'+str(col)+'_1'][i] <= 5) == False:
            print('error col :', col)
            print('error index :', i)
            
print('끝')

error col : 41
error index : 3985
끝


In [17]:
train_2017_a['aq41_1'][3985]

33

In [18]:
# 평균처리를 위해 null값 처리
train_2017_a['aq41_1'][3985] = np.nan

#### 2_1_4. 2번 문항 이상치 확인

In [19]:
# 2번 문항은 0~7 입력 허용

for col in range (1, 42):
    for i in range(len(train_2017_a.index)):
        if (train_2017_a['aq'+str(col)+'_2'][i] > 7) == True:
            print('error col :', col)
            print('error index :', i)
                
print('끝')

끝


- 이상치 : 허용된 입력값 외의 입력이 된 경우
- 1번 문항 : 1건(33으로 입력)
- 2번 문항 : 0건

#### 2_1_5. 로직 확인 및 결측치 처리

- 1번 문항에 1이라고 응답한 것 중 2번 문항에 0이라고 응답하지 않은 것 확인
- 1번 뭉항에 1이라고 응답하지 않은 것 중 2번 문항에 0이라고 응답하지 않은 것 확인

##### 2_1_5_1. 1번 문항 ==1 & 2번 문항 != 0 확인

In [20]:
not_same_1 = pd.DataFrame()
not_same_idx = []
not_same_column = []

for col in range(1,42):
    for i in range(len(train_2017_a.index)):
        if (train_2017_a['aq'+str(col)+'_1'][i] == 1) ==True & (train_2017_a['aq'+str(col)+'_2'][i] != 0) == True :
            not_same_idx.append(train_2017_a.index[i])
            not_same_column.append(train_2017_a.columns[col])
            
not_same_1['not_same_idx']=not_same_idx
not_same_1['not_same_column']=not_same_column

not_same_1

,not_same_idx,not_same_column


- 1번 문항에 1이라고 응답한 것은 모두 2번 문항에 0이라고 응답함

##### 2_1_5_2. 1번 문항 != 1 & 2번 문항 == 0 확인

In [21]:
not_same_2 = pd.DataFrame()
not_same_idx = []
not_same_column = []

for col in range(1,42):
    for i in range(len(train_2017_a.index)):
        if (train_2017_a['aq'+str(col)+'_1'][i] != 1) ==True & (train_2017_a['aq'+str(col)+'_2'][i] == 0) == True :
            not_same_idx.append(train_2017_a.index[i])
            not_same_column.append(train_2017_a.columns[col])
            
not_same_2['not_same_idx']=not_same_idx
not_same_2['not_same_column']=not_same_column

not_same_2

,not_same_idx,not_same_column
0,89,aq6_1
1,577,aq6_1
2,2422,aq6_1
3,4568,aq6_1
4,3137,aq7_2
5,4564,aq7_2
6,6539,aq7_2
7,715,aq14_1
8,792,aq14_1
9,3960,aq14_1


In [22]:
len(not_same_2)

29

In [23]:
# 평균에 로직에 맞지 않는 값이 포함되지 않기 위해 0을 null값 처리

In [24]:
for col in range(1,42):
    for i in range(len(train_2017_a.index)):
        if (train_2017_a['aq'+str(col)+'_1'][i] != 1) ==True & (train_2017_a['aq'+str(col)+'_2'][i] == 0) == True :
            train_2017_a['aq'+str(col)+'_2'][i] = np.nan

In [25]:
train_2017_a.isna().sum().sum()

30

- 로직 부적합 확인
- 1번 문항에 1로 응답한 것 중 2번 문항에 응답한 것 0건
- 1번 문항에 2,3,4,5로 응답한 것 중 2번 문항에 응답하지 않은 것 29건
- 해당 인덱스에 해당하는 산업별, 업체유형별 평균값으로 처리 예정
- 로직에 맞지 않는 값을 평균에 포함되지 않게 하기 위해 우선 null 값 처리 후 평균 테이블을 만듬

#### 2_1_6. 결측치 및 이상치 평균값 처리

In [26]:
train_2017_a_mean = train_2017_a.groupby(['bq1', 'bq2']).mean().reset_index()

In [27]:
# 컬럼수 반드시 확인!
train_2017_a_mean

,bq1,bq2,idx,aq1_1,aq1_2,aq2_1,aq2_2,aq3_1,aq3_2,aq4_1,...,aq37_2,aq38_1,aq38_2,aq39_1,aq39_2,aq40_1,aq40_2,aq41_1,aq41_2,knowcode
0,1,1,4827.565217,2.782609,3.130435,2.695652,2.956522,2.739130,3.130435,2.869565,...,2.456522,2.326087,2.369565,1.891304,1.739130,1.673913,1.282609,2.391304,2.717391,1.081992e+06
1,1,2,4750.285714,3.857143,4.714286,3.714286,5.000000,3.714286,4.571429,2.714286,...,4.285714,2.857143,3.285714,3.142857,4.142857,2.428571,2.000000,2.571429,3.571429,1.065596e+05
2,1,4,4493.558824,2.500000,2.764706,2.460784,2.745098,2.382353,2.500000,2.529412,...,1.852941,1.941176,1.803922,1.578431,1.186275,1.431373,0.823529,2.186275,2.323529,2.150861e+06
3,1,6,5405.500000,4.000000,5.000000,3.000000,4.500000,3.000000,4.500000,3.000000,...,3.500000,3.500000,4.500000,2.500000,3.000000,2.500000,2.000000,3.500000,4.500000,9.016010e+05
4,2,1,4707.714286,2.785714,3.000000,2.500000,2.642857,3.285714,3.785714,3.071429,...,1.928571,2.142857,1.785714,2.142857,2.071429,1.571429,0.857143,1.785714,1.214286,7.312439e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,20,1,837.333333,3.000000,3.333333,2.666667,2.333333,3.000000,3.000000,1.666667,...,2.333333,2.000000,1.333333,1.333333,0.666667,1.000000,0.000000,1.000000,0.000000,6.611677e+05
89,20,4,5831.000000,2.714286,2.857143,3.142857,3.571429,2.857143,3.285714,1.714286,...,1.714286,2.142857,2.000000,1.285714,0.571429,1.428571,0.714286,1.000000,0.000000,5.405153e+05
90,20,6,2849.000000,1.250000,0.500000,1.500000,0.500000,1.250000,0.250000,1.000000,...,2.000000,2.000000,2.000000,1.750000,1.500000,1.500000,1.250000,1.500000,0.750000,5.502010e+05
91,21,1,4693.000000,2.500000,2.500000,3.000000,4.000000,2.500000,2.500000,1.500000,...,2.500000,2.000000,2.500000,3.000000,3.500000,1.000000,0.000000,1.500000,0.500000,4.112010e+05


##### 2_1_6_2. 이상치 및 결측치 평균 처리

In [28]:
# 1번 문항 이상치 처리
for col in range(1,42):
    for i in range(len(train_2017_a.index)):
        if train_2017_a.loc[:,'aq'+str(col)+'_1'].isna()[i] == True:
            train_2017_a['aq'+str(col)+'_1'][i] = train_2017_a_mean[
                train_2017_a_mean['bq1'] == train_2017_a['bq1'][i]][train_2017_a_mean['bq2'] == train_2017_a['bq2'][i]]['aq'+str(col)+'_1']

In [29]:

# 1번 문항에 2,3,4,5 로 응답했는데 2번 문항에 응답하지 않은 케이스 처리
for col in range(1,42):
    for i in range(len(train_2017_a.index)):
        if (train_2017_a['aq'+str(col)+'_1'][i] != 1) & (train_2017_a.loc[:,'aq'+str(col)+'_2'].isna()[i] == True):
            train_2017_a['aq'+str(col)+'_2'][i] = train_2017_a_mean[
                train_2017_a_mean['bq1'] == train_2017_a['bq1'][i]][train_2017_a_mean['bq2'] == train_2017_a['bq2'][i]]['aq'+str(col)+'_2']

In [30]:
# 결측치 처리 확인
train_2017_a.isna().sum().sum()

0

In [31]:
# 데이터 타입 정수형으로 다시 한번 변환 확인
train_2017_a = train_2017_a.astype(int)

In [32]:
train_2017_a = train_2017_a.drop(['bq1', 'bq2'], axis = 1)

In [33]:
# train_2017_a 전처리 완료
train_2017_a

,idx,aq1_1,aq1_2,aq2_1,aq2_2,aq3_1,aq3_2,aq4_1,aq4_2,aq5_1,...,aq37_2,aq38_1,aq38_2,aq39_1,aq39_2,aq40_1,aq40_2,aq41_1,aq41_2,knowcode
0,0,3,3,3,3,3,3,4,4,3,...,2,2,2,5,2,2,2,2,2,825101
1,1,4,5,4,5,3,4,3,4,3,...,4,3,4,2,2,1,0,1,0,140204
2,2,3,4,3,4,3,4,5,6,4,...,4,3,4,1,0,1,0,1,0,140204
3,3,3,3,3,3,3,5,4,5,4,...,4,4,4,4,4,4,2,4,3,140601
4,4,4,5,3,4,3,4,4,5,3,...,2,3,4,2,2,1,0,1,0,140204
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9481,9481,3,5,2,4,3,3,2,2,2,...,4,3,5,2,3,3,3,2,5,411301
9482,9482,5,5,5,5,5,5,3,4,4,...,4,4,5,2,1,1,0,1,0,151105
9483,9483,3,3,4,6,3,3,4,5,4,...,2,1,0,1,0,1,0,2,2,701101
9484,9484,3,5,3,5,4,5,3,4,3,...,5,4,5,4,4,1,0,3,4,25402


### 2_2. bq문항(인구통계학적 변인)

#### 2_2_1. 데이터프레임 분할

In [34]:
train_2017_b = train_2017.loc[:,'bq1':'knowcode']

#### 2_2_2. 결측치 확인

In [35]:
train_2017_b.isin(['없다']).sum().value_counts()

0       63
17       1
21       1
6606     1
497      1
5723     1
6667     1
7785     1
3        1
dtype: int64

In [36]:
train_2017_b.isin(['없음']).sum().value_counts()

0      66
251     1
25      1
210     1
218     1
344     1
dtype: int64

In [37]:
train_2017_b.isin(['무']).sum().value_counts()

0    70
1     1
dtype: int64

In [38]:
# 빈칸(' '), '없음', '없다', '무'  0으로 처리
train_2017_b.replace({' ':0, '없다':0, '없음':0, '무':0}, inplace = True)

#### 2_2_3. 데이터 타입 변환

In [39]:
# 데이터 타입 변환을 위한 처리
# 메타데이터 중 변수정보 파일을 참고
# 텍스트 데이터 --> object, 숫자형 데이터 --> int로 변환.
# 숫자형 데이터 중 명목형자료 들에 대해선 추후 재처리
# 서술형 텍스트 컬럼 삭제

In [40]:
text_columns = ['bq4_1a', 'bq5_2','bq19_1','bq30','bq31','bq32','bq33','bq34', 'bq38_1']

In [41]:
text_columns = pd.Series(text_columns)

In [42]:
train_2017_b.loc[:,text_columns] = train_2017_b.loc[:,text_columns].astype(str)

In [43]:
for col in range(len(train_2017_b.columns)):
    if (text_columns.isin([train_2017_b.columns[col]]).sum() == 1 ) == False:
        train_2017_b[train_2017_b.columns[col]] = train_2017_b[train_2017_b.columns[col]].astype(int)

In [44]:
# 데이터 타입 변환 완료
train_2017_b.info(max_cols=10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9486 entries, 0 to 9485
Columns: 71 entries, bq1 to knowcode
dtypes: int64(62), object(9)
memory usage: 5.1+ MB


#### 2_2_4. 로직 적합 여부 확인

- 4번 1응답시 4_1 != 0, 2응답시 4_1 == 0
- 5번 1응답시 5_1, 5_2 != 0, 2응답시 5_1, 5_2 == 0
- 38번 1 응답시 38_1 == 0, 2~6응답시 38_1 != 0
- 39번 1,2,3,4 응답시 40, 41_1, 41_2 != 0,  41_3 == 0
- 39번 5,6,7응답시 40, 41_1, 41_2 == 0, 41_3 != 0

In [45]:
#전체적인 입력값 확인(밀린 값 등이 있을수도 있어 확인)

for col in range(len(train_2017_b.columns)):
    if (train_2017_b.iloc[:,col].dtype == int)==True:
        print(train_2017_b.columns[col]+'입력값 :' , np.sort(train_2017_b.iloc[:,col].unique()))
    else:
        print(train_2017_b.columns[col]+'입력값 :',train_2017_b.iloc[:,col].unique())

bq1입력값 : [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21]
bq2입력값 : [1 2 3 4 5 6]
bq3입력값 : [1 2 3 4 5 6]
bq4입력값 : [1 2]
bq4_1a입력값 : ['자동차도장기능사' '건축전기설비기술사' '지적기사' ... '전기감리기술자격증' '전문의(의사면허증)' 'CKLU']
bq5입력값 : [1 2]
bq5_1입력값 : [0 1 2 3 4 5 6]
bq5_2입력값 : ['실무교육' '0' '측량 및 정보통신기술 교육' ... '회계 프로그램 사용 숙지' '편집기술에 원리와 이해능력'
 '도면 작성 교육, 전자 기초 이해']
bq6입력값 : [1 2 3 4 5 6 7]
bq7입력값 : [1 2 3 4 5 6 7]
bq8_1입력값 : [1 2 3 4 5]
bq8_2입력값 : [1 2 3 4 5]
bq8_3입력값 : [1 2 3 4 5]
bq9입력값 : [1 2 3 4 5]
bq10입력값 : [1 2 3 4 5]
bq11입력값 : [1 2 3 4 5]
bq12_1입력값 : [1 2 3 4 5]
bq12_2입력값 : [0 1 2 3 4 5]
bq12_3입력값 : [0 1 2 3 4 5]
bq12_4입력값 : [0 1 2 3 4 5]
bq12_5입력값 : [1 2 3 4 5]
bq13입력값 : [1 2 3 4 5]
bq14입력값 : [1 2 3 4 5]
bq15_1입력값 : [1 2 3 4 5]
bq15_2입력값 : [1 2 3 4 5]
bq15_3입력값 : [1 2 3 4 5]
bq16입력값 : [1 2 3 4 5]
bq17입력값 : [1 2 3 4 5]
bq18_1입력값 : [1 2 3 4 5]
bq18_2입력값 : [1 2 3 4 5]
bq18_3입력값 : [1 2 3 4 5]
bq18_4입력값 : [1 2 3 4 5]
bq18_5입력값 : [1 2 3 4 5]
bq18_6입력값 : [1 2 3 4 5]
bq18_7입력값 : [1 2 3 4 5]
bq19입

##### 2_2_4_1. bq4 확인 및 처리

In [46]:
# 1(있다)라고 응답했을 때, 뒷문항에 응답하지 않은 경우 확인
not_same_4_1 = pd.DataFrame()
not_same_idx = []

for i in range(len(train_2017_b.index)):
    if (train_2017_b['bq4'][i] == 1) ==True:
        if ((train_2017_b['bq4_1a'][i] == '0') == True) and ((train_2017_b['bq4_1a'][i] == '0') == True) and (
            (train_2017_b['bq4_1b'][i] == '0') == True):
            not_same_idx.append(i)
            
not_same_4_1['not_same_idx']=not_same_idx

not_same_4_1

,not_same_idx


In [47]:
# 2(없다)라고 응답했을 때, 뒷문항에 응답한 경우 확인
not_same_4_2 = pd.DataFrame()
not_same_idx = []

for i in range(len(train_2017_b.index)):
    if (train_2017_b['bq4'][i] == 2) ==True:
        if ((train_2017_b['bq4_1a'][i] != '0') == True):
            not_same_idx.append(i)
            
not_same_4_2['not_same_idx']=not_same_idx

not_same_4_2

,not_same_idx


##### 2_2_4_2. bq5 확인 및 처리

In [48]:
# 1(있다)라고 응답했을 때, 뒷문항에 응답하지 않은 경우 확인
not_same_5_1 = pd.DataFrame()
not_same_idx = []

for i in range(len(train_2017_b.index)):
    if (train_2017_b['bq5'][i] == 1) ==True:
        if ((train_2017_b['bq5_1'][i] == 0) == True) and ((train_2017_b['bq5_2'][i] == '0') == True):
            not_same_idx.append(i)
            
not_same_5_1['not_same_idx']=not_same_idx

not_same_5_1

,not_same_idx


In [49]:
# 2(없다)라고 응답했을 때, 뒷문항에 응답한 경우 확인
not_same_5_2 = pd.DataFrame()
not_same_idx = []

for i in range(len(train_2017_b.index)):
    if (train_2017_b['bq5'][i] == 2) ==True:
        if ((train_2017_b['bq5_1'][i] != 0) == True) and ((train_2017_b['bq5_2'][i] != '0') == True):
            not_same_idx.append(i)
            
not_same_5_2['not_same_idx']=not_same_idx

not_same_5_2

,not_same_idx


##### 2_2_4_5. bq38 확인 및 처리

In [50]:
# 옳은로직
# 1응답시 --> bq38_1 응답X 
# 2,3,4,5,6응답시 --> bq38_1 응답O

In [51]:
# 2,3,4,5,6응답시 bq38_1에 응답하지 않은 경우

for i in range(len(train_2017_b.index)):
    if (train_2017_b['bq38'][i] > 1) == True:
        if (train_2017_b['bq38_1'][i] == 0)==True:
            print('error index: ', i)
            
print('끝')

끝


In [52]:
# 1응답시 bq38_1에 응답한 경우


for i in range(len(train_2017_b.index)):
    if (train_2017_b['bq38'][i] == 1) == True:
        if (train_2017_b['bq38_1'][i] != '0')==True:
            print('error index: ', i)
            
print('끝')

끝


##### 2_2_4_6. bq39 확인 및 처리

In [53]:
# 옳은로직
# 39_1에 1 응답시 --> 39_2는 1,2,3,4
# 39_2에 2 응답시 --> 39_2는 5,6,7

# 1,2,3,4 응답시 --> bq40, bq41_1, bq41_2 응답O, bq41_3 응답X
# 5,6,7 응답시 --> bq40, bq41_1, bq41_2 응답X, bq41_3 응답O

In [54]:
# 39_1에 1 응답시 39_2에 5,6,7 응답한 경우
for i in range(len(train_2017_b.index)):
    if (train_2017_b['bq39_1'][i] == 1) == True:
        if (train_2017_b['bq39_2'][i] >= 5 ) == True:
            print('error index :', i)
            
print('끝')

끝


In [55]:
# 39_1에 2 응답시 39_2에 1,2,3,4 응답한 경우
for i in range(len(train_2017_b.index)):
    if (train_2017_b['bq39_1'][i] == 2) == True:
        if (train_2017_b['bq39_2'][i] < 5 ) == True:
            print('error index :', i)
            
print('끝')

끝


In [56]:
# 39_1에 1응답시 bq40, bq41_1, bq41_2에 응답하지 않은 경우
error_bq40_1=[]
error_bq41_1_1=[]
error_bq41_2_1=[]

for i in range(len(train_2017_b.index)):
    if (train_2017_b['bq39_1'][i] == 1) == True:
        if ((train_2017_b['bq40'][i] ==0)==True):
            error_bq40_1.append(i)
            
for i in range(len(train_2017_b.index)):
    if (train_2017_b['bq39_1'][i] == 1) == True:
        if ((train_2017_b['bq41_1'][i] ==0)==True):
            error_bq41_1_1.append(i)
            
for i in range(len(train_2017_b.index)):
    if (train_2017_b['bq39_1'][i] == 1) == True:
        if ((train_2017_b['bq41_2'][i] ==0)==True):
            error_bq41_2_1.append(i)
            
print('bq40_1 오류 개수 :', len(error_bq40_1))
print('bq41_1_1 오류 개수 :', len(error_bq41_1_1))
print('bq41_2_1 오류 개수 :', len(error_bq41_2_1))

bq40_1 오류 개수 : 0
bq41_1_1 오류 개수 : 31
bq41_2_1 오류 개수 : 8185


In [57]:
# 39_1에 1 응답시 bq41_3에 응답한 경우
error_bq41_3_1 = []

for i in range(len(train_2017_b.index)):
    if (train_2017_b['bq39_1'][i] == 1) == True:
        if (train_2017_b['bq41_3'][i] != 0) == True:
            error_bq41_3_1.append(i)
            
print('bq41_3_1 오류 개수 :', len(error_bq41_3_1))

bq41_3_1 오류 개수 : 8170


In [58]:
# 39_1에 2응답시 bq40, bq41_1, bq41_2에 응답한 경우
error_bq40_2=[]
error_bq41_1_2=[]
error_bq41_2_2=[]

for i in range(len(train_2017_b.index)):
    if (train_2017_b['bq39_1'][i] == 2) == True:
        if ((train_2017_b['bq40'][i] !=0)==True):
            error_bq40_2.append(i)
            
for i in range(len(train_2017_b.index)):
    if (train_2017_b['bq39_1'][i] == 2) == True:
        if ((train_2017_b['bq41_1'][i] !=0)==True):
            error_bq41_1_2.append(i)
            
for i in range(len(train_2017_b.index)):
    if (train_2017_b['bq39_1'][i] == 2) == True:
        if ((train_2017_b['bq41_2'][i] !=0)==True):
            error_bq41_2_2.append(i)
            
print('bq40_2 오류 개수 :', len(error_bq40_2))
print('bq41_1_2 오류 개수 :', len(error_bq41_1_2))
print('bq41_2_2 오류 개수 :', len(error_bq41_2_2))

bq40_2 오류 개수 : 0
bq41_1_2 오류 개수 : 0
bq41_2_2 오류 개수 : 1264


In [59]:
# 39_1에 2 응답시 bq41_3에 응답하지 않은 경우
error_bq41_3_2 = []

for i in range(len(train_2017_b.index)):
    if (train_2017_b['bq39_1'][i] == 2) == True:
        if (train_2017_b['bq41_3'][i] == 0) == True:
            error_bq41_3_2.append(i)
            
print('bq41_3_2 오류 개수 :', len(error_bq41_3_2))

bq41_3_2 오류 개수 : 1301


##### 2_2_4_6_1. bq41 잘못입력된 값들 조정

In [60]:
# bq41_2와 bq41_3 값 변경
train_2017_b.rename(columns = {'bq41_2' : 'bq41_3', 'bq41_3':'bq41_2'}, inplace = True)

##### 2_2_4_6_2. bq41 값 위치 조정 후 재확인

In [61]:
# 39_1에 1응답시 bq40, bq41_1, bq41_2에 응답하지 않은 경우
error_bq40_1=[]
error_bq41_1_1=[]
error_bq41_2_1=[]

for i in range(len(train_2017_b.index)):
    if (train_2017_b['bq39_1'][i] == 1) == True:
        if ((train_2017_b['bq40'][i] ==0)==True):
            error_bq40_1.append(i)
            
for i in range(len(train_2017_b.index)):
    if (train_2017_b['bq39_1'][i] == 1) == True:
        if ((train_2017_b['bq41_1'][i] ==0)==True):
            error_bq41_1_1.append(i)
            
for i in range(len(train_2017_b.index)):
    if (train_2017_b['bq39_1'][i] == 1) == True:
        if ((train_2017_b['bq41_2'][i] ==0)==True):
            error_bq41_2_1.append(i)
            
print('bq40_1 오류 개수 :', len(error_bq40_1))
print('bq41_1_1 오류 개수 :', len(error_bq41_1_1))
print('bq41_2_1 오류 개수 :', len(error_bq41_2_1))

bq40_1 오류 개수 : 0
bq41_1_1 오류 개수 : 31
bq41_2_1 오류 개수 : 15


In [62]:
# 39_1에 1 응답시 bq41_3에 응답한 경우
error_bq41_3_1 = []

for i in range(len(train_2017_b.index)):
    if (train_2017_b['bq39_1'][i] == 1) == True:
        if (train_2017_b['bq41_3'][i] != 0) == True:
            error_bq41_3_1.append(i)
            
print('bq41_3_1 오류 개수 :', len(error_bq41_3_1))

bq41_3_1 오류 개수 : 0


In [63]:
# 39_1에 2응답시 bq40, bq41_1, bq41_2에 응답한 경우
error_bq40_2=[]
error_bq41_1_2=[]
error_bq41_2_2=[]

for i in range(len(train_2017_b.index)):
    if (train_2017_b['bq39_1'][i] == 2) == True:
        if ((train_2017_b['bq40'][i] !=0)==True):
            error_bq40_2.append(i)
            
for i in range(len(train_2017_b.index)):
    if (train_2017_b['bq39_1'][i] == 2) == True:
        if ((train_2017_b['bq41_1'][i] !=0)==True):
            error_bq41_1_2.append(i)
            
for i in range(len(train_2017_b.index)):
    if (train_2017_b['bq39_1'][i] == 2) == True:
        if ((train_2017_b['bq41_2'][i] !=0)==True):
            error_bq41_2_2.append(i)
            
print('bq40_2 오류 개수 :', len(error_bq40_2))
print('bq41_1_2 오류 개수 :', len(error_bq41_1_2))
print('bq41_2_2 오류 개수 :', len(error_bq41_2_2))

bq40_2 오류 개수 : 0
bq41_1_2 오류 개수 : 0
bq41_2_2 오류 개수 : 0


In [64]:
# 39_1에 2 응답시 bq41_3에 응답하지 않은 경우
error_bq41_3_2 = []

for i in range(len(train_2017_b.index)):
    if (train_2017_b['bq39_1'][i] == 2) == True:
        if (train_2017_b['bq41_3'][i] == 0) == True:
            error_bq41_3_2.append(i)
            
print('bq41_3_2 오류 개수 :', len(error_bq41_3_2))

bq41_3_2 오류 개수 : 37


In [65]:
# 로직에 어긋난 값 null 값으로 처리

In [66]:
for i in range(len(train_2017_b.index)):
    if (train_2017_b['bq39_1'][i] == 1) == True:
        if ((train_2017_b['bq41_1'][i] ==0)==True):
            train_2017_b['bq41_1'][i] = np.nan
            
for i in range(len(train_2017_b.index)):
    if (train_2017_b['bq39_1'][i] == 1) == True:
        if ((train_2017_b['bq41_2'][i] ==0)==True):
            train_2017_b['bq41_2'][i] = np.nan
    
            
for i in range(len(train_2017_b.index)):
    if (train_2017_b['bq39_1'][i] == 2) == True:
        if (train_2017_b['bq41_3'][i] == 0) == True:
            train_2017_b['bq41_3'][i] = np.nan

##### 2_2_4_6_3. 로직에 어긋난 값 평균처리

In [67]:
train_2017_b_mean = train_2017_b.groupby(['bq1', 'bq2']).mean().reset_index()

In [68]:
train_2017_b_mean.isna().sum().sum()

1

In [69]:
train_2017_b_mean['bq41_2'][train_2017_b_mean['bq41_2'].isna()==True]

86   NaN
Name: bq41_2, dtype: float64

In [70]:
train_2017_b_mean.iloc[86]

bq1             19.0
bq2              5.0
bq3              2.0
bq4              1.0
bq5              1.0
              ...   
bq40             1.0
bq41_1        4500.0
bq41_3           0.0
bq41_2           NaN
knowcode    110103.0
Name: 86, Length: 62, dtype: float64

In [71]:
train_2017_b.query('bq1 == 19 and bq2 ==5')

,bq1,bq2,bq3,bq4,bq4_1a,bq5,bq5_1,bq5_2,bq6,bq7,...,bq37,bq38,bq38_1,bq39_1,bq39_2,bq40,bq41_1,bq41_3,bq41_2,knowcode
4087,19,5,2,1,언어재활사 자격증,1,3,"관찰,실습",4,5,...,41,5,언어치료학,1,1,1,4500.0,0.0,NaN,110103


In [72]:
# 처리가 되지 않은 결측치는 해당 산업, 업체유형에서 유일한 값이므로 최저시급(2300만원) 기준 대치

In [73]:
train_2017_b['bq41_2'][4087] = 2300

In [74]:
train_2017_b_mean = train_2017_b.groupby(['bq1', 'bq2']).mean().reset_index()

In [75]:
train_2017_b_mean = train_2017_b_mean.astype(int)

In [76]:
train_2017_b_mean

,bq1,bq2,bq3,bq4,bq5,bq5_1,bq6,bq7,bq8_1,bq8_2,...,bq36,bq37,bq38,bq39_1,bq39_2,bq40,bq41_1,bq41_3,bq41_2,knowcode
0,1,1,1,1,1,0,4,1,2,2,...,1,54,2,1,2,1,1873,1311,1286,1081992
1,1,2,4,1,1,1,5,5,4,4,...,1,39,5,1,1,1,4185,0,3000,106559
2,1,4,1,1,1,0,5,1,2,2,...,1,56,1,1,5,0,281,3137,219,2150860
3,1,6,1,2,2,0,6,1,3,2,...,1,58,3,2,6,0,0,3000,0,901601
4,2,1,2,1,1,2,3,2,2,2,...,1,44,2,1,1,1,3707,0,2384,731243
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,20,1,2,2,1,1,2,1,2,2,...,1,46,2,1,2,1,2933,0,1666,661167
89,20,4,1,2,1,0,6,1,2,2,...,2,57,1,1,5,0,485,1185,457,540515
90,20,6,1,2,2,0,1,1,2,1,...,2,48,2,1,3,2,1675,0,1287,550201
91,21,1,2,2,2,0,5,5,4,3,...,2,32,4,1,2,1,3650,0,2500,411201


In [77]:
# bq39_1에 1이라 응답한 것 중 41_1에 응답하지 않은 것 처리
for i in range(len(train_2017_b.index)):
    if (train_2017_b['bq39_1'][i] == 1) == True:
        if train_2017_b.loc[:,'bq41_1'].isna()[i] == True:
            train_2017_b['bq41_1'][i] = train_2017_b_mean[
                train_2017_b_mean['bq1'] == train_2017_b['bq1'][i]][train_2017_b_mean['bq2'] == train_2017_b['bq2'][i]]['bq41_1']

In [78]:
# bq39_1에 1이라 응답한 것 중 bq41_2에 응답하지 않은 것 처리

for i in range(len(train_2017_b.index)):
    if (train_2017_b['bq39_1'][i] == 1) == True:
        if train_2017_b.loc[:,'bq41_2'].isna()[i] ==True:
            train_2017_b['bq41_2'][i] = train_2017_b_mean[
                train_2017_b_mean['bq1'] == train_2017_b['bq1'][i]][train_2017_b_mean['bq2'] == train_2017_b['bq2'][i]]['bq41_2']

In [79]:
# bq39_1에 2이라 응답한 것 중 bq41_3에 응답하지 않은 것들 처리

for i in range(len(train_2017_b.index)):
    if (train_2017_b['bq39_1'][i] == 2) == True:
        if train_2017_b.loc[:,'bq41_3'].isna()[i] == True:
            train_2017_b['bq41_3'][i] = train_2017_b_mean[
                train_2017_b_mean['bq1'] == train_2017_b['bq1'][i]][train_2017_b_mean['bq2'] == train_2017_b['bq2'][i]]['bq41_3']

In [80]:
# 결측치 처리 확인
train_2017_b.isna().sum().sum()

0

In [81]:
#df_2017_b 숫자 데이터 결측치 처리 완료
train_2017_b

,bq1,bq2,bq3,bq4,bq4_1a,bq5,bq5_1,bq5_2,bq6,bq7,...,bq37,bq38,bq38_1,bq39_1,bq39_2,bq40,bq41_1,bq41_3,bq41_2,knowcode
0,3,1,6,1,자동차도장기능사,1,2,실무교육,5,2,...,52,2,실업,1,1,1,4000.0,0.0,2200.0,825101
1,13,1,3,1,건축전기설비기술사,2,0,0,6,5,...,38,4,건축공학,1,1,1,3744.0,0.0,2400.0,140204
2,13,1,3,1,건축전기설비기술사,2,0,0,6,5,...,50,4,건축공학,1,1,1,4000.0,0.0,2400.0,140204
3,13,2,6,1,지적기사,1,1,측량 및 정보통신기술 교육,2,4,...,42,4,환경학과,1,1,1,7000.0,0.0,3500.0,140601
4,6,1,4,1,건축전기설비기술사,2,0,0,6,5,...,51,4,건축공학,1,1,1,4000.0,0.0,2500.0,140204
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9481,13,1,2,2,0,1,2,편집기술에 원리와 이해능력,7,4,...,50,4,산업디자인,1,1,1,5200.0,0.0,1800.0,411301
9482,3,1,6,2,0,1,5,"도면 작성 교육, 전자 기초 이해",6,5,...,37,4,우주항공공학,1,1,1,4000.0,0.0,3000.0,151105
9483,6,4,1,2,0,2,0,0,6,1,...,32,2,기계,1,4,2,2700.0,0.0,1500.0,701101
9484,15,2,4,2,0,2,0,0,5,2,...,40,4,문헌정보학,1,1,1,6800.0,0.0,2500.0,25402


#### 2_2_5. 텍스트 데이터 컬럼 삭제

- bq5_2, bq19_1, bq30, bq31, bq32, bq33, bq34 삭제

In [82]:
train_2017_b.drop(['bq5_2', 'bq19_1','bq30', 'bq31', 'bq32', 'bq33', 'bq34'], axis = 1 ,inplace = True)

#### 2_2_6. 학습을 위한 전처리 데이터 통합

In [83]:
train_2017_a = train_2017_a.drop('knowcode', axis = 1)

In [84]:
train_2017 = pd.concat([train_2017_a, train_2017_b], axis = 1)

In [85]:
# 텍스트 컬럼을 제외한 나머지 컬럼 데이터 형태 int형으로 변환
for col in range(len(train_2017.columns)):
    if train_2017.columns[col] == 'bq4_1a':
        pass
    elif train_2017.columns[col] == 'bq5_2':
        pass
    elif train_2017.columns[col] == 'bq30':
        pass
    elif train_2017.columns[col] == 'bq31':
        pass
    elif train_2017.columns[col] == 'bq32':
        pass
    elif train_2017.columns[col] == 'bq33':
        pass
    elif train_2017.columns[col] == 'bq34':
        pass
    elif train_2017.columns[col] == 'bq38_1':
        pass
    else:
        train_2017.iloc[:,col] = train_2017.iloc[:,col].astype(int)

### 2.1.2.  2017년 Test 데이터 전처리

In [86]:
test_2017 = know_test[0].copy()

### 3_1. aq문항(일반업무활동능력)

#### 3_1_1. 데이터프레임 분할

In [87]:
# aq문항 나누기
test_2017_a = test_2017.loc[:,'idx':'aq41_2']   

In [88]:
test_2017_a['bq1'] = test_2017['bq1']
test_2017_a['bq2'] = test_2017['bq2']

#### 3_1_2. 결측치 확인

- null, ' ' 을 결측치로 간주
- 결측치는 우선 0으로 처리
- 최종적으로 결측치를 산업별, 업체유형별(bq1, bq2) 평균으로 처리 예정

In [89]:
# 데이터 타입을 정수형으로 변환(정수형으로 변환하지 않으면 그룹핑 이후 평균값을 나타낼 때 정수형으로 변환하지 않은 컬럼은 표시되지 않음)
test_2017_a = test_2017_a.astype(int)

#### 3_1_3. 1번 문항 이상치 및 결측치 확인

In [90]:
# 1번 문항은 1~5 입력만 허용

for col in range (1, 42):
    for i in range(len(test_2017_a.index)):
        if (1 <= test_2017_a['aq'+str(col)+'_1'][i] <= 5) == False:
            print('error col :', col)
            print('error index :', i)
            
print('끝')

error col : 32
error index : 7749
끝


In [91]:
test_2017_a['aq32_1'][7749]

33

In [92]:
# 0으로 변환 후 평균 처리 예정
test_2017_a['aq32_1'][7749] = 0 

#### 3_1_4. 2번 문항 이상치 확인

In [93]:
# 2번 문항은 0~7 입력 허용

for col in range (1, 42):
    for i in range(len(test_2017_a.index)):
        if (test_2017_a['aq'+str(col)+'_2'][i] > 7) == True:
            print('error col :', col)
            print('error index :', i)
                
print('끝')

끝


- 이상치 : 허용된 입력값 외의 입력이 된 경우
- 1번 문항 : 1건(33으로 입력)
- 2번 문항 : 0건

#### 3_1_5. 로직 확인 및 결측치 처리

- 1번 문항에 1이라고 응답한 것 중 2번 문항에 0이라고 응답하지 않은 것 확인
- 1번 뭉항에 1이라고 응답하지 않은 것 중 2번 문항에 0이라고 응답하지 않은 것 확인

##### 3_1_5_1. 1번 문항 ==1 & 2번 문항 != 0 확인

In [94]:
not_same_1 = pd.DataFrame()
not_same_idx = []
not_same_column = []

for col in range(1,42):
    for i in range(len(test_2017_a.index)):
        if (test_2017_a['aq'+str(col)+'_1'][i] == 1) ==True & (test_2017_a['aq'+str(col)+'_2'][i] != 0) == True :
            not_same_idx.append(test_2017_a.index[i])
            not_same_column.append(test_2017_a.columns[col])
            
not_same_1['not_same_idx']=not_same_idx
not_same_1['not_same_column']=not_same_column

not_same_1

,not_same_idx,not_same_column


- 1번 문항에 1이라고 응답한 것은 모두 2번 문항에 0이라고 응답함

##### 3_1_5_2. 1번 문항 != 1 & 2번 문항 == 0 확인

In [95]:
not_same_2 = pd.DataFrame()
not_same_idx = []
not_same_column = []

for col in range(1,42):
    for i in range(len(test_2017_a.index)):
        if (test_2017_a['aq'+str(col)+'_1'][i] != 1) ==True & (test_2017_a['aq'+str(col)+'_2'][i] == 0) == True :
            not_same_idx.append(test_2017_a.index[i])
            not_same_column.append(test_2017_a.columns[col])
            
not_same_2['not_same_idx']=not_same_idx
not_same_2['not_same_column']=not_same_column

len(not_same_2)

36

- 로직 부적합 확인
- 1번 문항에 1로 응답한 것 중 2번 문항에 응답한 것 0건
- 1번 문항에 2,3,4,5로 응답한 것 중 2번 문항에 응답하지 않은 것 36건
- 해당 인덱스에 해당하는 산업별, 업체유형별 평균값으로 처리 예정

#### 3_1_6. 결측치 및 이상치 평균값 처리

In [96]:
mean = train_2017.groupby(['bq1', 'bq2']).mean().reset_index()

In [97]:
mean = mean.astype(int)

In [98]:
# 전처리 완료한 train데이터에서 'bq1', 'bq2' 기준으로 결측치 처리 예정
mean

,bq1,bq2,idx,aq1_1,aq1_2,aq2_1,aq2_2,aq3_1,aq3_2,aq4_1,...,bq36,bq37,bq38,bq39_1,bq39_2,bq40,bq41_1,bq41_3,bq41_2,knowcode
0,1,1,4827,2,3,2,2,2,3,2,...,1,54,2,1,2,1,1873,1311,1286,1081992
1,1,2,4750,3,4,3,5,3,4,2,...,1,39,5,1,1,1,4185,0,3000,106559
2,1,4,4493,2,2,2,2,2,2,2,...,1,56,1,1,5,0,281,3137,219,2150860
3,1,6,5405,4,5,3,4,3,4,3,...,1,58,3,2,6,0,0,3000,0,901601
4,2,1,4707,2,3,2,2,3,3,3,...,1,44,2,1,1,1,3707,0,2384,731243
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,20,1,837,3,3,2,2,3,3,1,...,1,46,2,1,2,1,2933,0,1666,661167
89,20,4,5831,2,2,3,3,2,3,1,...,2,57,1,1,5,0,485,1185,457,540515
90,20,6,2849,1,0,1,0,1,0,1,...,2,48,2,1,3,2,1675,0,1287,550201
91,21,1,4693,2,2,3,4,2,2,1,...,2,32,4,1,2,1,3650,0,2500,411201


##### 3_1_6_2. 이상치 및 결측치 평균 처리

In [99]:
# 1번 문항 결측치 처리
for col in range(1,42):
    for i in range(len(test_2017_a.index)):
        if (test_2017_a['aq'+str(col)+'_1'][i] == 0) ==True:
            test_2017_a['aq'+str(col)+'_1'][i] = mean[mean['bq1'] == test_2017_a['bq1'][i]][mean['bq2'] == test_2017_a['bq2'][i]]['aq'+str(col)+'_1']

In [100]:
# 1번 문항에 2,3,4,5 로 응답했는데 2번 문항에 0으로 응답한 케이스 처리
for col in range(1,42):
    for i in range(len(test_2017_a.index)):
        if (test_2017_a['aq'+str(col)+'_1'][i] != 1) & (test_2017_a['aq'+str(col)+'_2'][i] == 0) == True:
            test_2017_a['aq'+str(col)+'_2'][i] = mean[mean['bq1'] == test_2017_a['bq1'][i]][mean['bq2'] == test_2017_a['bq2'][i]]['aq'+str(col)+'_2']

In [101]:
# 결측치 정상 처리 여부 확인
for col in range(1,42):
    for i in range(len(test_2017_a.index)):
        if (test_2017_a['aq'+str(col)+'_1'][i] != 1) ==True & (test_2017_a['aq'+str(col)+'_2'][i] == 0) == True:
            print('error col :', col)
            print('error index :', i)

print('끝')

끝


In [102]:
# 데이터 타입 정수형으로 다시 한번 변환 확인
test_2017_a = test_2017_a.astype(int)

In [103]:
test_2017_a = test_2017_a.drop(['bq1', 'bq2'], axis = 1)

In [104]:
# train_2017_a 전처리 완료
test_2017_a

,idx,aq1_1,aq1_2,aq2_1,aq2_2,aq3_1,aq3_2,aq4_1,aq4_2,aq5_1,...,aq37_1,aq37_2,aq38_1,aq38_2,aq39_1,aq39_2,aq40_1,aq40_2,aq41_1,aq41_2
0,0,3,4,2,2,3,3,1,0,3,...,1,0,2,2,4,5,1,0,1,0
1,1,5,5,3,5,5,5,5,5,4,...,4,4,4,4,4,4,1,0,4,5
2,2,5,5,5,4,5,4,1,0,1,...,4,5,5,4,3,4,2,2,5,5
3,3,4,5,5,6,4,6,3,4,4,...,3,4,4,4,2,3,1,0,1,0
4,4,5,6,4,5,4,5,1,0,1,...,1,0,1,0,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9481,9481,3,4,4,5,4,5,5,6,4,...,1,0,2,2,2,2,1,0,1,0
9482,9482,4,5,4,6,5,6,5,6,5,...,4,5,4,6,4,5,3,5,3,4
9483,9483,3,2,1,0,2,1,3,3,1,...,1,0,1,0,1,0,1,0,1,0
9484,9484,4,5,3,4,3,4,1,0,1,...,1,0,3,4,1,0,1,0,1,0


### 3_2. bq문항(인구통계학적 변인)

#### 3_2_1. 데이터프레임 분할

In [105]:
test_2017_b = test_2017.loc[:,'bq1':]

#### 3_2_2. 결측치 확인

In [106]:
test_2017_b.isin(['없다']).sum().value_counts()

0       62
6561     2
16       1
10       1
480      1
5565     1
7751     1
2        1
dtype: int64

In [107]:
test_2017_b.isin(['없음']).sum().value_counts()

0      65
292     1
15      1
235     1
241     1
339     1
dtype: int64

In [108]:
test_2017_b.isin(['무']).sum().value_counts()

0    70
dtype: int64

In [109]:
# 빈칸(' '), '없음', '없다', '무'  0으로 처리
test_2017_b.replace({' ':0, '없다':0, '없음':0, '무':0}, inplace = True)

#### 3_2_3. 데이터 타입 변환

In [110]:
# 데이터 타입 변환을 위한 처리
# 메타데이터 중 변수정보 파일을 참고
# 텍스트 데이터 --> object, 숫자형 데이터 --> int로 변환.
# 숫자형 데이터 중 명목형자료 들에 대해선 추후 재처리

In [111]:
text_columns = ['bq4_1a','bq5_2','bq19_1','bq30','bq31','bq32','bq33','bq34','bq38_1']

In [112]:
text_columns = pd.Series(text_columns)

In [113]:
test_2017_b.loc[:,text_columns] = test_2017_b.loc[:,text_columns].astype(str)

In [114]:
for col in range(len(test_2017_b.columns)):
    if (text_columns.isin([test_2017_b.columns[col]]).sum() == 1 ) == False:
        test_2017_b[test_2017_b.columns[col]] = test_2017_b[test_2017_b.columns[col]].astype(int)

In [115]:
# 데이터 타입 변환 완료
test_2017_b.info(max_cols=10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9486 entries, 0 to 9485
Columns: 70 entries, bq1 to bq41_3
dtypes: int64(61), object(9)
memory usage: 5.1+ MB


#### 2_2_4. 로직 적합 여부 확인

- 4번 1응답시 4_1 != 0, 2응답시 4_1 == 0
- 5번 1응답시 5_1, 5_2 != 0, 2응답시 5_1, 5_2 == 0
- 38번 1 응답시 38_1 == 0, 2~6응답시 38_1 != 0
- 39번 1,2,3,4 응답시 40, 41_1, 41_2 != 0,  41_3 == 0
- 39번 5,6,7응답시 40, 41_1, 41_2 == 0, 41_3 != 0

In [116]:
for col in range(len(test_2017_b.columns)):
    if (test_2017_b.iloc[:,col].dtype == int)==True:
        print(test_2017_b.columns[col]+'입력값 :' , np.sort(test_2017_b.iloc[:,col].unique()))
    else:
        print(test_2017_b.columns[col]+'입력값 :',test_2017_b.iloc[:,col].unique())

bq1입력값 : [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21]
bq2입력값 : [1 2 3 4 5 6]
bq3입력값 : [1 2 3 4 5 6]
bq4입력값 : [1 2]
bq4_1a입력값 : ['0' '품질관리기사' '보육교사자격증' ... '기중기 운전' '굴삭기 운전' 'CCIE']
bq5입력값 : [1 2]
bq5_1입력값 : [0 1 2 3 4 5 6]
bq5_2입력값 : ['0' '실무교육' '기획력, 현장경험 등' ... '안전 직무교육' '도면의 이해,기본서류작성(PPT,EXCEL)'
 '장비사용법과 간단한정비방법']
bq6입력값 : [1 2 3 4 5 6 7]
bq7입력값 : [1 2 3 4 5 6 7]
bq8_1입력값 : [1 2 3 4 5]
bq8_2입력값 : [1 2 3 4 5]
bq8_3입력값 : [1 2 3 4 5]
bq9입력값 : [1 2 3 4 5]
bq10입력값 : [1 2 3 4 5]
bq11입력값 : [1 2 3 4 5]
bq12_1입력값 : [1 2 3 4 5]
bq12_2입력값 : [0 1 2 3 4 5]
bq12_3입력값 : [0 1 2 3 4 5]
bq12_4입력값 : [0 1 2 3 4 5]
bq12_5입력값 : [1 2 3 4 5]
bq13입력값 : [1 2 3 4 5]
bq14입력값 : [1 2 3 4 5]
bq15_1입력값 : [1 2 3 4 5]
bq15_2입력값 : [1 2 3 4 5]
bq15_3입력값 : [1 2 3 4 5]
bq16입력값 : [1 2 3 4 5]
bq17입력값 : [1 2 3 4 5]
bq18_1입력값 : [1 2 3 4 5]
bq18_2입력값 : [1 2 3 4 5]
bq18_3입력값 : [1 2 3 4 5]
bq18_4입력값 : [1 2 3 4 5]
bq18_5입력값 : [1 2 3 4 5]
bq18_6입력값 : [1 2 3 4 5]
bq18_7입력값 : [1 2 3 4 5]
bq19입력값 : [1 2 3 4 5]


##### 3_2_4_1. bq4 확인 및 처리

In [117]:
# 1(있다)라고 응답했을 때, 뒷문항에 응답하지 않은 경우 확인
not_same_4_1 = pd.DataFrame()
not_same_idx = []

for i in range(len(test_2017_b.index)):
    if (test_2017_b['bq4'][i] == 1) ==True:
        if ((test_2017_b['bq4_1a'][i] == '0') == True):
            not_same_idx.append(i)
            
not_same_4_1['not_same_idx']=not_same_idx

not_same_4_1

,not_same_idx


In [118]:
# 2(없다)라고 응답했을 때, 뒷문항에 응답한 경우 확인
not_same_4_2 = pd.DataFrame()
not_same_idx = []

for i in range(len(test_2017_b.index)):
    if (test_2017_b['bq4'][i] == 2) ==True:
        if ((test_2017_b['bq4_1a'][i] != '0') == True):
            not_same_idx.append(i)
            
not_same_4_2['not_same_idx']=not_same_idx

not_same_4_2

,not_same_idx


##### 3_2_4_2. bq5 확인 및 처리

In [119]:
# 1(있다)라고 응답했을 때, 뒷문항에 응답하지 않은 경우 확인
not_same_5_1 = pd.DataFrame()
not_same_idx = []

for i in range(len(test_2017_b.index)):
    if (test_2017_b['bq5'][i] == 1):
        if ((test_2017_b['bq5_1'][i] == 0) == True) and ((test_2017_b['bq5_2'][i] == '0') == True):
            not_same_idx.append(i)
            
not_same_5_1['not_same_idx']=not_same_idx

not_same_5_1

,not_same_idx


In [120]:
# 2(없다)라고 응답했을 때, 뒷문항에 응답한 경우 확인
not_same_5_2 = pd.DataFrame()
not_same_idx = []

for i in range(len(test_2017_b.index)):
    if (test_2017_b['bq5'][i] == 2) ==True:
        if ((test_2017_b['bq5_1'][i] != 0) == True) or ((test_2017_b['bq5_2'][i] != '0') == True):
            not_same_idx.append(i)
            
not_same_5_2['not_same_idx']=not_same_idx

not_same_5_2

,not_same_idx


##### 3_2_4_5. bq38 확인 및 처리

In [121]:
# 옳은로직
# 1응답시 --> bq38_1 응답X 
# 2,3,4,5,6응답시 --> bq38_1 응답O

In [122]:
# 2,3,4,5,6응답시 bq38_1에 응답하지 않은 경우

for i in range(len(test_2017_b.index)):
    if (test_2017_b['bq38'][i] > 1) == True:
        if (test_2017_b['bq38_1'][i] == 0)==True:
            print('error index: ', i)
            
print('끝')

끝


In [123]:
# 1응답시 bq38_1에 응답한 경우


for i in range(len(test_2017_b.index)):
    if (test_2017_b['bq38'][i] == 1) == True:
        if (test_2017_b['bq38_1'][i] != '0')==True:
            print('error index: ', i)
            
print('끝')

끝


##### 3_2_4_6. bq39 확인 및 처리

In [124]:
# 옳은로직
# 39_1에 1 응답시 --> 39_2는 1,2,3,4
# 39_2에 2 응답시 --> 39_2는 5,6,7

# 1,2,3,4 응답시 --> bq40, bq41_1, bq41_2 응답O, bq41_3 응답X
# 5,6,7 응답시 --> bq40, bq41_1, bq41_2 응답X, bq41_3 응답O

In [125]:
# 39_1에 1 응답시 39_2에 5,6,7 응답한 경우
for i in range(len(test_2017_b.index)):
    if (test_2017_b['bq39_1'][i] == 1) == True:
        if (test_2017_b['bq39_2'][i] >= 5 ) == True:
            print('error index :', i)
            
print('끝')

끝


In [126]:
# 39_1에 2 응답시 39_2에 1,2,3,4 응답한 경우
for i in range(len(test_2017_b.index)):
    if (test_2017_b['bq39_1'][i] == 2) == True:
        if (test_2017_b['bq39_2'][i] < 5 ) == True:
            print('error index :', i)
            
print('끝')

끝


In [127]:
# 39_1에 1응답시 bq40, bq41_1, bq41_2에 응답하지 않은 경우
error_bq40_1=[]
error_bq41_1_1=[]
error_bq41_2_1=[]

for i in range(len(test_2017_b.index)):
    if (test_2017_b['bq39_1'][i] == 1) == True:
        if ((test_2017_b['bq40'][i] ==0)==True):
            error_bq40_1.append(i)
            
for i in range(len(test_2017_b.index)):
    if (test_2017_b['bq39_1'][i] == 1) == True:
        if ((test_2017_b['bq41_1'][i] ==0)==True):
            error_bq41_1_1.append(i)
            
for i in range(len(test_2017_b.index)):
    if (test_2017_b['bq39_1'][i] == 1) == True:
        if ((test_2017_b['bq41_2'][i] ==0)==True):
            error_bq41_2_1.append(i)
            
print('bq40_1 오류 개수 :', len(error_bq40_1))
print('bq41_1_1 오류 개수 :', len(error_bq41_1_1))
print('bq41_2_1 오류 개수 :', len(error_bq41_2_1))

bq40_1 오류 개수 : 0
bq41_1_1 오류 개수 : 38
bq41_2_1 오류 개수 : 8153


In [128]:
# 39_1에 1 응답시 bq41_3에 응답한 경우
error_bq41_3_1 = []

for i in range(len(test_2017_b.index)):
    if (test_2017_b['bq39_1'][i] == 1) == True:
        if (test_2017_b['bq41_3'][i] != 0) == True:
            error_bq41_3_1.append(i)
            
print('bq41_3_1 오류 개수 :', len(error_bq41_3_1))

bq41_3_1 오류 개수 : 8144


In [129]:
# 39_1에 2응답시 bq40, bq41_1, bq41_2에 응답한 경우
error_bq40_2=[]
error_bq41_1_2=[]
error_bq41_2_2=[]

for i in range(len(test_2017_b.index)):
    if (test_2017_b['bq39_1'][i] == 2) == True:
        if ((test_2017_b['bq40'][i] !=0)==True):
            error_bq40_2.append(i)
            
for i in range(len(test_2017_b.index)):
    if (test_2017_b['bq39_1'][i] == 2) == True:
        if ((test_2017_b['bq41_1'][i] !=0)==True):
            error_bq41_1_2.append(i)
            
for i in range(len(test_2017_b.index)):
    if (test_2017_b['bq39_1'][i] == 2) == True:
        if ((test_2017_b['bq41_2'][i] !=0)==True):
            error_bq41_2_2.append(i)
            
print('bq40_2 오류 개수 :', len(error_bq40_2))
print('bq41_1_2 오류 개수 :', len(error_bq41_1_2))
print('bq41_2_2 오류 개수 :', len(error_bq41_2_2))

bq40_2 오류 개수 : 0
bq41_1_2 오류 개수 : 0
bq41_2_2 오류 개수 : 1295


In [130]:
# 39_1에 2 응답시 bq41_3에 응답하지 않은 경우
error_bq41_3_2 = []

for i in range(len(test_2017_b.index)):
    if (test_2017_b['bq39_1'][i] == 2) == True:
        if (test_2017_b['bq41_3'][i] == 0) == True:
            error_bq41_3_2.append(i)
            
print('bq41_3_2 오류 개수 :', len(error_bq41_3_2))

bq41_3_2 오류 개수 : 1333


##### 3_2_4_6_1. bq41 잘못입력된 값들 조정

In [131]:
# bq41_2와 bq41_3 값 변경
test_2017_b.rename(columns = {'bq41_2' : 'bq41_3', 'bq41_3':'bq41_2'}, inplace = True)

##### 3_2_4_6_2. bq41 값 위치 조정 후 재확인

In [132]:
# 39_1에 1응답시 bq40, bq41_1, bq41_2에 응답하지 않은 경우
error_bq40_1=[]
error_bq41_1_1=[]
error_bq41_2_1=[]

for i in range(len(test_2017_b.index)):
    if (test_2017_b['bq39_1'][i] == 1) == True:
        if ((test_2017_b['bq40'][i] ==0)==True):
            error_bq40_1.append(i)
            
for i in range(len(test_2017_b.index)):
    if (test_2017_b['bq39_1'][i] == 1) == True:
        if ((test_2017_b['bq41_1'][i] ==0)==True):
            error_bq41_1_1.append(i)
            
for i in range(len(test_2017_b.index)):
    if (test_2017_b['bq39_1'][i] == 1) == True:
        if ((test_2017_b['bq41_2'][i] ==0)==True):
            error_bq41_2_1.append(i)
            
print('bq40_1 오류 개수 :', len(error_bq40_1))
print('bq41_1_1 오류 개수 :', len(error_bq41_1_1))
print('bq41_2_1 오류 개수 :', len(error_bq41_2_1))

bq40_1 오류 개수 : 0
bq41_1_1 오류 개수 : 38
bq41_2_1 오류 개수 : 9


In [133]:
# 39_1에 1 응답시 bq41_3에 응답한 경우
error_bq41_3_1 = []

for i in range(len(test_2017_b.index)):
    if (test_2017_b['bq39_1'][i] == 1) == True:
        if (test_2017_b['bq41_3'][i] != 0) == True:
            error_bq41_3_1.append(i)
            
print('bq41_3_1 오류 개수 :', len(error_bq41_3_1))

bq41_3_1 오류 개수 : 0


In [134]:
# 39_1에 2응답시 bq40, bq41_1, bq41_2에 응답한 경우
error_bq40_2=[]
error_bq41_1_2=[]
error_bq41_2_2=[]

for i in range(len(test_2017_b.index)):
    if (test_2017_b['bq39_1'][i] == 2) == True:
        if ((test_2017_b['bq40'][i] !=0)==True):
            error_bq40_2.append(i)
            
for i in range(len(test_2017_b.index)):
    if (test_2017_b['bq39_1'][i] == 2) == True:
        if ((test_2017_b['bq41_1'][i] !=0)==True):
            error_bq41_1_2.append(i)
            
for i in range(len(test_2017_b.index)):
    if (test_2017_b['bq39_1'][i] == 2) == True:
        if ((test_2017_b['bq41_2'][i] !=0)==True):
            error_bq41_2_2.append(i)
            
print('bq40_2 오류 개수 :', len(error_bq40_2))
print('bq41_1_2 오류 개수 :', len(error_bq41_1_2))
print('bq41_2_2 오류 개수 :', len(error_bq41_2_2))

bq40_2 오류 개수 : 0
bq41_1_2 오류 개수 : 0
bq41_2_2 오류 개수 : 0


In [135]:
# 39_1에 2 응답시 bq41_3에 응답하지 않은 경우
error_bq41_3_2 = []

for i in range(len(test_2017_b.index)):
    if (test_2017_b['bq39_1'][i] == 2) == True:
        if (test_2017_b['bq41_3'][i] == 0) == True:
            error_bq41_3_2.append(i)
            
print('bq41_3_2 오류 개수 :', len(error_bq41_3_2))

bq41_3_2 오류 개수 : 38


##### 3_2_4_6_3. 로직에 어긋난 값 평균처리

In [136]:
# bq39_1에 1이라 응답한 것 중 41_1에 응답하지 않은 것 처리
for i in range(len(test_2017_b.index)):
    if (test_2017_b['bq39_1'][i] == 1) == True:
        if ((test_2017_b['bq41_1'][i] ==0)==True):
            test_2017_b['bq41_1'][i] = mean[mean['bq1'] == test_2017_b['bq1'][i]][mean['bq2'] == test_2017_b['bq2'][i]]['bq41_1']

In [137]:
# bq39_1에 1이라 응답한 것 중 bq41_2에 응답하지 않은 것 처리

for i in range(len(test_2017_b.index)):
    if (test_2017_b['bq39_1'][i] == 1) == True:
        if ((test_2017_b['bq41_2'][i] ==0)==True):
            test_2017_b['bq41_2'][i] = mean[mean['bq1'] == test_2017_b['bq1'][i]][mean['bq2'] == test_2017_b['bq2'][i]]['bq41_2']

In [138]:
# bq39_1에 2이라 응답한 것 중 bq41_3에 응답하지 않은 것들 처리

for i in range(len(test_2017_b.index)):
    if (test_2017_b['bq39_1'][i] == 2) == True:
        if (test_2017_b['bq41_3'][i] == 0) == True:
            test_2017_b['bq41_3'][i] = mean[mean['bq1'] == test_2017_b['bq1'][i]][mean['bq2'] == test_2017_b['bq2'][i]]['bq41_3']

In [139]:
# bq39에 대한 오류 처리 재확인

In [140]:
# 39_1에 1응답시 bq40, bq41_1, bq41_2에 응답하지 않은 경우
error_bq40_1=[]
error_bq41_1_1=[]
error_bq41_2_1=[]

for i in range(len(test_2017_b.index)):
    if (test_2017_b['bq39_1'][i] == 1) == True:
        if ((test_2017_b['bq40'][i] ==0)==True):
            error_bq40_1.append(i)
            
for i in range(len(test_2017_b.index)):
    if (test_2017_b['bq39_1'][i] == 1) == True:
        if ((test_2017_b['bq41_1'][i] ==0)==True):
            error_bq41_1_1.append(i)
            
for i in range(len(test_2017_b.index)):
    if (test_2017_b['bq39_1'][i] == 1) == True:
        if ((test_2017_b['bq41_2'][i] ==0)==True):
            error_bq41_2_1.append(i)
            
print('bq40_1 오류 개수 :', len(error_bq40_1))
print('bq41_1_1 오류 개수 :', len(error_bq41_1_1))
print('bq41_2_1 오류 개수 :', len(error_bq41_2_1))
print(error_bq41_2_1)

bq40_1 오류 개수 : 0
bq41_1_1 오류 개수 : 0
bq41_2_1 오류 개수 : 0
[]


In [141]:
# 39_1에 1 응답시 bq41_3에 응답한 경우
error_bq41_3_1 = []

for i in range(len(test_2017_b.index)):
    if (test_2017_b['bq39_1'][i] == 1) == True:
        if (test_2017_b['bq41_3'][i] != 0) == True:
            error_bq41_3_1.append(i)
            
print('bq41_3_1 오류 개수 :', len(error_bq41_3_1))

bq41_3_1 오류 개수 : 0


In [142]:
# 39_1에 2응답시 bq40, bq41_1, bq41_2에 응답한 경우
error_bq40_2=[]
error_bq41_1_2=[]
error_bq41_2_2=[]

for i in range(len(test_2017_b.index)):
    if (test_2017_b['bq39_1'][i] == 2) == True:
        if ((test_2017_b['bq40'][i] !=0)==True):
            error_bq40_2.append(i)
            
for i in range(len(test_2017_b.index)):
    if (test_2017_b['bq39_1'][i] == 2) == True:
        if ((test_2017_b['bq41_1'][i] !=0)==True):
            error_bq41_1_2.append(i)
            
for i in range(len(test_2017_b.index)):
    if (test_2017_b['bq39_1'][i] == 2) == True:
        if ((test_2017_b['bq41_2'][i] !=0)==True):
            error_bq41_2_2.append(i)
            
print('bq40_2 오류 개수 :', len(error_bq40_2))
print('bq41_1_2 오류 개수 :', len(error_bq41_1_2))
print('bq41_2_2 오류 개수 :', len(error_bq41_2_2))

bq40_2 오류 개수 : 0
bq41_1_2 오류 개수 : 0
bq41_2_2 오류 개수 : 0


In [143]:
# 39_1에 2 응답시 bq41_3에 응답하지 않은 경우
error_bq41_3_2 = []

for i in range(len(test_2017_b.index)):
    if (test_2017_b['bq39_1'][i] == 2) == True:
        if (test_2017_b['bq41_3'][i] == 0) == True:
            error_bq41_3_2.append(i)
            
print('bq41_3_2 오류 개수 :', len(error_bq41_3_2))
print(error_bq41_3_2)

bq41_3_2 오류 개수 : 0
[]


In [144]:
#df_2017_b 숫자 데이터 결측치 처리 완료
test_2017_b

,bq1,bq2,bq3,bq4,bq4_1a,bq5,bq5_1,bq5_2,bq6,bq7,...,bq36,bq37,bq38,bq38_1,bq39_1,bq39_2,bq40,bq41_1,bq41_3,bq41_2
0,13,2,6,2,0,2,0,0,7,4,...,2,26,3,비서학,1,1,1,3000,0,2300
1,3,1,4,1,품질관리기사,1,1,실무교육,5,4,...,1,57,4,농화학,1,1,1,5500,0,2500
2,10,1,5,2,0,1,6,"기획력, 현장경험 등",5,5,...,1,31,4,신문방송,1,1,1,4300,0,4000
3,3,1,4,2,0,2,0,0,7,6,...,1,35,6,화학,1,1,1,4100,0,3000
4,19,1,2,2,0,2,0,0,7,5,...,1,36,4,광고홍보,1,1,1,2800,0,2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9481,6,4,1,1,굴삭기 운전,2,0,0,6,1,...,1,52,1,0,2,6,0,0,3400,0
9482,3,1,3,2,0,1,1,"도면의 이해,기본서류작성(PPT,EXCEL)",6,6,...,1,48,5,요업과,1,1,1,7000,0,2400
9483,8,4,1,1,건설기계 면허,2,0,0,7,1,...,1,44,2,인문계,2,6,0,0,4500,0
9484,10,1,6,1,CCIE,1,5,직무교육,6,5,...,1,44,4,컴퓨터공학,1,1,1,6000,0,4000


#### 3_2_5. 텍스트 데이터 컬럼 삭제

- 'bq5_2', 'bq19_1','bq30', 'bq31', 'bq32', 'bq33', 'bq34'삭제

In [145]:
test_2017_b.drop(['bq5_2', 'bq19_1','bq30', 'bq31', 'bq32', 'bq33', 'bq34'], axis = 1 ,inplace = True)

#### 3_2_6. 예측을 위한 전처리 데이터 통합

In [146]:
test_2017 = pd.concat([test_2017_a, test_2017_b], axis = 1)

In [147]:
# train 데이터 aq_2번문항 지우기
for col in range(1, 42):
    train_2017 = train_2017.drop(['aq'+str(col)+'_2'], axis = 1)

In [148]:
# test 데이터 aq_2번문항 지우기
for col in range(1, 42):
    test_2017 = test_2017.drop(['aq'+str(col)+'_2'], axis = 1)

In [149]:
processing_train_2017 = train_2017
processing_test_2017 = test_2017

In [150]:
# test 데이터 전처리 끝

## 2.2. 2018년 전처리

### 2.2.1.  2018년 Train 데이터 전처리

In [151]:
train_2018 = know_train[1].copy()

In [152]:
len(train_2018)

9072

- 행 수 9072 

#### 1) 결측치 및 오류 데이터 처리

##### 1-1) NULL 값 처리

In [153]:
processing_train_2018 = train_2018.copy()

In [154]:
null_count_per_columns = processing_train_2018.isnull().sum()
null_count_per_columns

idx         0
cq1         0
cq2         0
cq3         0
cq4         0
           ..
bq40        0
bq41_1      0
bq41_2      0
bq41_3      0
knowcode    0
Length: 139, dtype: int64

- NULL값 : 0으로 대체

In [155]:
processing_train_2018 = processing_train_2018.fillna(0)

In [156]:
processing_train_2018.isnull().sum()

idx         0
cq1         0
cq2         0
cq3         0
cq4         0
           ..
bq40        0
bq41_1      0
bq41_2      0
bq41_3      0
knowcode    0
Length: 139, dtype: int64

In [157]:
processing_train_2018.isnull().sum().sum()

0

##### 1-2) 공백처리 : 0 으로 대체

In [158]:
for col in processing_train_2018.columns:
    for i in range(1,10):
        processing_train_2018[col].replace(' '*i, 0, inplace = True)

- 0이 1개라도 있는 행 전부 출력

In [159]:
for col in processing_train_2018.columns:
    print( col , len(processing_train_2018.index[processing_train_2018[col]==0].tolist()) )

idx 0
cq1 0
cq2 0
cq3 0
cq4 0
cq5 0
cq6 0
cq7 0
cq8 0
cq9 0
cq10 0
cq11 0
cq12 0
cq13 0
cq14 0
cq15 0
cq16 0
cq17 0
cq18 0
cq19 0
cq20 0
cq21 0
cq22 0
cq23 0
cq24 0
cq25 0
cq26 0
cq27 0
cq28 0
cq29 0
cq30 0
cq31 0
cq32 0
cq33 0
cq34 0
cq35 0
cq36 0
cq37 0
cq38 0
cq39 0
cq40 0
cq41 0
cq42 0
cq43 0
cq44 0
cq45 0
cq46 0
cq47 0
cq48 0
cq49 0
cq50_1 0
cq50_2 0
cq50_3 0
cq50_4 0
cq50_5 0
cq50_6 0
cq50_7 0
cq50_8 0
iq1 0
iq2 0
iq3 0
iq4 0
iq5 0
iq6 0
bq1 0
bq2 0
bq3 0
bq4 0
bq4_1a 0
bq5 0
bq5_1 0
bq5_2 0
bq6 0
bq7 0
bq8_1 0
bq8_2 0
bq8_3 0
bq9 0
bq10 0
bq11 0
bq12_1 0
bq12_2 0
bq12_3 0
bq12_4 0
bq12_5 0
bq13 0
bq14 0
bq15 0
bq16 0
bq17 0
bq18 0
bq19 0
bq20 0
bq21 746
bq221 0
bq222 0
bq223 801
bq231 0
bq232 8862
bq233 8775
bq234 0
bq235 0
bq241 0
bq242 0
bq243 0
bq244 0
bq245 0
bq25 0
bq25_1 0
bq26_1 0
bq26_1a 0
bq26_2 0
bq26_2a 0
bq26_3 0
bq26_3a 0
bq26_4 0
bq26_4a 0
bq27 0
bq28 0
bq28_1 0
bq29 0
bq30 0
bq31 0
bq32 0
bq33 0
bq34 0
bq35 0
bq36 0
bq37 0
bq37_1 0
bq38 0
bq38_1 0
bq38_2 0
bq39 0


##### 1-3) "없다" 의미 답변 대체 

In [160]:
# 2018년 칼럼 리스트 
col_list_2018 = processing_train_2018.columns.tolist()

In [161]:
nope_word_list = ['없다','없음','없음.','없음_','없습니다','없습니다.','없어요','없어요~','업음', '.']

In [162]:
# "없다" 라는 의미의 단어를 0으로 대체하는 함수
def nope_word_replace(col):
    for word in nope_word_list:
        col.replace(word, 0, inplace = True)

In [ ]:
col_list = processing_train.columns.tolist()
nope_word_list = ['없다','없음','없음.','없음_','없습니다','없습니다.','없어요','없어요~','업음', '.']
def nope_word_replace(col):
    for word in nope_word_list:
        col.replace(word, 0, inplace = True)

In [163]:
# 문자가 들어가는 문항 칼럼 리스트
obj_col_list_2018 = []

In [164]:
for col in col_list_2018:
    
    # 문자열이 포함된 칼럼에서 "없다" 대체
    if processing_train_2018[col].dtype == 'object':
        nope_word_replace(processing_train_2018[col])
        # object 타입 칼럼 리스트 
        obj_col_list_2018.append(col)

##### 1-4) knowcode 결측 : 없음 

In [165]:
processing_train_2018['knowcode'].value_counts().sort_index()

11101     15
11102     16
11201     15
12101     18
12102     15
          ..
902101    16
902201    15
903101    15
904101    15
904201    15
Name: knowcode, Length: 576, dtype: int64

##### 1-5) 밀려쓴 응답 처리 

- bq4, bq28, bq35 등의 문항을 통해 밀려쓴 답안 확인 

In [166]:
processing_train_2018['bq4'].value_counts()

2             5045
1             3996
0               16
전기기사            10
제강기능사            2
금속재료시험 기능사       1
전기철도기사           1
열처리기능사           1
Name: bq4, dtype: int64

In [167]:
processing_train_2018['bq35'].value_counts()

1.0                   5336
2.0                   2828
1                      556
2                      318
39.0                     3
35.0                     3
31.0                     2
26.0                     2
32.0                     2
40.0                     2
50.0                     2
27.0                     2
37.0                     2
38                       1
33                       1
42                       1
 라이트룸 등의 컴퓨터 프로그램"       1
45                       1
46.0                     1
56.0                     1
47.0                     1
44.0                     1
25.0                     1
42.0                     1
52.0                     1
53.0                     1
30                       1
Name: bq35, dtype: int64

In [168]:
processing_train_2018['bq28'].value_counts()

2                                                     3200
3                                                     2546
4                                                     2074
1                                                      919
5                                                      301
자동화                                                      2
자동화가 가속화 될것                                              1
일이 힘이 들어서                                                1
기계화,자동화,외주화                                              1
차량도입이 증가하고 있기때문                                          1
기술의 발전으로 적은 인원으로 일처리가 가능                                 1
자동화가 되어가기 때문에                                            1
통일이 된다면 비젼이 매우 크기 때문                                     1
수요가 꾸준                                                   1
외주화,자동화                                                  1
자동화설비의 보급 확대로                                            1
배우는 인력이 없어 일거리가 늘어남                                     

- A. 어디서 부터 밀렸는지 모르겠는 행 : 정확성을 위해 제거
- idx : 16179, 18048, 17700, 12555

In [169]:
processing_train_2018 = processing_train_2018[processing_train_2018['idx'] != 16179]
processing_train_2018 = processing_train_2018[processing_train_2018['idx'] != 18048]
processing_train_2018 = processing_train_2018[processing_train_2018['idx'] != 17700]
processing_train_2018 = processing_train_2018[processing_train_2018['idx'] != 12555]

- B. cq1 ~ bq41 밀린 행 : 원래 위치로 복원 후 cq1 은 무응답으로 처리 
- 즉 모든 문항이 밀린 행

In [170]:
forward_1_pushed_list_train_2018 =[]

- bq28 에서 숫자가 아닌 답변을 한 행의 idx 를 추출

In [171]:
for idx in processing_train_2018['idx']:
    
    target_row = processing_train_2018[processing_train_2018['idx'] == idx]
    
    if ~target_row['bq28'].isin([0,1,2,3,4,5, '0','1','2','3','4','5']).any():
        forward_1_pushed_list_train_2018.append(idx)

In [172]:
len(forward_1_pushed_list_train_2018)

31

- C. 밀린 행 원복하기 
- 밀린 행 원복 함수 

In [173]:
# 앞 문항을 뒷 문항으로 덮어쓴 케이스 보정 

## idx             : 밀려쓰기가 발생한 행의 idx 
## pushed_step    : 밀린 칸의 수
## start_col      : 밀려쓰기 시작한 칼럼 
## end_col        : 밀려쓴 마지막 칼럼 

def fix_forward_pushed_answer_2018(idx, pushed_step, start_col, end_col):
    
    target_row = processing_train_2018[processing_train_2018['idx'] == idx]
    
    start_col_index = col_list_2018.index(start_col)
    end_col_index = col_list_2018.index(end_col)

    # 밀려쓴 마지막 칼럼부터, pushed_step 만큼 앞 칼럼 값으로 덮어씌우기
    for i in range(end_col_index, start_col_index, -1):
        target_row.iloc[:,i] = target_row.iloc[:,i - pushed_step]
    
    # 밀린행의 자격증 2, 3
    target_row['bq5'] = 0
    target_row['bq5_1'] = 0
    
    
    
    # 밀려쓴 가장 첫 칼럼 값 : 무응답으로 처리 
    target_row.iloc[:,start_col_index] = 0
    
    # 밀려쓴 행을 수정한 행으로 대체 
    processing_train_2018[processing_train_2018['idx'] == idx] = target_row 

In [174]:
for pushed_idx in forward_1_pushed_list_train_2018:
    fix_forward_pushed_answer_2018(pushed_idx, 1,'cq1','bq41_3' )

##### 1-6) 설문 데이터 형식 오류 

- bq221 ~ bq223, bq231 ~ bq235, bq241 ~ bq245 은 각각 하나의 설문 문항이나, traindata에서는 선택지의 갯수만큼 문항이 개별 칼럼으로 분리되어 있음
- 이 경우 각 칼럼은 예/아니오 로 답변되어야 하지만, 응답 내용이 1~5 등으로 제각각임
- 데이터의 신뢰성을 보장할 수 없으므로 해당 칼럼은 삭제 

In [175]:
processing_train_2018.drop(['bq221','bq222','bq223','bq231','bq232','bq233','bq234','bq235','bq241','bq242','bq243','bq244','bq245'], axis = 1, inplace = True)

#### 2) 데이터 타입 통일

##### 2-1) object -> int

In [176]:
obj_to_int_list_2018 = [
'bq4',
'bq5',
'bq5_1', # 요구훈련기간'
'bq21', # 기본급, 고정급 비중 
'bq25_1',
'bq26_1',    
'bq26_1a',    
'bq26_2'  ,  
'bq26_2a'  , 
'bq26_3'  ,
'bq26_3a' ,  
'bq26_4' ,  
'bq26_4a' ,  
'bq27',
'bq28',  # 향후 일자리 변화 
'bq34', # 근로시간 단축 가능 
'bq36',  # 나이 
'bq37',
'bq39',
'bq40',
"bq25",
"bq37",
"bq38",
"bq38_1",
"bq38_2",
"bq41_1",
"bq41_2",
"bq41_3"
]

In [177]:
for column in obj_to_int_list_2018:
    processing_train_2018[column]= pd.to_numeric(processing_train_2018[column], downcast='integer')
    print('완료 : ' , column )

완료 :  bq4
완료 :  bq5
완료 :  bq5_1
완료 :  bq21
완료 :  bq25_1
완료 :  bq26_1
완료 :  bq26_1a
완료 :  bq26_2
완료 :  bq26_2a
완료 :  bq26_3
완료 :  bq26_3a
완료 :  bq26_4
완료 :  bq26_4a
완료 :  bq27
완료 :  bq28
완료 :  bq34
완료 :  bq36
완료 :  bq37
완료 :  bq39
완료 :  bq40
완료 :  bq25
완료 :  bq37
완료 :  bq38
완료 :  bq38_1
완료 :  bq38_2
완료 :  bq41_1
완료 :  bq41_2
완료 :  bq41_3


##### 2-2) float -> int

In [178]:
float_to_int_list_2018 = [
'bq27' , # 플랫폼 근무자 여부 
'bq7',  # 교육수준    
"bq13", #	업무내용 규정성
"bq14", #	제 규정 적용 여부
"bq15", #	지휘감독여부
"bq16", #	거부 가능성
"bq17", #	근무시간장소
"bq18", #	작업도구 소유여부
"bq19", #	업무 대체성
"cq48",  #   재택근무 가능여부 
"cq18",
"cq19",
"cq20",
"cq21",
"cq22",
"cq23",
"cq24",
"cq25",
"cq26",
"cq27",
"cq28",
"cq29",
"cq30",
"cq31",
"cq32",
"cq33",
"cq34",
"cq35",
"cq36",
"cq37",
"cq38",
"cq39",
"cq40",
"cq41",
"cq42",
"cq43",
"cq44",
"cq45",
"cq46",
"cq47",
"cq48",
"cq49",
"cq50_1",
"cq50_2",
"cq50_3",
"cq50_4",
"cq50_5",
"cq50_6",
"cq50_7",
"cq50_8",
"iq1",
"iq2",
"iq3",
"iq4",
"iq5",
"iq6",
"bq3",
"bq6",
"bq8_1",
"bq8_2",
"bq8_3",
"bq9",
"bq10",
"bq11",
"bq12_1",
"bq12_5",
"bq20" 
]

In [179]:
for column in float_to_int_list_2018:
    # 정수형으로 먼저 변환  
    processing_train_2018[column]= pd.to_numeric(processing_train_2018[column], downcast='integer')
    print('완료 : ' , column )

완료 :  bq27
완료 :  bq7
완료 :  bq13
완료 :  bq14
완료 :  bq15
완료 :  bq16
완료 :  bq17
완료 :  bq18
완료 :  bq19
완료 :  cq48
완료 :  cq18
완료 :  cq19
완료 :  cq20
완료 :  cq21
완료 :  cq22
완료 :  cq23
완료 :  cq24
완료 :  cq25
완료 :  cq26
완료 :  cq27
완료 :  cq28
완료 :  cq29
완료 :  cq30
완료 :  cq31
완료 :  cq32
완료 :  cq33
완료 :  cq34
완료 :  cq35
완료 :  cq36
완료 :  cq37
완료 :  cq38
완료 :  cq39
완료 :  cq40
완료 :  cq41
완료 :  cq42
완료 :  cq43
완료 :  cq44
완료 :  cq45
완료 :  cq46
완료 :  cq47
완료 :  cq48
완료 :  cq49
완료 :  cq50_1
완료 :  cq50_2
완료 :  cq50_3
완료 :  cq50_4
완료 :  cq50_5
완료 :  cq50_6
완료 :  cq50_7
완료 :  cq50_8
완료 :  iq1
완료 :  iq2
완료 :  iq3
완료 :  iq4
완료 :  iq5
완료 :  iq6
완료 :  bq3
완료 :  bq6
완료 :  bq8_1
완료 :  bq8_2
완료 :  bq8_3
완료 :  bq9
완료 :  bq10
완료 :  bq11
완료 :  bq12_1
완료 :  bq12_5
완료 :  bq20


- 확인

In [180]:

processing_train_2018.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9068 entries, 0 to 9071
Data columns (total 126 columns):
 #    Column    Dtype  
---   ------    -----  
 0    idx       int64  
 1    cq1       int64  
 2    cq2       int64  
 3    cq3       int64  
 4    cq4       int64  
 5    cq5       int64  
 6    cq6       int64  
 7    cq7       int64  
 8    cq8       int64  
 9    cq9       int64  
 10   cq10      int64  
 11   cq11      int64  
 12   cq12      int64  
 13   cq13      int64  
 14   cq14      int64  
 15   cq15      int64  
 16   cq16      int64  
 17   cq17      int64  
 18   cq18      int8   
 19   cq19      int8   
 20   cq20      int8   
 21   cq21      int8   
 22   cq22      int8   
 23   cq23      int8   
 24   cq24      int8   
 25   cq25      int8   
 26   cq26      int8   
 27   cq27      int8   
 28   cq28      int8   
 29   cq29      int8   
 30   cq30      int8   
 31   cq31      int8   
 32   cq32      int8   
 33   cq33      int8   
 34   cq34      int8   
 35 

#### 3) 객관식 문항 정상 답변 여부

###### 3-1) 답안 범위가 1~5 인 답변

In [181]:
one_to_five_question_list_2018  = [
"cq1",
"cq2",
"cq3",
"cq4",
"cq5",
"cq6",
"cq7",
"cq8",
"cq9",
"cq10",
"cq11",
"cq12",
"cq13",
"cq14",
"cq15",
"cq16",
"cq17",
"cq18",
"cq19",
"cq20",
"cq21",
"cq22",
"cq23",
"cq24",
"cq25",
"cq26",
"cq27",
"cq28",
"cq29",
"cq30",
"cq31",
"cq32",
"cq33",
"cq34",
"cq35",
"cq36",
"cq37",
"cq38",
"cq39",
"cq40",
"cq41",
"cq42",
"cq43",
"cq44",
"cq45",
"cq46",
"cq49",
"iq1",
"iq2",
"iq3",
"iq4",
"iq5",
"iq6",
"bq8_1",
"bq8_2",
"bq8_3",
"bq9",
"bq10",
"bq11",
"bq12_1",
"bq12_5",
"bq20",
"bq28",    
"bq34"]

In [182]:
wrong_1_5_2018_list = []

for col in one_to_five_question_list_2018:
    
    if ~processing_train_2018[col].isin([0,1,2,3,4,5]).any():
        wrong_1_5_2018_list.append(col)
        
print(len(wrong_1_5_2018_list))

0


- 모든 문항 정상 답변

###### 3-2) 답안 범위가 1~3 인 답변

In [183]:
one_to_three_question_list_2018  = [
"cq47",
"cq50_1",
"cq50_2",
"cq50_3",
"cq50_4",
"cq50_5",
"cq50_6",
"cq50_7",
"cq50_8",
"bq13",
"bq17",
"bq18",
"bq19",
"bq25",
"bq26_1",
"bq26_1a",
"bq26_2",
"bq26_2a",
"bq26_3",
"bq26_3a",
"bq26_4",
"bq26_4a"
]

In [184]:
wrong_1_3_2018_list = []

for col in one_to_three_question_list_2018:
    
    # float형 칼럼과 object 형 칼럼이 모두 존재
    if ~processing_train_2018[col].isin([0,1,2,3, '1','2','3']).any():
        wrong_1_3_2018_list.append(col)
        
print(len(wrong_1_3_2018_list))

0


- 모든 문항 정상 답변 

###### 3-3) 답안 범위가 1~2 인 답변

In [185]:
one_to_two_question_list_2018 = [
"cq48",
"bq4",
"bq5",
"bq14",
"bq16",
"bq25_1",
"bq27",
"bq35",
"bq38",
"bq39"]

In [186]:
wrong_1_2_2018_list = []

for col in one_to_two_question_list_2018:
    
    # float형 칼럼과 object 형 칼럼이 모두 존재
    if ~processing_train_2018[col].isin([0,1,2, '1','2']).any():
        wrong_1_2_2018_list.append(col)
        
print(len(wrong_1_2_2018_list))

0


- 모든 문항 정상 답변

###### 3-4) 답안 범위가 1~6 인 답변

In [187]:
one_to_six_question_list_2018 = [
"bq2",
"bq3",
"bq5_1",
"bq37"]

In [188]:
wrong_1_6_2018_list = []

for col in one_to_six_question_list_2018:
    
    # float형 칼럼과 object 형 칼럼이 모두 존재
    if ~processing_train_2018[col].isin([0,1,2,3,4,5,6, '1','2','3','4','5','6']).any():
        wrong_1_6_2018_list.append(col)
        
print(len(wrong_1_6_2018_list))

0


- 모든 문항 정상 답변 

###### 3-5) 답안 범위가 1~5, 9 인 답변

In [189]:
one_to_five_nine_question_list_2018 = [
"bq12_2",
"bq12_3",
"bq12_4"
]

In [190]:
wrong_1_5_9_2018_list = []

for col in one_to_five_nine_question_list_2018:
    
    # float형 칼럼과 object 형 칼럼이 모두 존재
    if ~processing_train_2018[col].isin([0,1,2,3,4,5,9, '1','2','3','4','5','9']).any():
        wrong_1_5_9_2018_list.append(col)
        
print(len(wrong_1_5_9_2018_list))

0


- 모든 문항 정상 답변

###### 3-6) 세트 문항 합쳐서 평균으로 제시 

In [191]:
processing_train_2018['bq8_2']

0       4
1       4
2       2
3       5
4       4
       ..
9067    4
9068    3
9069    5
9070    4
9071    4
Name: bq8_2, Length: 9068, dtype: int8

In [192]:
processing_train_2018['bq8'] = round((processing_train_2018['bq8_1']+processing_train_2018['bq8_2']+processing_train_2018['bq8_3'])/3)
processing_train_2018 = processing_train_2018.drop(['bq8_1','bq8_2','bq8_3'],axis=1)
processing_train_2018 = processing_train_2018.drop(['bq12_1','bq12_2','bq12_3','bq12_4'],axis = 1)

##### 4) 비상식적 답변 확인 

- 산업 유형 & 근무업체 유형 별 각 문항의 평균 표시
- 비정상 답변 대체 용

In [193]:
mean_by_IW_2018 = processing_train_2018.groupby(['bq1','bq2']).mean().reset_index()
mean_by_IW_2018

,bq1,bq2,idx,cq1,cq2,cq3,cq4,cq5,cq6,cq7,...,bq38,bq38_1,bq38_2,bq39,bq40,bq41_1,bq41_2,bq41_3,knowcode,bq8
0,1.0,1.0,13055.769231,4.384615,1.961538,3.230769,1.576923,1.923077,3.576923,3.269231,...,1.346154,0.769231,2.076923,0.730769,46.076923,2534.615385,1315.384615,1353.846154,902227.923077,2.576923
1,1.0,2.0,14841.833333,4.666667,2.166667,4.000000,4.166667,4.333333,4.166667,4.000000,...,1.000000,1.333333,0.000000,1.166667,43.333333,5800.000000,3916.666667,0.000000,122203.000000,3.666667
2,1.0,3.0,12747.714286,4.857143,3.142857,4.857143,4.428571,4.000000,4.714286,4.285714,...,1.000000,1.000000,0.000000,1.000000,58.285714,5285.714286,3714.285714,0.000000,903101.000000,3.000000
3,1.0,4.0,13536.637168,3.752212,1.300885,3.327434,1.398230,1.601770,3.212389,2.734513,...,1.955752,0.044248,5.584071,0.044248,47.973451,118.584071,88.495575,3177.168142,902204.539823,2.796460
4,1.0,5.0,14995.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,...,1.000000,3.000000,0.000000,1.000000,45.000000,3000.000000,2500.000000,0.000000,122101.000000,4.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,19.0,6.0,12961.000000,4.166667,2.666667,4.000000,3.833333,3.500000,4.000000,4.000000,...,1.500000,0.500000,3.000000,0.833333,45.666667,1533.333333,933.333333,1700.000000,348168.500000,3.833333
90,20.0,1.0,14029.200000,4.400000,1.400000,4.000000,1.800000,1.600000,3.000000,2.600000,...,1.600000,1.000000,3.600000,0.600000,36.600000,1260.000000,780.000000,1200.000000,454161.600000,1.800000
91,20.0,4.0,13605.823529,3.411765,2.088235,3.470588,2.941176,2.735294,3.294118,3.147059,...,1.323529,2.617647,1.941176,1.323529,37.970588,2620.588235,1104.411765,926.470588,413041.411765,2.882353
92,20.0,6.0,14424.916667,3.250000,1.333333,3.333333,1.166667,1.000000,2.083333,1.166667,...,2.000000,0.000000,6.000000,0.000000,33.000000,0.000000,0.000000,1830.000000,549059.416667,2.250000


- 산업 유형 & 근무업체 유형 & 임금/비임금  별 각 문항의 평균 표시

In [194]:
mean_by_IWS_2018 = processing_train_2018.groupby(['bq1','bq2','bq38']).mean().reset_index()
mean_by_IWS_2018

,bq1,bq2,bq38,idx,cq1,cq2,cq3,cq4,cq5,cq6,...,bq37,bq38_1,bq38_2,bq39,bq40,bq41_1,bq41_2,bq41_3,knowcode,bq8
0,1.0,1.0,1,13221.117647,4.470588,2.058824,3.352941,1.470588,1.882353,3.764706,...,2.411765,1.176471,0.0,1.117647,48.058824,3876.470588,2011.764706,129.411765,902518.647059,2.352941
1,1.0,1.0,2,12743.444444,4.222222,1.777778,3.000000,1.777778,2.000000,3.222222,...,2.222222,0.000000,6.0,0.000000,42.333333,0.000000,0.000000,3666.666667,901678.777778,3.000000
2,1.0,2.0,1,14841.833333,4.666667,2.166667,4.000000,4.166667,4.333333,4.166667,...,4.833333,1.333333,0.0,1.166667,43.333333,5800.000000,3916.666667,0.000000,122203.000000,3.666667
3,1.0,3.0,1,12747.714286,4.857143,3.142857,4.857143,4.428571,4.000000,4.714286,...,4.000000,1.000000,0.0,1.000000,58.285714,5285.714286,3714.285714,0.000000,903101.000000,3.000000
4,1.0,4.0,1,13866.200000,4.800000,1.600000,4.600000,2.200000,1.800000,4.600000,...,3.000000,1.000000,0.0,1.000000,50.400000,2680.000000,2000.000000,0.000000,901781.000000,3.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,20.0,1.0,2,14131.333333,4.666667,1.333333,4.333333,1.333333,1.000000,2.333333,...,1.333333,0.000000,6.0,0.000000,31.666667,0.000000,0.000000,2000.000000,561601.000000,2.000000
135,20.0,4.0,1,13925.043478,3.608696,2.521739,3.826087,3.478261,3.260870,3.304348,...,3.913043,3.869565,0.0,1.956522,42.217391,3873.913043,1632.608696,0.000000,412221.434783,3.130435
136,20.0,4.0,2,12938.363636,3.000000,1.181818,2.727273,1.818182,1.636364,3.272727,...,3.454545,0.000000,6.0,0.000000,29.090909,0.000000,0.000000,2863.636364,414755.909091,2.363636
137,20.0,6.0,2,14424.916667,3.250000,1.333333,3.333333,1.166667,1.000000,2.083333,...,2.083333,0.000000,6.0,0.000000,33.000000,0.000000,0.000000,1830.000000,549059.416667,2.250000


###### 4-1) 연령 답변 확인 

- 나이가 음수, 혹은 비상식적인 값이 있는가? 

In [195]:
processing_train_2018['bq36'].value_counts().sort_index()

17     1
19     6
20    11
21    22
22    32
      ..
80     2
81     1
82     1
83     2
85     1
Name: bq36, Length: 67, dtype: int64

- 10세 미만 존재한다면 --> 오기입으로 보고 산업 & 근무업체 유형별 평균으로 대체

In [196]:
too_young_list_train_2018 = list(processing_train_2018[processing_train_2018['bq36'] < 10]['idx'])
too_young_list_train_2018 

[]

In [197]:
for fake_young_idx in too_young_list_train_2018:

    target_bq1 = processing_train_2018[processing_train_2018['idx']==fake_young_idx]['bq1'].values[0]
    target_bq2 = processing_train_2018[processing_train_2018['idx']==fake_young_idx]['bq2'].values[0]
    
    processing_train_2018.loc[processing_train_2018[processing_train_2018['idx'] == fake_young_idx].index, ['bq36']] = mean_by_IW_2018[(mean_by_IW_2018['bq1'] == target_bq1) & (mean_by_IW_2018['bq2'] == target_bq2)]['bq36']                                      

###### 4-2) 주당 근로시간 답변 확인 

- 주당 근로시간이 음수, 혹은 터무니없이 높은 값이 있는가? 

In [198]:
processing_train_2018['bq40'].value_counts().sort_index()

0      1
3      1
5      1
6      2
7      1
      ..
78     2
80    11
84     9
90    11
98     1
Name: bq40, Length: 73, dtype: int64

- 1주일 : 168 시간이므로 168시간을 초과하는 답변 -> 산업 & 근무업체 유형별 평균으로 대체

In [199]:
too_busy_list_train_2018 = list(processing_train_2018[processing_train_2018['bq40'] > 168]['idx'])
too_busy_list_train_2018 

[]

In [200]:
for fake_busy_idx in too_busy_list_train_2018:

    target_bq1 = processing_train_2018[processing_train_2018['idx']==fake_busy_idx]['bq1'].values[0]
    target_bq2 = processing_train_2018[processing_train_2018['idx']==fake_busy_idx]['bq2'].values[0]
    
    processing_train_2018.loc[processing_train_2018[processing_train_2018['idx'] == fake_busy_idx].index, ['bq40']] = mean_by_IW_2018[(mean_by_IW_2018['bq1'] == target_bq1) & (mean_by_IW_2018['bq2'] == target_bq2)]['bq40']                                      

###### 4-3) 근로소득 답변 확인 

- 근로소득이 음수, 혹은 터무니없이 높은 값이 있는가? 

In [201]:
processing_train_2018['bq41_1'].value_counts().sort_index()

0        1268
300         1
400         1
500         1
600         7
         ... 
30000       5
35000       1
40000       2
45000       1
60000       1
Name: bq41_1, Length: 273, dtype: int64

In [202]:
processing_train_2018['bq41_2'].value_counts().sort_index()

0        1355
100         2
135         1
150         1
200         2
         ... 
13500       1
14052      15
15000       2
17000       1
30000       1
Name: bq41_2, Length: 182, dtype: int64

In [203]:
processing_train_2018['bq41_3'].value_counts().sort_index()

0        7820
140         1
150         1
200         5
300         2
         ... 
22000       1
24000       1
25000       1
26000       1
40000       1
Name: bq41_3, Length: 94, dtype: int64

- 이상없음

##### 5) 설문지 로직 체크

###### 5-1) bq41 로직 체크

In [204]:
#2018년 bq41 로직 체크 함수
def check_bq41_logic_2018(row):

    # CASE1 : 임금근로자 & 비임금 근로자 문항에 모두 응답   --> 1 건 
    if ((row['bq41_1'] != 0 ) or (row['bq41_2'] != 0)) and (row['bq41_3'] != 0)  :    
        bq41_breaker_1_2018.append(row['idx'])
        
    # CASE2 : 임금근로자 & 비임금 근로자 문항에 모두 무응답   -->  21건  
    elif ((row['bq41_1'] == 0 ) and (row['bq41_2'] == 0)) and (row['bq41_3'] == 0 ) :
        bq41_breaker_2_2018.append(row['idx'])

In [205]:
# 전역변수 --> 나중에 바꿀예정

global bq41_breaker_1_2018
global bq41_breaker_2_2018


bq41_breaker_1_2018 = []
bq41_breaker_2_2018 = []

# 반복문 말고 applymap
for i in range(len(processing_train_2018)):
    check_bq41_logic_2018(processing_train_2018.iloc[i,:])

print(len(bq41_breaker_1_2018) , '건')
print(bq41_breaker_1_2018)
    
print(len(bq41_breaker_2_2018) , '건')
print(bq41_breaker_2_2018)

1 건
[14568]
21 건
[9571, 9606, 9627, 9669, 9944, 10407, 10516, 10785, 11216, 11363, 11605, 12966, 13135, 13305, 14252, 15228, 17424, 17867, 17924, 18031, 18057]


In [206]:
#2018년 bq41 로직 무시 case1  답변 제거 함수
def fix_bq41_breaker_1_2018(row, idx):
    target_bq38 = row['bq38'].values[0]
    
    
    # 임금 근로자 이면 
    if target_bq38 == 1.0 :
        # 비임금근로자 문항에 한 답변 삭제
        processing_train_2018.loc[processing_train_2018[processing_train_2018['idx'] == idx].index, ['bq41_3']] = 0
        
        
    # 비임금근로자 이면 
    elif target_bq38 == 2.0 :
        # 임금근로자 문항에 한 답변 삭제         
        processing_train_2018.loc[processing_train_2018[processing_train_2018['idx'] == idx].index, ['bq41_1']] = 0
        processing_train_2018.loc[processing_train_2018[processing_train_2018['idx'] == idx].index, ['bq41_2']] = 0
        

In [207]:
#2018년 bq41 로직 무시 case2 답변 대체 함수
def fix_bq41_breaker_2_2018(row, idx):
    
    target_bq1  = row['bq1'].values[0]
    target_bq2  = row['bq2'].values[0]
    target_bq38 = row['bq38'].values[0]
    
    # 임금 근로자
    if target_bq38 == 1.0  :
        # 같은 산업군 & 근무유형 & 임금근로자인 사람들 답변의 평균으로 대체  
        
        processing_train_2018.loc[processing_train_2018[processing_train_2018['idx'] == idx].index, ['bq41_1']] = mean_by_IWS_2018[((mean_by_IWS_2018['bq1'] == target_bq1) & (mean_by_IWS_2018['bq2'] == target_bq2)) & (mean_by_IWS_2018['bq38'] == target_bq38)]['bq41_1'].values[0]    
        processing_train_2018.loc[processing_train_2018[processing_train_2018['idx'] == idx].index, ['bq41_2']] = mean_by_IWS_2018[((mean_by_IWS_2018['bq1'] == target_bq1) & (mean_by_IWS_2018['bq2'] == target_bq2)) & (mean_by_IWS_2018['bq38'] == target_bq38)]['bq41_2'].values[0]
        
    # 비임금근로자
    elif target_bq38 == 2.0 :
        # 같은 산업군 & 근무유형 & 비임금근로자인 사람들 답변의 평균으로 대체  
        processing_train_2018.loc[processing_train_2018[processing_train_2018['idx'] == idx].index, ['bq41_3']] = mean_by_IWS_2018[((mean_by_IWS_2018['bq1'] == target_bq1) & (mean_by_IWS_2018['bq2'] == target_bq2)) & (mean_by_IWS_2018['bq38'] == target_bq38)]['bq41_3'].values[0]
        
    else :
        print("패스되었음")

In [208]:
#2018년 bq41 로직 위반 답안 수정

# CASE1 : 임금근로자 & 비임금 근로자 문항에 모두 응답함
for idx in bq41_breaker_1_2018:
    fix_bq41_breaker_1_2018(processing_train_2018[processing_train_2018['idx'] == idx] , idx)

# CASE2 : 임금근로자 & 비임금 근로자 문항에 모두 응답 안함
for idx in bq41_breaker_2_2018:
    fix_bq41_breaker_2_2018(processing_train_2018[processing_train_2018['idx'] == idx] , idx)

###### 5-2) bq38 로직 체크

In [209]:
def check_bq38_logic_2018(row):
#bq38 로직 체크 함수

    # CASE1 : 임금 근로자인데 비임금 근로자 문항에 응답 
    if (row['bq38'] == 1.0  and  row['bq38_2'] != 0):
        bq38_breaker_1_2018.append(row['idx'])


    # CASE2 : 비임금 근로자 인데  임금근로자 문항에 응답 
    elif (row['bq38'] == 2.0  and   row['bq38_1'] != 0):
        bq38_breaker_2_2018.append(row['idx'])

In [210]:
global bq38_breaker_1_2018
global bq38_breaker_2_2018

bq38_breaker_1_2018 = []
bq38_breaker_2_2018 = []

# 반복문 말고 applymap
for i in range(len(processing_train_2018)):
    check_bq38_logic_2018(processing_train_2018.iloc[i,:])
    
print(len(bq38_breaker_1_2018) , '건')
print(bq38_breaker_1_2018)

print(len(bq38_breaker_2_2018) , '건')
print(bq38_breaker_2_2018)

0 건
[]
0 건
[]


-------> 해당 로직 위반 답안 없음  

###### 5-3) bq39 로직 체크

In [211]:
def check_bq39_logic_2018(row):
#bq39 로직 체크 함수

    # CASE : 비임금 근로자인데 임금 근로자 문항에 응답 
    if (row['bq38'] == 2  and  row['bq39'] != 0):
        bq39_breaker_2018.append(row['idx'])


In [212]:
global bq39_breaker_2018

bq39_breaker_2018 = []
for i in range(len(processing_train_2018)):
    check_bq39_logic_2018(processing_train_2018.iloc[i,:])

    
    
print(len(bq39_breaker_2018) , '건')
print(bq39_breaker_2018)

1 건
[14473]


In [213]:
for idx in bq39_breaker_2018:
    # 미응답으로 대체 
    processing_train_2018.loc[processing_train_2018[processing_train_2018['idx'] == idx].index, ['bq39']] = 0

###### 5-4) bq37 로직 체크

In [214]:
def check_bq37_logic_2018(row):
#bq37 로직 체크 함수

    # "중학교 졸업 이하"인데  최종학력 학과 문항에 응답 
    if (row['bq37'] == 1  and   row['bq37_1'] != (0 or'0') ):
        bq37_breaker_1_2018.append(row['idx'])

In [215]:
global bq37_breaker_1_2018

bq37_breaker_1_2018 =[]
for i in range(len(processing_train_2018)):
    check_bq37_logic_2018(processing_train_2018.iloc[i,:])

    
print(len(bq37_breaker_1_2018) , '건')
print(bq37_breaker_1_2018)


1 건
[17084]


In [216]:
for idx in bq37_breaker_1_2018:
    # 미응답으로 대체 
    processing_train_2018.loc[processing_train_2018[processing_train_2018['idx'] == idx].index, ['bq37_1']] = 0

###### 5-5) bq25 로직 체크

In [217]:
def check_bq25_logic_2018(row):
#bq25 로직 체크 함수

    # 2개 이상 업체 근로.도급계약 가능 인데 전속성2 설문에 응답한 경우 
    if (row['bq25'] == (2 or 3) and  row['bq25_1'] != 0):
        bq25_breaker_1_2018.append(row['idx'])

In [218]:
global bq25_breaker_1_2018
bq25_breaker_1_2018 =[]

# 반복문 말고 applymap
for i in range(len(processing_train_2018)):
    check_bq25_logic_2018(processing_train_2018.iloc[i,:])

In [219]:
    
global bq25_breaker_1_2018
bq25_breaker_1_2018 =[]

for i in range(len(processing_train_2018)):
    check_bq25_logic_2018(processing_train_2018.iloc[i,:])

print(len(bq25_breaker_1_2018) , '건')
print(bq25_breaker_1_2018)

0 건
[]


-------> 해당 로직 위반 답안 없음  

- index 재정리

In [220]:
processing_train_2018.reset_index(drop=True, inplace = True)
processing_train_2018.head()

,idx,cq1,cq2,cq3,cq4,cq5,cq6,cq7,cq8,cq9,...,bq38,bq38_1,bq38_2,bq39,bq40,bq41_1,bq41_2,bq41_3,knowcode,bq8
0,9486,5,3,4,2,2,5,4,5,4,...,1,1,0,1,42,3800.0,3400.0,0.0,306301,4.0
1,9487,4,3,4,4,4,4,4,4,4,...,1,2,0,1,40,2300.0,1800.0,0.0,133204,3.0
2,9488,4,1,5,3,3,4,2,4,1,...,2,0,6,0,30,0.0,0.0,3000.0,702502,2.0
3,9489,4,3,4,4,4,3,3,2,3,...,1,1,0,1,40,3350.0,2750.0,0.0,121201,5.0
4,9490,3,2,3,1,1,4,3,4,3,...,1,1,0,1,40,2500.0,2300.0,0.0,306301,4.0


- 서술형 문항 제거

In [221]:
# 서술형 문항  리스트 

essay_list_2018  = [
"bq5_2",
"bq28_1",
"bq29",
"bq30",
"bq31",
"bq32",
"bq33",
'bq4_1a',
'bq37_1'   
]

In [222]:
drop_list = essay_list_2018

#  자격증 질문 
drop_list.remove('bq4_1a')
# 전공질문
drop_list.remove('bq37_1')


In [223]:
processing_train_2018 = processing_train_2018.drop(drop_list, axis = 1)

In [224]:
for column in processing_train_2018.columns:
    if (column == 'bq37_1') or (column == 'bq4_1a'):
        pass
    else:  
        if processing_train_2018[column].dtype != 'int':
            processing_train_2018[column] = pd.to_numeric(processing_train_2018[column], downcast='integer')
            print('완료 : ' , column )


완료 :  cq18
완료 :  cq19
완료 :  cq20
완료 :  cq21
완료 :  cq22
완료 :  cq23
완료 :  cq24
완료 :  cq25
완료 :  cq26
완료 :  cq27
완료 :  cq28
완료 :  cq29
완료 :  cq30
완료 :  cq31
완료 :  cq32
완료 :  cq33
완료 :  cq34
완료 :  cq35
완료 :  cq36
완료 :  cq37
완료 :  cq38
완료 :  cq39
완료 :  cq40
완료 :  cq41
완료 :  cq42
완료 :  cq43
완료 :  cq44
완료 :  cq45
완료 :  cq46
완료 :  cq47
완료 :  cq48
완료 :  cq49
완료 :  cq50_1
완료 :  cq50_2
완료 :  cq50_3
완료 :  cq50_4
완료 :  cq50_5
완료 :  cq50_6
완료 :  cq50_7
완료 :  cq50_8
완료 :  iq1
완료 :  iq2
완료 :  iq3
완료 :  iq4
완료 :  iq5
완료 :  iq6
완료 :  bq1
완료 :  bq2
완료 :  bq3
완료 :  bq4
완료 :  bq5
완료 :  bq5_1
완료 :  bq6
완료 :  bq7
완료 :  bq9
완료 :  bq10
완료 :  bq11
완료 :  bq12_5
완료 :  bq13
완료 :  bq14
완료 :  bq15
완료 :  bq16
완료 :  bq17
완료 :  bq18
완료 :  bq19
완료 :  bq20
완료 :  bq21
완료 :  bq25
완료 :  bq25_1
완료 :  bq26_1
완료 :  bq26_1a
완료 :  bq26_2
완료 :  bq26_2a
완료 :  bq26_3
완료 :  bq26_3a
완료 :  bq26_4
완료 :  bq26_4a
완료 :  bq27
완료 :  bq28
완료 :  bq34
완료 :  bq35
완료 :  bq36
완료 :  bq37
완료 :  bq38
완료 :  bq38_1
완료 :  bq38_2
완료 :  bq39
완료 :  bq40
완

In [225]:
processing_train_2018.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9068 entries, 0 to 9067
Data columns (total 113 columns):
 #    Column    Dtype  
---   ------    -----  
 0    idx       int64  
 1    cq1       int64  
 2    cq2       int64  
 3    cq3       int64  
 4    cq4       int64  
 5    cq5       int64  
 6    cq6       int64  
 7    cq7       int64  
 8    cq8       int64  
 9    cq9       int64  
 10   cq10      int64  
 11   cq11      int64  
 12   cq12      int64  
 13   cq13      int64  
 14   cq14      int64  
 15   cq15      int64  
 16   cq16      int64  
 17   cq17      int64  
 18   cq18      int8   
 19   cq19      int8   
 20   cq20      int8   
 21   cq21      int8   
 22   cq22      int8   
 23   cq23      int8   
 24   cq24      int8   
 25   cq25      int8   
 26   cq26      int8   
 27   cq27      int8   
 28   cq28      int8   
 29   cq29      int8   
 30   cq30      int8   
 31   cq31      int8   
 32   cq32      int8   
 33   cq33      int8   
 34   cq34      int8   
 35 

### 2.2.2.  2018년 Test 데이터 전처리

In [226]:
test_2018 = know_test[1]

In [227]:
len(test_2018)

9069

- 행 수 9069

In [228]:
# 2018년 칼럼 리스트 
col_list_2018 = test_2018.columns.tolist()

In [229]:
len(col_list_2018)

138

In [230]:
test_2018.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9069 entries, 0 to 9068
Data columns (total 138 columns):
 #    Column   Dtype  
---   ------   -----  
 0    idx      int64  
 1    cq1      int64  
 2    cq2      int64  
 3    cq3      int64  
 4    cq4      int64  
 5    cq5      int64  
 6    cq6      int64  
 7    cq7      int64  
 8    cq8      int64  
 9    cq9      int64  
 10   cq10     int64  
 11   cq11     int64  
 12   cq12     int64  
 13   cq13     int64  
 14   cq14     int64  
 15   cq15     int64  
 16   cq16     int64  
 17   cq17     float64
 18   cq18     int64  
 19   cq19     float64
 20   cq20     float64
 21   cq21     float64
 22   cq22     float64
 23   cq23     float64
 24   cq24     float64
 25   cq25     float64
 26   cq26     float64
 27   cq27     int64  
 28   cq28     float64
 29   cq29     float64
 30   cq30     float64
 31   cq31     float64
 32   cq32     float64
 33   cq33     int64  
 34   cq34     float64
 35   cq35     float64
 36   cq36     flo

##### 1) 결측치 및 오류 데이터 처리

###### 1-1) NULL 값 처리

In [231]:
processing_test_2018 = test_2018.copy()

In [232]:
null_count_per_columns = processing_test_2018.isnull().sum()
null_count_per_columns

idx       0
cq1       0
cq2       0
cq3       0
cq4       0
         ..
bq39      0
bq40      0
bq41_1    0
bq41_2    0
bq41_3    0
Length: 138, dtype: int64

- NULL값 : 0으로 대체

In [233]:
processing_test_2018 = processing_test_2018.fillna(0)

In [234]:
processing_test_2018.isnull().sum()

idx       0
cq1       0
cq2       0
cq3       0
cq4       0
         ..
bq39      0
bq40      0
bq41_1    0
bq41_2    0
bq41_3    0
Length: 138, dtype: int64

In [235]:
processing_test_2018.isnull().sum().sum()

0

###### 1-2) 공백처리 : 0 으로 대체

In [236]:
for col in processing_test_2018.columns:
    for i in range(1,10):
        processing_test_2018[col].replace(' '*i, 0, inplace = True)

- 0이 1개라도 있는 행 전부 출력

In [237]:
for col in processing_test_2018.columns:
    print( col , len(processing_test_2018.index[processing_test_2018[col]==0].tolist()) )

idx 0
cq1 0
cq2 0
cq3 0
cq4 0
cq5 0
cq6 0
cq7 0
cq8 0
cq9 0
cq10 0
cq11 0
cq12 0
cq13 0
cq14 0
cq15 0
cq16 0
cq17 0
cq18 0
cq19 0
cq20 0
cq21 0
cq22 0
cq23 0
cq24 0
cq25 0
cq26 0
cq27 0
cq28 0
cq29 0
cq30 0
cq31 0
cq32 0
cq33 0
cq34 0
cq35 0
cq36 0
cq37 0
cq38 0
cq39 0
cq40 0
cq41 0
cq42 0
cq43 0
cq44 0
cq45 0
cq46 0
cq47 0
cq48 0
cq49 0
cq50_1 0
cq50_2 0
cq50_3 0
cq50_4 0
cq50_5 0
cq50_6 0
cq50_7 0
cq50_8 0
iq1 0
iq2 0
iq3 0
iq4 0
iq5 0
iq6 0
bq1 0
bq2 0
bq3 0
bq4 0
bq4_1a 0
bq5 0
bq5_1 0
bq5_2 0
bq6 0
bq7 0
bq8_1 0
bq8_2 0
bq8_3 0
bq9 0
bq10 0
bq11 0
bq12_1 0
bq12_2 0
bq12_3 0
bq12_4 0
bq12_5 0
bq13 0
bq14 0
bq15 0
bq16 0
bq17 0
bq18 0
bq19 0
bq20 0
bq21 1375
bq221 0
bq222 0
bq223 4532
bq231 0
bq232 8842
bq233 8760
bq234 0
bq235 0
bq241 0
bq242 0
bq243 0
bq244 0
bq245 0
bq25 0
bq25_1 0
bq26_1 0
bq26_1a 0
bq26_2 0
bq26_2a 0
bq26_3 0
bq26_3a 0
bq26_4 0
bq26_4a 0
bq27 0
bq28 0
bq28_1 0
bq29 0
bq30 0
bq31 0
bq32 0
bq33 0
bq34 0
bq35 0
bq36 0
bq37 0
bq37_1 0
bq38 0
bq38_1 0
bq38_2 0
bq39 

###### 1-3) "없다" 의미 답변 대체 

- "없다" 가 미응답과 같은지는 생각해볼 문제 

In [238]:
# 2018년 칼럼 리스트 
col_list_2018 = processing_test_2018.columns.tolist()

In [239]:
nope_word_list = ['없다','없음','없음.','없음_','없습니다','없습니다.','없어요','없어요~','업음', '.']

In [240]:
# "없다" 라는 의미의 단어를 0으로 대체하는 함수
def nope_word_replace(col):
    for word in nope_word_list:
        col.replace(word, 0, inplace = True)

In [241]:
# 문자가 들어가는 문항 칼럼 리스트
obj_col_list_2018 = []

In [242]:
for col in col_list_2018:
    
    # 문자열이 포함된 칼럼에서 "없다" 대체
    if processing_test_2018[col].dtype == 'object':
        nope_word_replace(processing_test_2018[col])
        # object 타입 칼럼 리스트 
        obj_col_list_2018.append(col)

###### 1-4) 밀려쓴 응답 처리 

- bq4, bq28, bq35 등의 문항을 통해 밀려쓴 답안 확인 

In [243]:
processing_test_2018['bq4'].value_counts()

2         5091
1         3948
0           14
전기기사        11
전기관련         1
지게차기능사       1
압연 기능사       1
산업안전         1
제강기능사        1
Name: bq4, dtype: int64

In [244]:
processing_test_2018['bq35'].value_counts()

1.0     3252
1       2656
2.0     1704
2       1424
48         3
53         3
38         3
28.0       2
38.0       2
31.0       2
54.0       1
39.0       1
69.0       1
26.0       1
50.0       1
44.0       1
41.0       1
40.0       1
49.0       1
30.0       1
문예창작       1
52         1
29         1
46         1
35         1
50         1
39         1
51.0       1
Name: bq35, dtype: int64

In [245]:
processing_test_2018['bq28'].value_counts()

2                                       3204
3                                       2589
4                                       2089
1                                        873
5                                        284
외주화                                        4
사람이 해야 할 일이 어느 정도는 있을 것 같아서                1
사업체 감소로                                    1
힘든업종이라서                                    1
업무 자동화에 대한 한계가 있음, 주로 직접 사람이 작업을 해야함       1
전기는 변하지 않음                                 1
꾸준함/변하지 않고 일정함                             1
일을 할려는 사람이 없다                              1
자동화로 인한 일자리 감소                             1
자동화되어도 수작업이 필요                             1
근무환경이 열악해서 젊은이들이 배울려고 하질 않음                1
어느정도는 사람이 해야하는 일이므로                        1
근무형태 변화                                    1
자동화 될 가능성이 높다                              1
전산화,인공지능, 4차혁명, 빅데이터 등 발전하여 변화됨            1
차량도입 및 유지보수의 증가 필요                         1
기계의 자동화로 인해서                               1
KTX의 이용증가로

###### A. cq1 ~ bq41 까지 1칸씩 밀린 행 
- 원래 위치로 복원 후 cq1 은 무응답으로 처리 

In [246]:
forward_1_pushed_list_2018 = []

- bq28 에서 숫자가 아닌 답변을 한 행의 idx 를 추출

In [247]:
for idx in processing_test_2018['idx']:
    
    target_row = processing_test_2018[processing_test_2018['idx'] == idx]
    
    if ~target_row['bq28'].isin([0,1,2,3,4,5, '0','1','2','3','4','5']).any():
        forward_1_pushed_list_2018.append(idx)

In [248]:
len(forward_1_pushed_list_2018)

30

###### B. 밀린 행 원복하기 
- 밀린 행 원복 함수 

In [249]:
# 앞 문항을 뒷 문항으로 덮어쓴 케이스 보정 

## idx             : 밀려쓰기가 발생한 행의 idx 
## pushed_step    : 밀린 칸의 수
## start_col      : 밀려쓰기 시작한 칼럼 
## end_col        : 밀려쓴 마지막 칼럼 

def fix_forward_pushed_answer_2018(idx, pushed_step, start_col, end_col):
    
    target_row = processing_test_2018[processing_test_2018['idx'] == idx]
    
    start_col_index = col_list_2018.index(start_col)
    end_col_index = col_list_2018.index(end_col)

    # 밀려쓴 마지막 칼럼부터, pushed_step 만큼 앞 칼럼 값으로 덮어씌우기
    for i in range(end_col_index, start_col_index, -1):
        target_row.iloc[:,i] = target_row.iloc[:,i - pushed_step]
    
    # 밀린행 자격증 2, 3 
    target_row['bq5'] = 0
    target_row['bq5_1'] = 0
    
    # 밀려쓴 가장 첫 칼럼 값 : 무응답으로 처리 
    target_row.iloc[:,start_col_index] = 0
    
    # 밀려쓴 행을 수정한 행으로 대체 
    processing_test_2018[processing_test_2018['idx'] == idx] = target_row 

In [250]:
for pushed_idx in forward_1_pushed_list_2018:
    fix_forward_pushed_answer_2018(pushed_idx, 1,'cq1','bq41_3' )

###### C. bq29 ~ bq41 까지 2칸씩 밀린 행 
- bq29, bq30, bq31 의 답변이 bq28의 마지막 문장에 통합된 경우 : 원래 위치로 복원 후 bq28의 마지막 답으로 처리 

- idx : 13474

In [251]:
forward_3_pushed_list_2018 = [13474]

In [252]:
for pushed_idx in forward_3_pushed_list_2018:
    fix_forward_pushed_answer_2018(pushed_idx, 3,'bq29','bq41_3' )

In [253]:
# bq29 : 유사직업명 
processing_test_2018.loc[processing_test_2018[processing_test_2018['idx'] == 13474].index, ['bq29']] = '방송작가'

# bq30 : 사용하는 도구 및 프로그램명  
processing_test_2018.loc[processing_test_2018[processing_test_2018['idx'] == 13474].index, ['bq30']] = '한글, 동영상편집'

###### C. bq29 ~ bq41 까지 1칸씩 밀리고, bq29의 답변이 bq28의 마지막 문장에 통합된 경우 
- 원래 위치로 복원 후 bq29 은 bq28의 마지막 답으로 처리 
- idx : 14432  --> bq29 은 "연구간호사"로 처리
- idx : 15891  --> bq29 은 "설계선생님"로 처리

In [254]:
forward_1_C_pushed_list_2018 = [14432, 15891]

In [255]:
for pushed_idx in forward_1_C_pushed_list_2018:
    fix_forward_pushed_answer_2018(pushed_idx, 1,'bq29','bq41_3' )

In [256]:
# bq29 : 유사직업명 
processing_test_2018.loc[processing_test_2018[processing_test_2018['idx'] == 14432].index, ['bq29']] = '연구간호사'
processing_test_2018.loc[processing_test_2018[processing_test_2018['idx'] == 15891].index, ['bq29']] = '설계선생님'

###### 1-6) 설문 데이터 형식 오류 

- bq221 ~ bq223, bq231 ~ bq235, bq241 ~ bq245 은 각각 하나의 설문 문항이나, testdata에서는 선택지의 갯수만큼 문항이 개별 칼럼으로 분리되어 있음
- 이 경우 각 칼럼은 예/아니오 로 답변되어야 하지만, 응답 내용이 1~5 등으로 제각각임
- 데이터의 신뢰성을 보장할 수 없으므로 해당 칼럼은 삭제 

In [257]:
processing_test_2018.drop(['bq221','bq222','bq223','bq231','bq232','bq233','bq234','bq235','bq241','bq242','bq243','bq244','bq245'], axis = 1, inplace = True)

#### 2) 데이터 타입 통일

##### 2-1) object -> int

In [258]:
obj_to_int_list_2018 = [
'bq4',
'bq5',
'bq5_1', # 요구훈련기간'
'bq21', # 기본급, 고정급 비중 
'bq25_1',
'bq26_1',    
'bq26_1a',    
'bq26_2'  ,  
'bq26_2a'  , 
'bq26_3'  ,
'bq26_3a' ,  
'bq26_4' ,  
'bq26_4a' ,  
'bq27',
'bq28',  # 향후 일자리 변화 
'bq34', # 근로시간 단축 가능 
'bq36',  # 나이 
'bq37',
'bq39',
'bq40',
"bq25",
"bq37",
"bq38",
"bq38_1",
"bq38_2",
"bq41_1",
"bq41_2",
"bq41_3"
]

In [259]:
for column in obj_to_int_list_2018:
    processing_test_2018[column]= pd.to_numeric(processing_test_2018[column], downcast='integer')
    print('완료 : ' , column )

완료 :  bq4
완료 :  bq5
완료 :  bq5_1
완료 :  bq21
완료 :  bq25_1
완료 :  bq26_1
완료 :  bq26_1a
완료 :  bq26_2
완료 :  bq26_2a
완료 :  bq26_3
완료 :  bq26_3a
완료 :  bq26_4
완료 :  bq26_4a
완료 :  bq27
완료 :  bq28
완료 :  bq34
완료 :  bq36
완료 :  bq37
완료 :  bq39
완료 :  bq40
완료 :  bq25
완료 :  bq37
완료 :  bq38
완료 :  bq38_1
완료 :  bq38_2
완료 :  bq41_1
완료 :  bq41_2
완료 :  bq41_3


##### 2-2) float -> int

In [260]:
float_to_int_list_2018 = [
'bq27' , # 플랫폼 근무자 여부 
'bq7',  # 교육수준    
"bq13", #	업무내용 규정성
"bq14", #	제 규정 적용 여부
"bq15", #	지휘감독여부
"bq16", #	거부 가능성
"bq17", #	근무시간장소
"bq18", #	작업도구 소유여부
"bq19", #	업무 대체성
"cq48",  #   재택근무 가능여부 
"cq18",
"cq19",
"cq20",
"cq21",
"cq22",
"cq23",
"cq24",
"cq25",
"cq26",
"cq27",
"cq28",
"cq29",
"cq30",
"cq31",
"cq32",
"cq33",
"cq34",
"cq35",
"cq36",
"cq37",
"cq38",
"cq39",
"cq40",
"cq41",
"cq42",
"cq43",
"cq44",
"cq45",
"cq46",
"cq47",
"cq48",
"cq49",
"cq50_1",
"cq50_2",
"cq50_3",
"cq50_4",
"cq50_5",
"cq50_6",
"cq50_7",
"cq50_8",
"iq1",
"iq2",
"iq3",
"iq4",
"iq5",
"iq6",
"bq3",
"bq6",
"bq8_1",
"bq8_2",
"bq8_3",
"bq9",
"bq10",
"bq11",
"bq12_1",
"bq12_5",
"bq20" 
]

In [261]:
for column in float_to_int_list_2018:
    # 정수형으로 먼저 변환  
    processing_test_2018[column]= pd.to_numeric(processing_test_2018[column], downcast='integer')
    print('완료 : ' , column )

완료 :  bq27
완료 :  bq7
완료 :  bq13
완료 :  bq14
완료 :  bq15
완료 :  bq16
완료 :  bq17
완료 :  bq18
완료 :  bq19
완료 :  cq48
완료 :  cq18
완료 :  cq19
완료 :  cq20
완료 :  cq21
완료 :  cq22
완료 :  cq23
완료 :  cq24
완료 :  cq25
완료 :  cq26
완료 :  cq27
완료 :  cq28
완료 :  cq29
완료 :  cq30
완료 :  cq31
완료 :  cq32
완료 :  cq33
완료 :  cq34
완료 :  cq35
완료 :  cq36
완료 :  cq37
완료 :  cq38
완료 :  cq39
완료 :  cq40
완료 :  cq41
완료 :  cq42
완료 :  cq43
완료 :  cq44
완료 :  cq45
완료 :  cq46
완료 :  cq47
완료 :  cq48
완료 :  cq49
완료 :  cq50_1
완료 :  cq50_2
완료 :  cq50_3
완료 :  cq50_4
완료 :  cq50_5
완료 :  cq50_6
완료 :  cq50_7
완료 :  cq50_8
완료 :  iq1
완료 :  iq2
완료 :  iq3
완료 :  iq4
완료 :  iq5
완료 :  iq6
완료 :  bq3
완료 :  bq6
완료 :  bq8_1
완료 :  bq8_2
완료 :  bq8_3
완료 :  bq9
완료 :  bq10
완료 :  bq11
완료 :  bq12_1
완료 :  bq12_5
완료 :  bq20


- 확인

In [262]:

processing_test_2018.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9069 entries, 0 to 9068
Data columns (total 125 columns):
 #    Column   Dtype  
---   ------   -----  
 0    idx      int64  
 1    cq1      int64  
 2    cq2      int64  
 3    cq3      int64  
 4    cq4      int64  
 5    cq5      int64  
 6    cq6      int64  
 7    cq7      int64  
 8    cq8      int64  
 9    cq9      int64  
 10   cq10     int64  
 11   cq11     int64  
 12   cq12     int64  
 13   cq13     int64  
 14   cq14     int64  
 15   cq15     int64  
 16   cq16     int64  
 17   cq17     float64
 18   cq18     int8   
 19   cq19     int8   
 20   cq20     int8   
 21   cq21     int8   
 22   cq22     int8   
 23   cq23     int8   
 24   cq24     int8   
 25   cq25     int8   
 26   cq26     int8   
 27   cq27     int8   
 28   cq28     int8   
 29   cq29     int8   
 30   cq30     int8   
 31   cq31     int8   
 32   cq32     int8   
 33   cq33     int8   
 34   cq34     int8   
 35   cq35     int8   
 36   cq36     int

##### 3) 객관식 문항 정상 답변 여부

###### 3-1) 답안 범위가 1~5 인 답변

In [263]:
one_to_five_question_list_2018  = [
"cq1",
"cq2",
"cq3",
"cq4",
"cq5",
"cq6",
"cq7",
"cq8",
"cq9",
"cq10",
"cq11",
"cq12",
"cq13",
"cq14",
"cq15",
"cq16",
"cq17",
"cq18",
"cq19",
"cq20",
"cq21",
"cq22",
"cq23",
"cq24",
"cq25",
"cq26",
"cq27",
"cq28",
"cq29",
"cq30",
"cq31",
"cq32",
"cq33",
"cq34",
"cq35",
"cq36",
"cq37",
"cq38",
"cq39",
"cq40",
"cq41",
"cq42",
"cq43",
"cq44",
"cq45",
"cq46",
"cq49",
"iq1",
"iq2",
"iq3",
"iq4",
"iq5",
"iq6",
"bq8_1",
"bq8_2",
"bq8_3",
"bq9",
"bq10",
"bq11",
"bq12_1",
"bq12_5",
"bq20",
"bq28",    
"bq34"]

- 모든 문항 정상 답변

###### 3-3) 답안 범위가 1~3 인 답변

In [264]:
one_to_three_question_list_2018  = [
"cq47",
"cq50_1",
"cq50_2",
"cq50_3",
"cq50_4",
"cq50_5",
"cq50_6",
"cq50_7",
"cq50_8",
"bq13",
"bq17",
"bq18",
"bq19",
"bq25",
"bq26_1",
"bq26_1a",
"bq26_2",
"bq26_2a",
"bq26_3",
"bq26_3a",
"bq26_4",
"bq26_4a"
]

In [265]:
wrong_1_3_2018_list = []

for col in one_to_three_question_list_2018:
    
    # float형 칼럼과 object 형 칼럼이 모두 존재
    if ~processing_test_2018[col].isin([0,1,2,3, '1','2','3']).any():
        wrong_1_3_2018_list.append(col)
        
print(len(wrong_1_3_2018_list))

0


- 모든 문항 정상 답변 

###### 3-3) 답안 범위가 1~2 인 답변

In [266]:
one_to_two_question_list_2018 = [
"cq48",
"bq4",
"bq5",
"bq14",
"bq16",
"bq25_1",
"bq27",
"bq35",
"bq38",
"bq39"]

In [267]:
wrong_1_2_2018_list = []

for col in one_to_two_question_list_2018:
    
    # float형 칼럼과 object 형 칼럼이 모두 존재
    if ~processing_test_2018[col].isin([0,1,2, '1','2']).any():
        wrong_1_2_2018_list.append(col)
        
print(len(wrong_1_2_2018_list))

0


- 모든 문항 정상 답변

###### 3-4) 답안 범위가 1~6 인 답변

In [268]:
one_to_six_question_list_2018 = [
"bq2",
"bq3",
"bq5_1",
"bq37"]

In [269]:
wrong_1_6_2018_list = []

for col in one_to_six_question_list_2018:
    
    # float형 칼럼과 object 형 칼럼이 모두 존재
    if ~processing_test_2018[col].isin([0,1,2,3,4,5,6, '1','2','3','4','5','6']).any():
        wrong_1_6_2018_list.append(col)
        
print(len(wrong_1_6_2018_list))

0


- 모든 문항 정상 답변 

###### 3-5) 답안 범위가 1~5, 9 인 답변

In [270]:
one_to_five_nine_question_list_2018 = [
"bq12_2",
"bq12_3",
"bq12_4"
]

In [271]:
wrong_1_5_9_2018_list = []

for col in one_to_five_nine_question_list_2018:
    
    # float형 칼럼과 object 형 칼럼이 모두 존재
    if ~processing_test_2018[col].isin([0,1,2,3,4,5,9, '1','2','3','4','5','9']).any():
        wrong_1_5_9_2018_list.append(col)
        
print(len(wrong_1_5_9_2018_list))

0


###### 3-6) 세트 문항 합쳐서 평균으로 제시 

In [272]:
processing_test_2018['bq8'] = round((processing_test_2018['bq8_1']+processing_test_2018['bq8_2']+processing_test_2018['bq8_3'])/3)
processing_test_2018 = processing_test_2018.drop(['bq8_1','bq8_2','bq8_3'],axis=1)
processing_test_2018 = processing_test_2018.drop(['bq12_1','bq12_2','bq12_3','bq12_4'],axis = 1)

- 모든 문항 정상 답변

##### 4) 비상식적 답변 확인 

- 산업 유형 & 근무업체 유형 별 각 문항의 평균 표시
- 비정상 답변 대체 용

In [273]:

mean_by_IW_2018

,bq1,bq2,idx,cq1,cq2,cq3,cq4,cq5,cq6,cq7,...,bq38,bq38_1,bq38_2,bq39,bq40,bq41_1,bq41_2,bq41_3,knowcode,bq8
0,1.0,1.0,13055.769231,4.384615,1.961538,3.230769,1.576923,1.923077,3.576923,3.269231,...,1.346154,0.769231,2.076923,0.730769,46.076923,2534.615385,1315.384615,1353.846154,902227.923077,2.576923
1,1.0,2.0,14841.833333,4.666667,2.166667,4.000000,4.166667,4.333333,4.166667,4.000000,...,1.000000,1.333333,0.000000,1.166667,43.333333,5800.000000,3916.666667,0.000000,122203.000000,3.666667
2,1.0,3.0,12747.714286,4.857143,3.142857,4.857143,4.428571,4.000000,4.714286,4.285714,...,1.000000,1.000000,0.000000,1.000000,58.285714,5285.714286,3714.285714,0.000000,903101.000000,3.000000
3,1.0,4.0,13536.637168,3.752212,1.300885,3.327434,1.398230,1.601770,3.212389,2.734513,...,1.955752,0.044248,5.584071,0.044248,47.973451,118.584071,88.495575,3177.168142,902204.539823,2.796460
4,1.0,5.0,14995.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,...,1.000000,3.000000,0.000000,1.000000,45.000000,3000.000000,2500.000000,0.000000,122101.000000,4.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,19.0,6.0,12961.000000,4.166667,2.666667,4.000000,3.833333,3.500000,4.000000,4.000000,...,1.500000,0.500000,3.000000,0.833333,45.666667,1533.333333,933.333333,1700.000000,348168.500000,3.833333
90,20.0,1.0,14029.200000,4.400000,1.400000,4.000000,1.800000,1.600000,3.000000,2.600000,...,1.600000,1.000000,3.600000,0.600000,36.600000,1260.000000,780.000000,1200.000000,454161.600000,1.800000
91,20.0,4.0,13605.823529,3.411765,2.088235,3.470588,2.941176,2.735294,3.294118,3.147059,...,1.323529,2.617647,1.941176,1.323529,37.970588,2620.588235,1104.411765,926.470588,413041.411765,2.882353
92,20.0,6.0,14424.916667,3.250000,1.333333,3.333333,1.166667,1.000000,2.083333,1.166667,...,2.000000,0.000000,6.000000,0.000000,33.000000,0.000000,0.000000,1830.000000,549059.416667,2.250000


- 산업 유형 & 근무업체 유형 & 임금/비임금  별 각 문항의 평균 표시

In [274]:

mean_by_IWS_2018

,bq1,bq2,bq38,idx,cq1,cq2,cq3,cq4,cq5,cq6,...,bq37,bq38_1,bq38_2,bq39,bq40,bq41_1,bq41_2,bq41_3,knowcode,bq8
0,1.0,1.0,1,13221.117647,4.470588,2.058824,3.352941,1.470588,1.882353,3.764706,...,2.411765,1.176471,0.0,1.117647,48.058824,3876.470588,2011.764706,129.411765,902518.647059,2.352941
1,1.0,1.0,2,12743.444444,4.222222,1.777778,3.000000,1.777778,2.000000,3.222222,...,2.222222,0.000000,6.0,0.000000,42.333333,0.000000,0.000000,3666.666667,901678.777778,3.000000
2,1.0,2.0,1,14841.833333,4.666667,2.166667,4.000000,4.166667,4.333333,4.166667,...,4.833333,1.333333,0.0,1.166667,43.333333,5800.000000,3916.666667,0.000000,122203.000000,3.666667
3,1.0,3.0,1,12747.714286,4.857143,3.142857,4.857143,4.428571,4.000000,4.714286,...,4.000000,1.000000,0.0,1.000000,58.285714,5285.714286,3714.285714,0.000000,903101.000000,3.000000
4,1.0,4.0,1,13866.200000,4.800000,1.600000,4.600000,2.200000,1.800000,4.600000,...,3.000000,1.000000,0.0,1.000000,50.400000,2680.000000,2000.000000,0.000000,901781.000000,3.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,20.0,1.0,2,14131.333333,4.666667,1.333333,4.333333,1.333333,1.000000,2.333333,...,1.333333,0.000000,6.0,0.000000,31.666667,0.000000,0.000000,2000.000000,561601.000000,2.000000
135,20.0,4.0,1,13925.043478,3.608696,2.521739,3.826087,3.478261,3.260870,3.304348,...,3.913043,3.869565,0.0,1.956522,42.217391,3873.913043,1632.608696,0.000000,412221.434783,3.130435
136,20.0,4.0,2,12938.363636,3.000000,1.181818,2.727273,1.818182,1.636364,3.272727,...,3.454545,0.000000,6.0,0.000000,29.090909,0.000000,0.000000,2863.636364,414755.909091,2.363636
137,20.0,6.0,2,14424.916667,3.250000,1.333333,3.333333,1.166667,1.000000,2.083333,...,2.083333,0.000000,6.0,0.000000,33.000000,0.000000,0.000000,1830.000000,549059.416667,2.250000


###### 4-1) 연령 답변 확인 

- 나이가 음수, 혹은 비상식적인 값이 있는가? 

In [275]:
processing_test_2018['bq36'].value_counts().sort_index()

18     5
19     5
20     9
21    27
22    55
      ..
78     5
79     1
80     3
81     1
86     1
Name: bq36, Length: 65, dtype: int64

- 10세 미만 존재한다면 --> 오기입으로 보고 산업 & 근무업체 유형별 평균으로 대체

In [276]:
too_young_list_test_2018 = list(processing_test_2018[processing_test_2018['bq36'] < 10]['idx'])
too_young_list_test_2018 

[]

In [277]:
for fake_young_idx in too_young_list_test_2018:

    target_bq1 = processing_test_2018[processing_test_2018['idx']==fake_young_idx]['bq1'].values[0]
    target_bq2 = processing_test_2018[processing_test_2018['idx']==fake_young_idx]['bq2'].values[0]
    
    processing_test_2018.loc[processing_test_2018[processing_test_2018['idx'] == fake_young_idx].index, ['bq36']] = mean_by_IW_2018[(mean_by_IW_2018['bq1'] == target_bq1) & (mean_by_IW_2018['bq2'] == target_bq2)]['bq36']                                      

###### 4-2) 주당 근로시간 답변 확인 

- 주당 근로시간이 음수, 혹은 터무니없이 높은 값이 있는가? 

In [278]:
processing_test_2018['bq40'].value_counts().sort_index()

4.0       1
6.0       2
8.0       2
9.0       3
10.0     29
         ..
84.0      8
90.0      8
91.0      2
98.0      1
100.0     3
Name: bq40, Length: 74, dtype: int64

- 1주일 : 168 시간이므로 168시간을 초과하는 답변 -> 산업 & 근무업체 유형별 평균으로 대체

In [279]:
too_busy_list_test_2018 = list(processing_test_2018[processing_test_2018['bq40'] > 168]['idx'])
too_busy_list_test_2018 

[]

In [280]:
for fake_busy_idx in too_busy_list_test_2018:

    target_bq1 = processing_test_2018[processing_test_2018['idx']==fake_busy_idx]['bq1'].values[0]
    target_bq2 = processing_test_2018[processing_test_2018['idx']==fake_busy_idx]['bq2'].values[0]
    
    processing_test_2018.loc[processing_test_2018[processing_test_2018['idx'] == fake_busy_idx].index, ['bq40']] = mean_by_IW_2018[(mean_by_IW_2018['bq1'] == target_bq1) & (mean_by_IW_2018['bq2'] == target_bq2)]['bq40']                                      

###### 4-3) 근로소득 답변 확인 

- 근로소득이 음수, 혹은 터무니없이 높은 값이 있는가? 

In [281]:
processing_test_2018['bq41_1'].value_counts().sort_index()

0        1298
300         2
400         1
500         3
540         1
         ... 
18000       4
20000       8
22000       1
23000       1
30000       1
Name: bq41_1, Length: 258, dtype: int64

In [282]:
processing_test_2018['bq41_2'].value_counts().sort_index()

0        1391
100         2
130         1
150         2
180         1
         ... 
13500       1
14000       1
14052      15
15000       1
25000       1
Name: bq41_2, Length: 174, dtype: int64

In [283]:
processing_test_2018['bq41_3'].value_counts().sort_index()

0        7793
120         1
150         1
200         2
300         7
         ... 
18000       4
19000       1
20000       1
25000       1
35000       1
Name: bq41_3, Length: 102, dtype: int64

- 이상없음

##### 5) 설문지 로직 체크

###### 5-1) bq41 로직 체크

In [284]:
#2018년 bq41 로직 체크 함수
def check_bq41_logic_2018(row):

    # CASE1 : 임금근로자 & 비임금 근로자 문항에 모두 응답   --> 1 건 
    if ((row['bq41_1'] != 0 ) or (row['bq41_2'] != 0)) and (row['bq41_3'] != 0)  :    
        bq41_breaker_1_2018.append(row['idx'])
        
    # CASE2 : 임금근로자 & 비임금 근로자 문항에 모두 무응답   --> 23 건  
    elif ((row['bq41_1'] == 0 ) and (row['bq41_2'] == 0)) and (row['bq41_3'] == 0 ) :
        bq41_breaker_2_2018.append(row['idx'])

In [285]:
# 전역변수 --> 나중에 바꿀예정

global bq41_breaker_1_2018
global bq41_breaker_2_2018


bq41_breaker_1_2018 = []
bq41_breaker_2_2018 = []

# 반복문 말고 applymap
for i in range(len(processing_test_2018)):
    check_bq41_logic_2018(processing_test_2018.iloc[i,:])

print(len(bq41_breaker_1_2018) , '건')
print(bq41_breaker_1_2018)
    
print(len(bq41_breaker_2_2018) , '건')
print(bq41_breaker_2_2018)

1 건
[11761]
23 건
[10450, 11239, 11364, 11683, 11978, 12178, 12198, 12560, 13896, 14009, 14261, 14408, 14430, 14437, 14441, 14982, 16336, 16421, 17105, 17567, 17927, 18123, 18422]


In [286]:
#2018년 bq41 로직 무시 case1  답변 제거 함수
def fix_bq41_breaker_1_2018(row, idx):
    target_bq38 = row['bq38'].values[0]
    
    
    # 임금 근로자 이면 
    if target_bq38 == 1.0 :
        # 비임금근로자 문항에 한 답변 삭제
        processing_test_2018.loc[processing_test_2018[processing_test_2018['idx'] == idx].index, ['bq41_3']] = 0
        
        
    # 비임금근로자 이면 
    elif target_bq38 == 2.0 :
        # 임금근로자 문항에 한 답변 삭제         
        processing_test_2018.loc[processing_test_2018[processing_test_2018['idx'] == idx].index, ['bq41_1']] = 0
        processing_test_2018.loc[processing_test_2018[processing_test_2018['idx'] == idx].index, ['bq41_2']] = 0
        

In [287]:
#2018년 bq41 로직 무시 case2 답변 대체 함수
def fix_bq41_breaker_2_2018(row, idx):
    
    target_bq1  = row['bq1'].values[0]
    target_bq2  = row['bq2'].values[0]
    target_bq38 = row['bq38'].values[0]
    
    # 임금 근로자
    if target_bq38 == 1.0  :
        # 같은 산업군 & 근무유형 & 임금근로자인 사람들 답변의 평균으로 대체  
        
        processing_test_2018.loc[processing_test_2018[processing_test_2018['idx'] == idx].index, ['bq41_1']] = mean_by_IWS_2018[((mean_by_IWS_2018['bq1'] == target_bq1) & (mean_by_IWS_2018['bq2'] == target_bq2)) & (mean_by_IWS_2018['bq38'] == target_bq38)]['bq41_1'].values[0]    
        processing_test_2018.loc[processing_test_2018[processing_test_2018['idx'] == idx].index, ['bq41_2']] = mean_by_IWS_2018[((mean_by_IWS_2018['bq1'] == target_bq1) & (mean_by_IWS_2018['bq2'] == target_bq2)) & (mean_by_IWS_2018['bq38'] == target_bq38)]['bq41_2'].values[0]
        
    # 비임금근로자
    elif target_bq38 == 2.0 :
        # 같은 산업군 & 근무유형 & 비임금근로자인 사람들 답변의 평균으로 대체  
        
        processing_test_2018.loc[processing_test_2018[processing_test_2018['idx'] == idx].index, ['bq41_3']] = mean_by_IWS_2018[((mean_by_IWS_2018['bq1'] == target_bq1) & (mean_by_IWS_2018['bq2'] == target_bq2)) & (mean_by_IWS_2018['bq38'] == target_bq38)]['bq41_3'].values[0]
        
    else :
        print("패스되었음")

In [288]:
#2018년 bq41 로직 위반 답안 수정

# CASE1 : 임금근로자 & 비임금 근로자 문항에 모두 응답함
for idx in bq41_breaker_1_2018:
    fix_bq41_breaker_1_2018(processing_test_2018[processing_test_2018['idx'] == idx] , idx)

# CASE2 : 임금근로자 & 비임금 근로자 문항에 모두 응답 안함
for idx in bq41_breaker_2_2018:
    fix_bq41_breaker_2_2018(processing_test_2018[processing_test_2018['idx'] == idx] , idx)

###### 5-2) bq38 로직 체크

In [289]:
def check_bq38_logic_2018(row):
#bq38 로직 체크 함수

    # CASE1 : 임금 근로자인데 비임금 근로자 문항에 응답 
    if (row['bq38'] == 1.0  and  row['bq38_2'] != 0):
        bq38_breaker_1_2018.append(row['idx'])


    # CASE2 : 비임금 근로자 인데  임금근로자 문항에 응답 
    elif (row['bq38'] == 2.0  and   row['bq38_1'] != 0):
        bq38_breaker_2_2018.append(row['idx'])

In [290]:
global bq38_breaker_1_2018
global bq38_breaker_2_2018

bq38_breaker_1_2018 = []
bq38_breaker_2_2018 = []

# 반복문 말고 applymap
for i in range(len(processing_test_2018)):
    check_bq38_logic_2018(processing_test_2018.iloc[i,:])
    
print(len(bq38_breaker_1_2018) , '건')
print(bq38_breaker_1_2018)

print(len(bq38_breaker_2_2018) , '건')
print(bq38_breaker_2_2018)

0 건
[]
0 건
[]


-------> 해당 로직 위반 답안 없음  

###### 5-3) bq39 로직 체크

In [291]:
def check_bq39_logic_2018(row):
#bq39 로직 체크 함수

    # CASE : 비임금 근로자인데 임금 근로자 문항에 응답 
    if (row['bq38'] == 2  and  row['bq39'] != 0):
        bq39_breaker_2018.append(row['idx'])


In [292]:
global bq39_breaker_2018

bq39_breaker_2018 = []

# 반복문 말고 applymap
for i in range(len(processing_test_2018)):
    check_bq39_logic_2018(processing_test_2018.iloc[i,:])
    
print(len(bq39_breaker_2018) , '건')
print(bq39_breaker_2018)

0 건
[]


In [293]:
for idx in bq39_breaker_2018:
    # 미응답으로 대체 
    processing_test_2018.loc[processing_test_2018[processing_test_2018['idx'] == idx].index, ['bq39']] = 0

###### 5-4) bq37 로직 체크

In [294]:
def check_bq37_logic_2018(row):
#bq37 로직 체크 함수

    # "중학교 졸업 이하"인데  최종학력 학과 문항에 응답 
    if (row['bq37'] == 1.0  and   row['bq37_1'] != (0 or '0') ):
        bq37_breaker_1_2018.append(row['idx'])

In [295]:
global bq37_breaker_1_2018

bq37_breaker_1_2018 =[]

# 반복문 말고 applymap
for i in range(len(processing_test_2018)):
    check_bq37_logic_2018(processing_test_2018.iloc[i,:])
    
print(len(bq37_breaker_1_2018) , '건')
print(bq37_breaker_1_2018)


1 건
[12176]


In [296]:
for idx in bq37_breaker_1_2018:
    # 미응답으로 대체 
    processing_test_2018.loc[processing_test_2018[processing_test_2018['idx'] == idx].index, ['bq37_1']] = 0

###### 5-5) bq25 로직 체크

In [297]:
def check_bq25_logic_2018(row):
#bq25 로직 체크 함수

    # 2개 이상 업체 근로.도급계약 가능 인데 전속성2 설문에 응답한 경우 
    if (row['bq25'] == (2 or 3) and  row['bq25_1'] != 0):
        bq25_breaker_1_2018.append(row['idx'])

In [298]:
global bq25_breaker_1_2018

bq25_breaker_1_2018 =[]

# 반복문 말고 applymap
for i in range(len(processing_test_2018)):
    check_bq25_logic_2018(processing_test_2018.iloc[i,:])
    
print(len(bq25_breaker_1_2018) , '건')
print(bq25_breaker_1_2018)


0 건
[]


-------> 해당 로직 위반 답안 없음  

In [299]:
processing_test_2018.reset_index()

,index,idx,cq1,cq2,cq3,cq4,cq5,cq6,cq7,cq8,...,bq37_1,bq38,bq38_1,bq38_2,bq39,bq40,bq41_1,bq41_2,bq41_3,bq8
0,0,9486,4,1,3,1,1,3,2,3,...,0,2,0,6,0,35.0,0.0,0.0,2000.0,2.0
1,1,9487,5,4,5,5,5,5,3,3,...,커뮤니케이션 디자인 전공,1,1,0,1,65.0,2700.0,2200.0,0.0,3.0
2,2,9488,3,3,3,4,3,3,4,4,...,국문학과,2,0,6,0,60.0,0.0,0.0,3200.0,3.0
3,3,9489,4,4,4,4,2,3,3,4,...,극작학과,2,0,6,0,20.0,0.0,0.0,2000.0,3.0
4,4,9490,5,2,3,2,4,4,4,4,...,실업,1,4,0,2,35.0,3000.0,1500.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9064,9064,18564,5,4,2,2,2,4,4,3,...,호텔조리학,2,0,5,0,72.0,0.0,0.0,8000.0,4.0
9065,9065,18565,2,2,5,5,5,4,4,4,...,식품공학,1,1,0,1,40.0,3000.0,2100.0,0.0,5.0
9066,9066,18566,4,4,3,2,2,3,4,4,...,환경공학,1,2,0,2,40.0,1800.0,1800.0,0.0,4.0
9067,9067,18567,4,2,5,5,5,5,3,4,...,식품공학,1,1,0,1,52.0,3500.0,2800.0,0.0,3.0


###### 5-6) 서술형 문항 제거

In [300]:
# 서술형 문항  리스트 
essay_list_2018  = [
"bq5_2",
"bq28_1",
"bq29",
"bq30",
"bq31",
"bq32",
"bq33",
'bq4_1a',
'bq37_1'   
]

In [301]:
drop_list = essay_list_2018

#  자격증 질문 
drop_list.remove('bq4_1a')
# 전공질문
drop_list.remove('bq37_1')


In [302]:
processing_test_2018 = processing_test_2018.drop(drop_list, axis = 1)

In [303]:
for column in processing_test_2018.columns:
    if (column == 'bq37_1') or (column == 'bq4_1a'):
        pass
    else:  
        if processing_test_2018[column].dtype == 'object':
            # 일단 실수형으로 바꾸고 
            processing_test_2018[column] = processing_test_2018[column].astype(float)
            # 정수로 바꿈
            processing_test_2018[column] = pd.to_numeric(processing_test_2018[column], downcast='integer')
            print('완료 : ' , column )
        elif processing_test_2018[column].dtype == 'float':
            # 정수로 바꿈
            processing_test_2018[column] = pd.to_numeric(processing_test_2018[column], downcast='integer')
            print('완료 : ' , column )


완료 :  cq17
완료 :  bq1
완료 :  bq2
완료 :  bq35
완료 :  bq40
완료 :  bq41_1
완료 :  bq41_2
완료 :  bq41_3
완료 :  bq8


# 2.3. 2019년 전처리

## 2.3.1.  2019년 Train 데이터 전처리

#### Shape
153 Columns * 8555 rows

- sq (1~16컬럼) : 성격과 관련된 16개 항목
- kq (17~82컬럼) : 지식과 관련된 33개 항목
    - kq_1 : 중요도 (1-5)
    - kq_2 : 수준 (1-7)
- bq (83~151컬럼) : 직업과 관련된 31개 항목

In [304]:
KNOW_2019 = know_train[2].copy()

In [305]:
KNOW_2019.isnull().sum()

idx         0
sq1         0
sq2         0
sq3         0
sq4         0
           ..
bq30        0
bq31_1      0
bq31_2      0
bq31_3      0
knowcode    0
Length: 151, dtype: int64

## 2.3.1.1. 밀린 데이터 처리

밀린 행 제거

In [306]:
# bq 문항 대부분이 결측치인 것
KNOW_2019 = KNOW_2019[KNOW_2019['idx'] != 19963]
KNOW_2019 = KNOW_2019[KNOW_2019['idx'] != 21180]
KNOW_2019 = KNOW_2019[KNOW_2019['idx'] != 21257]

KNOW_2019 = KNOW_2019[KNOW_2019['idx'] != 18746]
KNOW_2019 = KNOW_2019[KNOW_2019['idx'] != 19183]
KNOW_2019 = KNOW_2019[KNOW_2019['idx'] != 19871]
KNOW_2019 = KNOW_2019[KNOW_2019['idx'] != 21879]
KNOW_2019 = KNOW_2019[KNOW_2019['idx'] != 21931]
KNOW_2019 = KNOW_2019[KNOW_2019['idx'] != 25823]
KNOW_2019 = KNOW_2019[KNOW_2019['idx'] != 26330]

In [307]:
# 인덱스 reset
KNOW_2019 = KNOW_2019.reset_index(drop = True)

## 2.3.1.2. 결측치 0으로 대체

In [308]:
KNOW_2019 = KNOW_2019.replace(' ', 0)

In [309]:
KNOW_2019 = KNOW_2019.fillna(0)

In [310]:
KNOW_2019.head(3)

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,...,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3,knowcode
0,18569,4,4,4,3,4,4,4,4,4,...,기계공학과,1.0,1,0,1,40,3500,3000,0,812301
1,18570,4,3,4,4,4,4,3,4,5,...,경영학,2.0,0,5,0,40.0,0,0,5000,15201
2,18571,2,3,2,2,2,2,2,2,3,...,0,2.0,0,6,0,30,0,0,3000,901101


In [311]:
# 결측치 있는지 확인
KNOW_2019.isnull().sum().sum()

0

## 2.3.1.3. type 변환

In [312]:
# type 확인
KNOW_2019.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8545 entries, 0 to 8544
Data columns (total 151 columns):
 #    Column    Non-Null Count  Dtype  
---   ------    --------------  -----  
 0    idx       8545 non-null   int64  
 1    sq1       8545 non-null   int64  
 2    sq2       8545 non-null   int64  
 3    sq3       8545 non-null   int64  
 4    sq4       8545 non-null   int64  
 5    sq5       8545 non-null   int64  
 6    sq6       8545 non-null   int64  
 7    sq7       8545 non-null   int64  
 8    sq8       8545 non-null   int64  
 9    sq9       8545 non-null   int64  
 10   sq10      8545 non-null   int64  
 11   sq11      8545 non-null   int64  
 12   sq12      8545 non-null   int64  
 13   sq13      8545 non-null   int64  
 14   sq14      8545 non-null   int64  
 15   sq15      8545 non-null   int64  
 16   sq16      8545 non-null   int64  
 17   kq1_1     8545 non-null   int64  
 18   kq1_2     8545 non-null   int64  
 19   kq2_1     8545 non-null   int64  
 20   kq2_2 

In [313]:
# text 데이터인 컬럼 리스트화
text_columns = ['bq4_1a', 'bq5_2','bq18_10','bq20_1','bq22','bq23','bq24','bq27_1']

In [314]:
text_columns = pd.Series(text_columns)

In [315]:
KNOW_2019.loc[:,text_columns] = KNOW_2019.loc[:,text_columns].astype(str)

In [316]:
for col in range(len(KNOW_2019.columns)):
    if (text_columns.isin([KNOW_2019.columns[col]]).sum() == 1 ) == False:
        KNOW_2019[KNOW_2019.columns[col]] = pd.to_numeric(KNOW_2019[KNOW_2019.columns[col]])

In [317]:
# type float인 컬럼 int로 변환
for col in range(len(KNOW_2019.columns)):
    if KNOW_2019.iloc[:, col].dtype == 'float64':
        KNOW_2019.iloc[:,col] = KNOW_2019.iloc[:,col].astype(int)

In [318]:
KNOW_2019.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8545 entries, 0 to 8544
Data columns (total 151 columns):
 #    Column    Non-Null Count  Dtype 
---   ------    --------------  ----- 
 0    idx       8545 non-null   int64 
 1    sq1       8545 non-null   int64 
 2    sq2       8545 non-null   int64 
 3    sq3       8545 non-null   int64 
 4    sq4       8545 non-null   int64 
 5    sq5       8545 non-null   int64 
 6    sq6       8545 non-null   int64 
 7    sq7       8545 non-null   int64 
 8    sq8       8545 non-null   int64 
 9    sq9       8545 non-null   int64 
 10   sq10      8545 non-null   int64 
 11   sq11      8545 non-null   int64 
 12   sq12      8545 non-null   int64 
 13   sq13      8545 non-null   int64 
 14   sq14      8545 non-null   int64 
 15   sq15      8545 non-null   int64 
 16   sq16      8545 non-null   int64 
 17   kq1_1     8545 non-null   int64 
 18   kq1_2     8545 non-null   int64 
 19   kq2_1     8545 non-null   int64 
 20   kq2_2     8545 non-null   in

## 2.3.1.4. 도메인에서 벗어난 값이 있는지?

- sq
    - 1~5 값을 가짐
- kq
    - kq_1 1~5 값을 가짐
    - k1_2 0~7 값을 가짐
    - kq_1 = 1일 때, kq_2 = 0이어야 함.
- bq

In [319]:
# 질문 유형별 슬라이싱
sq = KNOW_2019.iloc[:, 1:17]
kq = KNOW_2019.iloc[:, 17:83]
bq = KNOW_2019.iloc[:, 83:146]

In [320]:
sq.describe()

,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,sq10,sq11,sq12,sq13,sq14,sq15,sq16
count,8545.000000,8545.000000,8545.000000,8545.000000,8545.000000,8545.000000,8545.000000,8545.000000,8545.000000,8545.000000,8545.000000,8545.000000,8545.000000,8545.000000,8545.000000,8545.000000
mean,3.756115,3.767817,3.917964,3.247513,3.737741,3.617788,3.577297,3.700410,3.770860,3.595319,3.886717,4.064833,3.712932,3.284962,2.988765,3.346167
std,0.940141,0.912060,0.891169,1.154067,0.968927,0.984173,0.978658,0.940471,0.910904,0.946646,0.916220,0.866841,1.003903,1.042693,1.154950,1.149725
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,3.000000,3.000000,3.000000,2.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,4.000000,3.000000,3.000000,2.000000,3.000000
50%,4.000000,4.000000,4.000000,3.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,3.000000,3.000000,3.000000
75%,4.000000,4.000000,5.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,5.000000,5.000000,5.000000,4.000000,4.000000,4.000000
max,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000


In [321]:
kq.describe()

,kq1_1,kq1_2,kq2_1,kq2_2,kq3_1,kq3_2,kq4_1,kq4_2,kq5_1,kq5_2,...,kq29_1,kq29_2,kq30_1,kq30_2,kq31_1,kq31_2,kq32_1,kq32_2,kq33_1,kq33_2
count,8545.000000,8545.000000,8545.000000,8545.000000,8545.000000,8545.000000,8545.000000,8545.000000,8545.000000,8545.000000,...,8545.000000,8545.000000,8545.000000,8545.000000,8545.000000,8545.000000,8545.000000,8545.000000,8545.000000,8545.000000
mean,2.093388,2.003043,2.377765,2.560913,1.904857,1.731188,2.139029,2.085313,2.840843,3.069163,...,2.456407,2.456290,2.152721,2.141018,2.150029,2.394149,2.496548,2.773318,1.626214,1.080047
std,1.154978,1.925093,1.127031,1.905005,1.079671,1.904974,1.257693,2.150658,1.312731,2.035216,...,1.299669,1.978818,1.190027,1.954691,1.118644,2.055256,1.155215,1.861104,1.029193,1.651355
min,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,...,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000
25%,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,2.000000,2.000000,...,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000
50%,2.000000,2.000000,2.000000,3.000000,2.000000,1.000000,2.000000,2.000000,3.000000,4.000000,...,2.000000,3.000000,2.000000,2.000000,2.000000,3.000000,3.000000,3.000000,1.000000,0.000000
75%,3.000000,4.000000,3.000000,4.000000,3.000000,3.000000,3.000000,4.000000,4.000000,5.000000,...,3.000000,4.000000,3.000000,4.000000,3.000000,4.000000,3.000000,4.000000,2.000000,2.000000
max,5.000000,7.000000,5.000000,7.000000,5.000000,7.000000,5.000000,7.000000,5.000000,7.000000,...,5.000000,7.000000,5.000000,7.000000,5.000000,7.000000,5.000000,7.000000,5.000000,7.000000


In [322]:
bq.describe()

,bq1,bq2,bq3,bq4,bq5,bq5_1,bq6,bq7,bq8_1,bq8_2,...,bq21_1,bq21_2,bq21_3,bq25,bq26,bq27,bq28,bq28_1,bq28_2,bq29
count,8545.000000,8545.000000,8545.000000,8545.000000,8545.000000,8545.000000,8545.000000,8545.000000,8545.000000,8545.000000,...,8545.000000,8545.000000,8545.000000,8545.000000,8545.000000,8545.000000,8545.00000,8545.000000,8545.000000,8545.000000
mean,11.338561,1.837683,2.830193,1.518198,1.466823,1.557636,4.540199,3.622118,3.531422,3.095377,...,2.443651,2.706378,2.577297,1.357753,41.757987,3.490579,1.14804,1.180573,0.834172,0.980690
std,5.706639,1.272175,1.574708,0.499698,0.498927,1.852530,1.918471,1.562929,0.858684,1.022297,...,0.996548,1.024415,0.986994,0.479367,11.206551,1.170980,0.35516,0.923348,2.013125,0.525766
min,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,18.000000,1.000000,1.00000,0.000000,0.000000,0.000000
25%,6.000000,1.000000,2.000000,1.000000,1.000000,0.000000,3.000000,2.000000,3.000000,2.000000,...,2.000000,2.000000,2.000000,1.000000,33.000000,2.000000,1.00000,1.000000,0.000000,1.000000
50%,13.000000,1.000000,2.000000,2.000000,1.000000,1.000000,5.000000,4.000000,4.000000,3.000000,...,2.000000,3.000000,3.000000,1.000000,41.000000,4.000000,1.00000,1.000000,0.000000,1.000000
75%,17.000000,2.000000,4.000000,2.000000,2.000000,3.000000,6.000000,5.000000,4.000000,4.000000,...,3.000000,3.000000,3.000000,2.000000,50.000000,4.000000,1.00000,1.000000,0.000000,1.000000
max,20.000000,6.000000,6.000000,2.000000,2.000000,6.000000,7.000000,7.000000,5.000000,5.000000,...,5.000000,5.000000,5.000000,2.000000,87.000000,6.000000,2.00000,4.000000,7.000000,2.000000


### 1) bq4 == 1인데 bq4_1a == 0인 것
- bq_4는 자격증 보유 여부로, 자격증이 있다면 bq_4_1에는 값이 있어야 함

In [323]:
KNOW_2019[(KNOW_2019['bq4'] == 1) & (KNOW_2019['bq4_1a'] == '0')]

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,...,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3,knowcode
717,19288,3,4,4,4,5,3,4,3,3,...,에너지공학,1,1,0,1,40,4800,3800,0,155103
1569,20143,4,4,4,4,5,5,5,5,5,...,관광중국어통역,1,1,0,1,40,2400,2200,0,28204
1842,20416,5,4,5,5,3,3,3,4,3,...,섬유공학,1,1,0,1,44,4500,2700,0,28401
2058,20632,3,4,5,2,3,5,2,5,5,...,전자공학,1,1,0,1,40,3500,2400,0,841201
2600,21174,5,3,4,5,4,5,2,3,3,...,실용음악,1,3,0,2,40,3600,0,0,211201
4894,23473,5,5,4,4,5,4,4,4,4,...,교육학,1,1,0,1,40,7500,6500,0,26201
5822,24401,5,5,4,4,5,4,4,4,4,...,교육학,1,1,0,1,40,7500,6500,0,26201
6062,24641,4,4,4,5,4,3,4,3,3,...,컴퓨터공학,1,1,0,1,48,4000,3000,0,134401
7555,26136,4,4,5,3,3,3,4,4,3,...,기계과,1,2,0,1,30,2800,1700,0,842101


9개 있으나 분석에 영향 X

2.3.3. bq4 == 2인데 bq4_1,2,3에 값이 있는 것
- bq_4는 자격증 보유 여부로, 자격증이 없다면 bq_4_1,2,3에 값이 없어야 함

In [324]:
KNOW_2019[(KNOW_2019['bq4'] == 2) & (KNOW_2019['bq4_1a'] != '0')]

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,...,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3,knowcode


없음!

### 2) bq5 == 2인데 bq5_1에 값이 없는 것
교육 훈련이 필요하다면 , bq_5_1에 값이 있어야 함

In [325]:
KNOW_2019[(KNOW_2019['bq5'] == 1) & (KNOW_2019['bq5_1'] == 0)]

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,...,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3,knowcode


없음!

### 3) bq5 == 2인데 bq5_1에 값이 있는 것
교육 훈련이 필요 없다면, bq_5_1에 값이 없어야 함

In [326]:
KNOW_2019[(KNOW_2019['bq5'] == 2) & (KNOW_2019['bq5_1'] != 0)]

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,...,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3,knowcode


없음!

## 2.3.1.5. 결측치 처리

### 1) Knowcode 결측치

In [327]:
KNOW_2019['knowcode'].value_counts().sort_index()

11102     15
11201     15
12101     15
12201     15
12301     15
          ..
902101    15
902201    15
903101    15
904101    15
904201    15
Name: knowcode, Length: 559, dtype: int64

없음!

### bq1, bq2 별 평균값

In [328]:
KNOW_2019_mean = round(KNOW_2019.groupby(['bq1', 'bq2']).mean().reset_index())
KNOW_2019_mean

,bq1,bq2,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,...,bq27,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3,knowcode
0,1,1,23655.0,4.0,4.0,4.0,3.0,4.0,3.0,4.0,...,2.0,1.0,1.0,3.0,1.0,46.0,1640.0,1106.0,1794.0,853678.0
1,1,2,21160.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,4.0,1.0,1.0,0.0,1.0,44.0,4229.0,2149.0,0.0,893272.0
2,1,3,22409.0,3.0,4.0,4.0,4.0,5.0,4.0,2.0,...,3.0,1.0,1.0,0.0,1.0,48.0,3000.0,2000.0,0.0,901201.0
3,1,4,22413.0,4.0,4.0,4.0,3.0,3.0,3.0,3.0,...,2.0,2.0,0.0,6.0,0.0,43.0,70.0,51.0,3085.0,902236.0
4,2,1,22252.0,4.0,4.0,4.0,3.0,4.0,4.0,3.0,...,2.0,1.0,1.0,0.0,1.0,44.0,5350.0,3611.0,0.0,705412.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,19,6,22237.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,4.0,2.0,0.0,4.0,0.0,31.0,793.0,700.0,3417.0,320369.0
77,20,1,25000.0,4.0,4.0,4.0,3.0,4.0,3.0,3.0,...,3.0,2.0,0.0,6.0,0.0,34.0,0.0,0.0,2850.0,440352.0
78,20,4,23232.0,4.0,4.0,4.0,3.0,4.0,4.0,4.0,...,4.0,1.0,3.0,1.0,2.0,38.0,2626.0,1456.0,428.0,436665.0
79,20,5,22800.0,3.0,2.0,4.0,4.0,4.0,4.0,4.0,...,4.0,1.0,3.0,0.0,2.0,24.0,1800.0,1500.0,0.0,415401.0


In [329]:
# 평균에 로직에 맞지 않는 값이 포함되지 않기 위해 0을 null값 처리

In [330]:
KNOW_2019_mean = KNOW_2019_mean.replace(0, np.NaN)
KNOW_2019_mean.head()

,bq1,bq2,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,...,bq27,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3,knowcode
0,1,1,23655.0,4.0,4.0,4.0,3.0,4.0,3.0,4.0,...,2.0,1.0,1.0,3.0,1.0,46.0,1640.0,1106.0,1794.0,853678.0
1,1,2,21160.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,4.0,1.0,1.0,NaN,1.0,44.0,4229.0,2149.0,NaN,893272.0
2,1,3,22409.0,3.0,4.0,4.0,4.0,5.0,4.0,2.0,...,3.0,1.0,1.0,NaN,1.0,48.0,3000.0,2000.0,NaN,901201.0
3,1,4,22413.0,4.0,4.0,4.0,3.0,3.0,3.0,3.0,...,2.0,2.0,NaN,6.0,NaN,43.0,70.0,51.0,3085.0,902236.0
4,2,1,22252.0,4.0,4.0,4.0,3.0,4.0,4.0,3.0,...,2.0,1.0,1.0,NaN,1.0,44.0,5350.0,3611.0,NaN,705412.0


In [331]:
KNOW_2019_mean = round(KNOW_2019.groupby(['bq1', 'bq2']).mean().reset_index())

### 2) bq20(향후 일자리변화) 결측치

In [332]:
KNOW_2019[KNOW_2019['bq20']==0]

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,...,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3,knowcode


없음

### 3) bq21_3(중장년 적합성) 결측치

In [333]:
KNOW_2019[KNOW_2019['bq21_3']==0]

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,...,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3,knowcode


없음

### 4) bq30(주당 근로시간) 결측치

In [334]:
KNOW_2019[KNOW_2019['bq30']==0]

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,...,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3,knowcode
174,18743,4,4,5,5,4,4,4,4,5,...,패션디자인섬유공학,2,0,5,0,0,0,0,0,415201
955,19526,2,4,5,2,3,2,4,4,5,...,문과,1,1,0,1,0,1800,1800,0,521201
1313,19885,4,3,4,4,4,4,4,3,3,...,이,1,1,0,1,0,5000,4000,0,702502
1447,20020,4,5,5,3,4,4,3,4,3,...,전기,1,1,0,1,0,4300,2700,0,416701
1544,20118,5,5,5,4,3,5,5,5,5,...,미디어예술과,2,0,6,0,0,0,0,5500,416205
2199,20773,5,4,5,4,4,4,2,5,4,...,고등학교,2,0,6,0,0,0,0,0,416901
2539,21113,2,2,3,2,4,2,2,2,2,...,전기공학,1,1,0,1,0,6800,4500,0,153103
4061,22639,3,4,5,4,4,5,5,3,5,...,식품생명 영양학과,1,3,0,2,0,0,0,0,416204
4078,22656,3,4,3,3,3,3,3,3,3,...,보육학,1,1,0,1,0,2160,1800,0,884201
5136,23715,4,3,4,2,1,2,3,4,4,...,문과,2,0,6,0,0,0,0,3000,615301


20건 → 산업군별 평균으로 대치

In [335]:
for i in range(len(KNOW_2019.index)):
    if KNOW_2019['bq30'][i] == 0:
        KNOW_2019['bq30'][i] = KNOW_2019_mean[KNOW_2019_mean['bq1']==KNOW_2019['bq1'][i]][KNOW_2019_mean['bq2']==KNOW_2019['bq2'][i]]['bq30']

In [336]:
KNOW_2019[KNOW_2019['bq30']==0]

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,...,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3,knowcode


### 5) bq31 결측치

### case1. 임금근로자 연봉 결측치

In [337]:
KNOW_2019[KNOW_2019['bq28']==1][KNOW_2019['bq31_1']==0]

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,...,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3,knowcode
12,18581,5,5,5,5,5,4,3,5,5,...,건축,1,1,0,1,50,0,2800,0,159103
25,18594,4,5,5,4,5,5,4,5,5,...,상업,1,3,0,2,30,0,0,0,532101
43,18612,4,4,4,4,4,3,4,3,5,...,0,1,1,0,1,50,0,0,0,13902
73,18642,5,5,5,4,5,4,4,5,5,...,기계공학과,1,1,0,1,60,0,4000,0,151107
171,18740,5,5,5,4,5,5,5,5,5,...,의예과,1,1,0,1,50,0,0,0,301108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8371,26953,4,4,5,3,4,4,4,4,3,...,패션디자인과,1,1,0,1,40,0,5000,0,211101
8421,27003,5,4,5,5,5,4,5,3,5,...,신방,1,1,0,1,50,0,0,0,110203
8457,27039,5,3,4,4,4,4,4,3,4,...,사회학,1,3,0,1,50,0,0,0,24203
8475,27057,4,3,3,4,4,3,3,3,2,...,국제통상학,1,3,0,1,50,0,0,0,13305


In [338]:
for i in range(len(KNOW_2019.index)):
    if KNOW_2019['bq28'][i] == 1:
        if KNOW_2019['bq31_1'][i] == 0:
            KNOW_2019['bq31_1'][i] = KNOW_2019_mean[KNOW_2019_mean['bq1']==KNOW_2019['bq1'][i]][KNOW_2019_mean['bq2']==KNOW_2019['bq2'][i]]['bq31_1']

In [339]:
KNOW_2019[KNOW_2019['bq28']==1][KNOW_2019['bq31_1']==0]

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,...,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3,knowcode


In [340]:
KNOW_2019[KNOW_2019['bq28']==1][KNOW_2019['bq31_2']==0]

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,...,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3,knowcode
25,18594,4,5,5,4,5,5,4,5,5,...,상업,1,3,0,2,30,2681,0,0,532101
43,18612,4,4,4,4,4,3,4,3,5,...,0,1,1,0,1,50,3572,0,0,13902
87,18656,4,4,5,4,5,4,5,4,3,...,실업계,1,4,0,2,40,3000,0,0,885901
171,18740,5,5,5,4,5,5,5,5,5,...,의예과,1,1,0,1,50,4940,0,0,301108
199,18769,5,4,4,5,5,4,5,5,5,...,경영학,1,1,0,1,50,5421,0,0,11201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8364,26946,3,3,2,1,3,1,4,3,4,...,법학,1,4,0,1,48,5000,0,0,702301
8421,27003,5,4,5,5,5,4,5,3,5,...,신방,1,1,0,1,50,5299,0,0,110203
8457,27039,5,3,4,4,4,4,4,3,4,...,사회학,1,3,0,1,50,3572,0,0,24203
8475,27057,4,3,3,4,4,3,3,3,2,...,국제통상학,1,3,0,1,50,4994,0,0,13305


In [341]:
for i in range(len(KNOW_2019.index)):
    if KNOW_2019['bq28'][i] == 1:
        if KNOW_2019['bq31_2'][i] == 0:
            KNOW_2019['bq31_2'][i] = KNOW_2019_mean[KNOW_2019_mean['bq1']==KNOW_2019['bq1'][i]][KNOW_2019_mean['bq2']==KNOW_2019['bq2'][i]]['bq31_2']

In [342]:
KNOW_2019[KNOW_2019['bq28']==1][KNOW_2019['bq31_2']==0]

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,...,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3,knowcode


### case2. 비임금근로자 연봉 결측치

In [343]:
KNOW_2019[KNOW_2019['bq28']==2][KNOW_2019['bq31_3']==0]

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,...,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3,knowcode
52,18621,1,3,3,1,3,2,3,3,3,...,0,2,0,7,0,60,0,0,0,901101
174,18743,4,4,5,5,4,4,4,4,5,...,패션디자인섬유공학,2,0,5,0,45,0,0,0,415201
366,18936,4,4,4,2,3,3,3,3,3,...,체육학,2,0,6,0,30,0,0,0,902201
963,19534,5,3,5,5,3,3,3,3,3,...,컴퓨터공학과,2,0,5,0,52,0,0,0,415201
1058,19629,3,3,3,3,3,3,3,3,3,...,사회복지,2,0,7,0,20,0,0,0,902201
1218,19789,2,2,2,1,2,2,1,2,2,...,0,2,0,7,0,60,0,0,0,901101
1242,19813,2,2,2,1,2,2,2,1,2,...,0,2,0,7,0,32,0,0,0,901101
1254,19825,4,4,4,4,4,4,4,4,4,...,연극과,2,0,6,0,20,0,0,0,416204
1810,20384,5,5,5,3,3,4,3,3,3,...,약학,2,0,7,0,40,0,0,0,303001
1881,20455,3,3,2,1,2,2,2,3,2,...,0,2,0,7,0,40,0,0,0,812902


In [344]:
for i in range(len(KNOW_2019.index)):
    if KNOW_2019['bq28'][i] == 2:
        if KNOW_2019['bq31_3'][i] == 0:
            KNOW_2019['bq31_3'][i] = KNOW_2019_mean[KNOW_2019_mean['bq1']==KNOW_2019['bq1'][i]][KNOW_2019_mean['bq2']==KNOW_2019['bq2'][i]]['bq31_3']

In [345]:
KNOW_2019[KNOW_2019['bq28']==2][KNOW_2019['bq31_3']==0]

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,...,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3,knowcode
5478,24057,5,5,5,5,5,5,5,5,5,...,연극영화과,2,0,6,0,36,0,0,0,416204
8146,26728,3,3,3,3,3,3,3,3,3,...,연영과,2,0,6,0,20,0,0,0,416204


## 2.3.1.6. 컬럼 줄이기

### 1) 불필요한 컬럼 drop

In [346]:
# 자격증, 학과를 제외한 텍스트 데이터 컬럼 삭제
KNOW_2019 = KNOW_2019.drop(['bq5_2','bq18_10','bq20_1','bq22','bq23','bq24'], axis = 1)

In [347]:
# kq_2 컬럼 삭제
for i in range(1, 34):
    KNOW_2019 = KNOW_2019.drop(['kq'+str(i)+'_2'], axis=1)

In [348]:
KNOW_2019.head()

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,...,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3,knowcode
0,18569,4,4,4,3,4,4,4,4,4,...,기계공학과,1,1,0,1,40,3500,3000,0,812301
1,18570,4,3,4,4,4,4,3,4,5,...,경영학,2,0,5,0,40,0,0,5000,15201
2,18571,2,3,2,2,2,2,2,2,3,...,0,2,0,6,0,30,0,0,3000,901101
3,18572,5,5,5,5,5,5,5,5,5,...,행정학,1,1,0,1,40,3600,2400,0,29903
4,18573,1,4,4,1,1,2,4,3,4,...,0,2,0,6,0,45,0,0,1500,561401


### 2) 평균값을 사용해 컬럼 줄이기

In [349]:
# bq8_1, bq8_2, bq8_3 통합하여 bq8 만들기
KNOW_2019['bq8_1'] = round((KNOW_2019['bq8_1']+KNOW_2019['bq8_2']+KNOW_2019['bq8_3'])/3)
KNOW_2019['bq8_1'] = KNOW_2019['bq8_1'].astype(int)
KNOW_2019 = KNOW_2019.rename({'bq8_1':'bq8'}, axis = 1 )
KNOW_2019 = KNOW_2019.drop(['bq8_2','bq8_3'], axis = 1)
KNOW_2019.head(3)

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,...,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3,knowcode
0,18569,4,4,4,3,4,4,4,4,4,...,기계공학과,1,1,0,1,40,3500,3000,0,812301
1,18570,4,3,4,4,4,4,3,4,5,...,경영학,2,0,5,0,40,0,0,5000,15201
2,18571,2,3,2,2,2,2,2,2,3,...,0,2,0,6,0,30,0,0,3000,901101


In [350]:
# bq12_5만 사용
KNOW_2019 = KNOW_2019.rename({'bq12_5':'bq12'}, axis = 1 )
KNOW_2019 = KNOW_2019.drop(['bq12_1','bq12_2','bq12_3','bq12_4'], axis = 1)

In [351]:
# bq13_1, bq13_2, bq13_3 통합하여 bq13 만들기
KNOW_2019['bq13_1'] = round((KNOW_2019['bq13_1']+KNOW_2019['bq13_2']+KNOW_2019['bq13_3'])/3)
KNOW_2019['bq13_1'] = KNOW_2019['bq13_1'].astype(int)
KNOW_2019 = KNOW_2019.rename({'bq13_1':'bq13'}, axis = 1 )
KNOW_2019 = KNOW_2019.drop(['bq13_2','bq13_3'], axis = 1)

In [352]:
# bq14_1, bq14_2, bq14_3, bq14_4, bq14_5 통합하여 bq14 만들기
KNOW_2019['bq14_1'] = round((KNOW_2019['bq14_1']+KNOW_2019['bq14_2']+KNOW_2019['bq14_3']+KNOW_2019['bq14_4']+KNOW_2019['bq14_5'])/5)
KNOW_2019['bq14_1'] = KNOW_2019['bq14_1'].astype(int)
KNOW_2019 = KNOW_2019.rename({'bq14_1':'bq14'}, axis = 1 )
KNOW_2019 = KNOW_2019.drop(['bq14_2','bq14_3','bq14_4','bq14_5'], axis = 1)

In [353]:
# bq16_1, bq16_2, bq16_3, bq16_4, bq16_5 통합하여 bq16 만들기
KNOW_2019['bq16_1'] = round((KNOW_2019['bq16_1']+KNOW_2019['bq16_2']+KNOW_2019['bq16_3']+KNOW_2019['bq16_4']+KNOW_2019['bq16_5'])/5)
KNOW_2019['bq16_1'] = KNOW_2019['bq16_1'].astype(int)
KNOW_2019 = KNOW_2019.rename({'bq16_1':'bq16'}, axis = 1 )
KNOW_2019 = KNOW_2019.drop(['bq16_2','bq16_3','bq16_4','bq16_5'], axis = 1)

In [354]:
# bq21_1, bq21_2, bq21_3 통합하여 bq21 만들기
KNOW_2019['bq21_1'] = round((KNOW_2019['bq21_1']+KNOW_2019['bq21_2']+KNOW_2019['bq21_3'])/3)
KNOW_2019['bq21_1'] = KNOW_2019['bq21_1'].astype(int)
KNOW_2019 = KNOW_2019.rename({'bq21_1':'bq21'}, axis = 1 )
KNOW_2019 = KNOW_2019.drop(['bq21_2','bq21_3'], axis = 1)

In [355]:
KNOW_2019.head(2)

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,...,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3,knowcode
0,18569,4,4,4,3,4,4,4,4,4,...,기계공학과,1,1,0,1,40,3500,3000,0,812301
1,18570,4,3,4,4,4,4,3,4,5,...,경영학,2,0,5,0,40,0,0,5000,15201


In [356]:
# 전공을 제외한 나머지 데이터 유형 인트형으로 변환
for col in range(len(KNOW_2019.columns)):
    if (KNOW_2019.columns[col] == 'bq27_1') or (KNOW_2019.columns[col] == 'bq4_1a')  :
        pass
    else:
        KNOW_2019.iloc[:,col] = KNOW_2019.iloc[:,col].astype(int)

In [357]:
processing_train_2019 = KNOW_2019

## 2.3.2.  2019년 Test 데이터 전처리

In [358]:
KNOW_2019_test = know_test[2].copy()
KNOW_2019_test.head(3)

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,...,bq27,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3
0,18569,5,5,5,4,4,5,4,5,5,...,4,경영,1,1,0,1,45,6700,3800,0
1,18570,2,3,3,1,3,2,3,3,3,...,3,화학,1,1,0,1,44,2700,2500,0
2,18571,5,4,5,4,4,4,4,5,4,...,4,건축공학,1,1,0,1,40,6000,3200,0


In [359]:
KNOW_2019_test.isnull().sum()

idx       0
sq1       0
sq2       0
sq3       0
sq4       0
         ..
bq29      0
bq30      0
bq31_1    0
bq31_2    0
bq31_3    0
Length: 150, dtype: int64

## 2.3.2.1. 불필요한 컬럼 먼저 삭제

In [360]:
# 자격증, 학과를 제외한 텍스트 데이터 컬럼 삭제
KNOW_2019_test = KNOW_2019_test.drop(['bq5_2','bq18_10','bq20_1','bq22','bq23','bq24'], axis = 1)
KNOW_2019_test.head(3)

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,...,bq27,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3
0,18569,5,5,5,4,4,5,4,5,5,...,4,경영,1,1,0,1,45,6700,3800,0
1,18570,2,3,3,1,3,2,3,3,3,...,3,화학,1,1,0,1,44,2700,2500,0
2,18571,5,4,5,4,4,4,4,5,4,...,4,건축공학,1,1,0,1,40,6000,3200,0


## 2.3.2.2. 밀린 데이터 처리

In [361]:
KNOW_2019_test.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8554 entries, 0 to 8553
Data columns (total 144 columns):
 #    Column  Non-Null Count  Dtype  
---   ------  --------------  -----  
 0    idx     8554 non-null   int64  
 1    sq1     8554 non-null   int64  
 2    sq2     8554 non-null   int64  
 3    sq3     8554 non-null   int64  
 4    sq4     8554 non-null   int64  
 5    sq5     8554 non-null   int64  
 6    sq6     8554 non-null   int64  
 7    sq7     8554 non-null   int64  
 8    sq8     8554 non-null   int64  
 9    sq9     8554 non-null   int64  
 10   sq10    8554 non-null   int64  
 11   sq11    8554 non-null   int64  
 12   sq12    8554 non-null   int64  
 13   sq13    8554 non-null   int64  
 14   sq14    8554 non-null   int64  
 15   sq15    8554 non-null   int64  
 16   sq16    8554 non-null   int64  
 17   kq1_1   8554 non-null   int64  
 18   kq1_2   8554 non-null   int64  
 19   kq2_1   8554 non-null   int64  
 20   kq2_2   8554 non-null   int64  
 21   kq3_1   8554

In [362]:
KNOW_2019_test['bq19'].value_counts()

2                                       3640
1                                       3244
3                                       1014
4                                        231
2                                        168
1                                        137
5                                         61
3                                         42
4                                         13
5                                          2
 독립 애니메이션 감독                               1
 역할만 바뀌었지 영상의 영역을 벗어나기는 힘들 것으로 보인다."       1
Name: bq19, dtype: int64

In [363]:
for i in range(len(KNOW_2019_test.index)):
    if KNOW_2019_test['idx'][i]==25663:
        KNOW_2019_test.loc[i,'bq19':'bq27_1'] = np.NaN

In [364]:
KNOW_2019_test[KNOW_2019_test['idx']==25663]

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,...,bq27,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3
7090,25663,4,5,5,5,4,4,4,4,4,...,NaN,NaN,0,0,0,0,0,0,0,0


In [365]:
for i in range(len(KNOW_2019_test.index)):
    if KNOW_2019_test['idx'][i]==22266:
        KNOW_2019_test.loc[i,'bq19':'bq28_1'] = np.NaN

In [366]:
KNOW_2019_test[KNOW_2019_test['idx']==22266]

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,...,bq27,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3
3694,22266,4,4,5,5,5,3,4,4,5,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0


In [367]:
for i in range(len(KNOW_2019_test.index)):
    if KNOW_2019_test['idx'][i]==24505:
        KNOW_2019_test.loc[i,'bq20':'bq31_1'] = np.NaN

In [368]:
KNOW_2019_test[KNOW_2019_test['idx']==24505]

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,...,bq27,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3
5933,24505,5,5,5,5,5,5,4,4,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0


In [369]:
for i in range(len(KNOW_2019_test.index)):
    if KNOW_2019_test['idx'][i]==21769:
        KNOW_2019_test.loc[i,'bq25':'bq29'] = np.NaN

In [370]:
KNOW_2019_test[KNOW_2019_test['idx']==21769]

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,...,bq27,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3
3197,21769,3,4,4,2,4,3,2,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,3500,0


## 2.3.2.3. 결측치 0으로 대체

In [371]:
KNOW_2019_test = KNOW_2019_test.replace(' ', 0)

In [372]:
KNOW_2019_test = KNOW_2019_test.fillna(0)

In [373]:
KNOW_2019_test.head(3)

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,...,bq27,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3
0,18569,5,5,5,4,4,5,4,5,5,...,4,경영,1,1,0,1,45,6700,3800,0
1,18570,2,3,3,1,3,2,3,3,3,...,3,화학,1,1,0,1,44,2700,2500,0
2,18571,5,4,5,4,4,4,4,5,4,...,4,건축공학,1,1,0,1,40,6000,3200,0


In [374]:
KNOW_2019_test.isnull().sum().sum()

0

## 2.3.2.4. type 변환

In [375]:
KNOW_2019_test.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8554 entries, 0 to 8553
Data columns (total 144 columns):
 #    Column  Non-Null Count  Dtype  
---   ------  --------------  -----  
 0    idx     8554 non-null   int64  
 1    sq1     8554 non-null   int64  
 2    sq2     8554 non-null   int64  
 3    sq3     8554 non-null   int64  
 4    sq4     8554 non-null   int64  
 5    sq5     8554 non-null   int64  
 6    sq6     8554 non-null   int64  
 7    sq7     8554 non-null   int64  
 8    sq8     8554 non-null   int64  
 9    sq9     8554 non-null   int64  
 10   sq10    8554 non-null   int64  
 11   sq11    8554 non-null   int64  
 12   sq12    8554 non-null   int64  
 13   sq13    8554 non-null   int64  
 14   sq14    8554 non-null   int64  
 15   sq15    8554 non-null   int64  
 16   sq16    8554 non-null   int64  
 17   kq1_1   8554 non-null   int64  
 18   kq1_2   8554 non-null   int64  
 19   kq2_1   8554 non-null   int64  
 20   kq2_2   8554 non-null   int64  
 21   kq3_1   8554

In [376]:
text_columns = ['bq4_1a','bq4_1b','bq4_1c', 'bq5_2','bq18_10','bq20_1','bq22','bq23','bq24','bq27_1']

In [377]:
text_columns = pd.Series(text_columns)

In [378]:
# KNOW_2019_test.loc[:,text_columns] = KNOW_2019_test.loc[:,text_columns].astype(str)

In [379]:
for col in range(len(KNOW_2019_test.columns)):
    if (text_columns.isin([KNOW_2019_test.columns[col]]).sum() == 1 ) == False:
        KNOW_2019_test[KNOW_2019_test.columns[col]] = pd.to_numeric(KNOW_2019_test[KNOW_2019_test.columns[col]])

In [380]:
for col in range(len(KNOW_2019_test.columns)):
    if KNOW_2019_test.iloc[:, col].dtype == 'float64':
        KNOW_2019_test.iloc[:,col] = KNOW_2019_test.iloc[:,col].astype(int)

In [381]:
KNOW_2019_test.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8554 entries, 0 to 8553
Data columns (total 144 columns):
 #    Column  Non-Null Count  Dtype 
---   ------  --------------  ----- 
 0    idx     8554 non-null   int64 
 1    sq1     8554 non-null   int64 
 2    sq2     8554 non-null   int64 
 3    sq3     8554 non-null   int64 
 4    sq4     8554 non-null   int64 
 5    sq5     8554 non-null   int64 
 6    sq6     8554 non-null   int64 
 7    sq7     8554 non-null   int64 
 8    sq8     8554 non-null   int64 
 9    sq9     8554 non-null   int64 
 10   sq10    8554 non-null   int64 
 11   sq11    8554 non-null   int64 
 12   sq12    8554 non-null   int64 
 13   sq13    8554 non-null   int64 
 14   sq14    8554 non-null   int64 
 15   sq15    8554 non-null   int64 
 16   sq16    8554 non-null   int64 
 17   kq1_1   8554 non-null   int64 
 18   kq1_2   8554 non-null   int64 
 19   kq2_1   8554 non-null   int64 
 20   kq2_2   8554 non-null   int64 
 21   kq3_1   8554 non-null   int64 
 22 

## 2.3.2.5. 도메인에서 벗어난 값이 있는지?

- sq
    - 1~5 값을 가짐
- kq
    - kq_1 1~5 값을 가짐
    - k1_2 0~7 값을 가짐
    - kq_1 = 1일 때, kq_2 = 0이어야 함.
- bq

In [382]:
# 질문 유형별 슬라이싱
sq = KNOW_2019_test.iloc[:, 1:17]
kq = KNOW_2019_test.iloc[:, 17:83]
bq = KNOW_2019_test.iloc[:, 83:]

In [383]:
sq.describe()

,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,sq10,sq11,sq12,sq13,sq14,sq15,sq16
count,8554.000000,8554.000000,8554.000000,8554.000000,8554.000000,8554.000000,8554.000000,8554.000000,8554.000000,8554.000000,8554.000000,8554.000000,8554.000000,8554.000000,8554.000000,8554.000000
mean,3.750760,3.783025,3.911270,3.230652,3.732874,3.616086,3.565233,3.701426,3.771803,3.604045,3.887889,4.087912,3.704115,3.309563,2.992518,3.371639
std,0.941193,0.913498,0.892677,1.169286,0.972714,0.988607,0.986054,0.939876,0.905609,0.946351,0.917851,0.856328,0.993850,1.036811,1.150721,1.153564
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,3.000000,3.000000,3.000000,2.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,4.000000,3.000000,3.000000,2.000000,3.000000
50%,4.000000,4.000000,4.000000,3.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,3.000000,3.000000,3.000000
75%,4.000000,4.000000,5.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,5.000000,5.000000,4.000000,4.000000,4.000000,4.000000
max,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000


In [384]:
kq.describe()

,kq1_1,kq1_2,kq2_1,kq2_2,kq3_1,kq3_2,kq4_1,kq4_2,kq5_1,kq5_2,...,kq29_1,kq29_2,kq30_1,kq30_2,kq31_1,kq31_2,kq32_1,kq32_2,kq33_1,kq33_2
count,8554.000000,8554.000000,8554.000000,8554.000000,8554.000000,8554.000000,8554.000000,8554.000000,8554.000000,8554.000000,...,8554.000000,8554.000000,8554.000000,8554.000000,8554.000000,8554.000000,8554.000000,8554.000000,8554.000000,8554.000000
mean,2.099486,2.021627,2.419102,2.610124,1.916063,1.742343,2.136778,2.087678,2.832125,3.077975,...,2.448679,2.438041,2.171499,2.166706,2.174538,2.466098,2.511223,2.812252,1.631401,1.106968
std,1.152691,1.925874,1.137487,1.888595,1.080623,1.897346,1.257363,2.159550,1.305849,2.026511,...,1.296551,1.971493,1.196931,1.959853,1.111803,2.052243,1.158311,1.866918,1.028803,1.676487
min,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,...,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000
25%,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,2.000000,2.000000,...,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000
50%,2.000000,2.000000,2.000000,3.000000,2.000000,2.000000,2.000000,2.000000,3.000000,4.000000,...,2.000000,3.000000,2.000000,2.000000,2.000000,3.000000,3.000000,3.000000,1.000000,0.000000
75%,3.000000,4.000000,3.000000,4.000000,3.000000,3.000000,3.000000,4.000000,4.000000,5.000000,...,3.000000,4.000000,3.000000,4.000000,3.000000,4.000000,3.000000,4.000000,2.000000,2.000000
max,5.000000,7.000000,5.000000,7.000000,5.000000,7.000000,5.000000,7.000000,5.000000,7.000000,...,5.000000,7.000000,5.000000,7.000000,5.000000,7.000000,5.000000,7.000000,5.000000,7.000000


In [385]:
bq.describe()

,bq1,bq2,bq3,bq4,bq5,bq5_1,bq6,bq7,bq8_1,bq8_2,...,bq26,bq27,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3
count,8554.000000,8554.000000,8554.000000,8554.000000,8554.000000,8554.000000,8554.000000,8554.000000,8554.000000,8554.000000,...,8554.000000,8554.000000,8554.000000,8554.000000,8554.000000,8554.000000,8554.000000,8554.000000,8554.000000,8554.000000
mean,11.326865,1.841478,2.838555,1.517536,1.475333,1.523732,4.485971,3.618892,3.537059,3.115852,...,41.631634,3.520575,1.142857,1.194996,0.803601,0.989712,44.424129,3552.659808,2334.293430,647.815057
std,5.680759,1.269628,1.579050,0.499722,0.499420,1.845350,1.924168,1.565947,0.851320,1.018259,...,11.245526,1.179390,0.351281,0.933342,1.976716,0.526520,9.196894,2588.473528,1551.237114,2247.085408
min,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,6.000000,1.000000,2.000000,1.000000,1.000000,0.000000,3.000000,2.000000,3.000000,2.000000,...,33.000000,3.000000,1.000000,1.000000,0.000000,1.000000,40.000000,2400.000000,1800.000000,0.000000
50%,13.000000,1.000000,2.000000,2.000000,1.000000,1.000000,5.000000,4.000000,4.000000,3.000000,...,40.000000,4.000000,1.000000,1.000000,0.000000,1.000000,43.000000,3350.000000,2400.000000,0.000000
75%,17.000000,2.000000,4.000000,2.000000,2.000000,3.000000,6.000000,5.000000,4.000000,4.000000,...,50.000000,4.000000,1.000000,1.000000,0.000000,1.000000,50.000000,4500.000000,3000.000000,0.000000
max,21.000000,6.000000,6.000000,2.000000,2.000000,6.000000,7.000000,7.000000,5.000000,5.000000,...,85.000000,6.000000,2.000000,4.000000,7.000000,2.000000,90.000000,37000.000000,21000.000000,43000.000000


### 1) bq4 == 1인데 bq4_1a == 0인 것
- bq_4는 자격증 보유 여부로, 자격증이 있다면 bq_4_1에는 값이 있어야 함

In [386]:
KNOW_2019_test[(KNOW_2019_test['bq4'] == 1) & (KNOW_2019_test['bq4_1a'] == 0)]

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,...,bq27,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3


분석에 영향 x

### 2) bq4 == 2인데 bq4_1,2,3에 값이 있는 것
- bq_4는 자격증 보유 여부로, 자격증이 없다면 bq_4_1,2,3에 값이 없어야 함

In [387]:
KNOW_2019_test[(KNOW_2019_test['bq4'] == 2) & (KNOW_2019_test['bq4_1a'] != 0)]

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,...,bq27,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3
1,18570,2,3,3,1,3,2,3,3,3,...,3,화학,1,1,0,1,44,2700,2500,0
4,18573,5,4,5,3,3,3,3,4,5,...,6,기계공학과,1,1,0,1,50,7000,5000,0
5,18574,5,4,4,5,3,3,5,4,4,...,4,경영학,1,1,0,1,70,10000,7000,0
7,18576,3,3,3,3,2,2,3,3,3,...,2,기계과,1,1,0,1,52,7800,3200,0
10,18579,4,3,4,3,3,4,4,3,4,...,2,문과,1,3,0,2,45,2300,1300,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8548,27121,2,3,2,1,3,1,3,2,2,...,2,기계,1,1,0,1,40,3800,2900,0
8550,27123,4,4,3,4,3,4,4,3,3,...,4,조소학과,1,3,0,2,48,2800,1650,0
8551,27124,3,4,4,2,2,2,3,4,4,...,3,컴퓨터공학,1,1,0,1,40,3500,2300,0
8552,27125,4,4,3,3,4,3,4,3,4,...,4,조소학과,1,1,0,1,48,2800,1900,0


없음!

### 3) bq5 == 2인데 bq5_1에 값이 없는 것
교육 훈련이 필요하다면 , bq_5_1에 값이 있어야 함

In [388]:
KNOW_2019_test[(KNOW_2019_test['bq5'] == 1) & (KNOW_2019_test['bq5_1'] == 0)]

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,...,bq27,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3


없음!

### 4) bq5 == 2인데 bq5_1에 값이 있는 것
교육 훈련이 필요 없다면, bq_5_1에 값이 없어야 함

In [389]:
KNOW_2019_test[(KNOW_2019_test['bq5'] == 2) & (KNOW_2019_test['bq5_1'] != 0)]

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,...,bq27,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3


없음!

## 2.3.2.6. 결측치 처리

### bq1, bq2 별 평균값

In [390]:
KNOW_2019_test_mean = round(KNOW_2019_test.groupby(['bq1', 'bq2']).mean().reset_index())
KNOW_2019_test_mean

,bq1,bq2,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,...,bq26,bq27,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3
0,1,1,23311.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,49.0,3.0,1.0,1.0,3.0,1.0,46.0,1752.0,1168.0,1629.0
1,1,2,24100.0,4.0,4.0,4.0,3.0,4.0,4.0,4.0,...,42.0,4.0,1.0,1.0,0.0,1.0,43.0,4017.0,2150.0,0.0
2,1,4,22956.0,4.0,4.0,4.0,2.0,3.0,3.0,3.0,...,59.0,2.0,2.0,0.0,6.0,0.0,44.0,102.0,76.0,3257.0
3,2,1,22632.0,3.0,3.0,4.0,3.0,4.0,3.0,3.0,...,49.0,3.0,1.0,1.0,0.0,1.0,46.0,5306.0,3589.0,0.0
4,2,2,20000.0,3.0,3.0,3.0,3.0,4.0,3.0,2.0,...,46.0,2.0,1.0,1.0,0.0,1.0,40.0,5000.0,4200.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,20,1,23560.0,4.0,4.0,4.0,2.0,4.0,4.0,4.0,...,32.0,2.0,2.0,1.0,4.0,1.0,28.0,833.0,667.0,3000.0
73,20,4,23119.0,4.0,4.0,4.0,3.0,4.0,3.0,3.0,...,42.0,4.0,1.0,3.0,1.0,2.0,38.0,2510.0,1372.0,368.0
74,20,6,19723.0,4.0,3.0,2.0,2.0,2.0,2.0,2.0,...,44.0,4.0,2.0,0.0,6.0,0.0,19.0,0.0,0.0,1600.0
75,21,1,23297.0,4.0,4.0,4.0,4.0,4.0,3.0,4.0,...,45.0,4.0,1.0,2.0,0.0,1.0,45.0,6932.0,3025.0,0.0


In [391]:
# 평균에 로직에 맞지 않는 값이 포함되지 않기 위해 0을 null값 처리

In [392]:
KNOW_2019_test_mean = KNOW_2019_test_mean.replace(0, np.NaN)
KNOW_2019_test_mean.head()

,bq1,bq2,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,...,bq26,bq27,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3
0,1,1,23311.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,49.0,3.0,1.0,1.0,3.0,1.0,46.0,1752.0,1168.0,1629.0
1,1,2,24100.0,4.0,4.0,4.0,3.0,4.0,4.0,4.0,...,42.0,4.0,1.0,1.0,NaN,1.0,43.0,4017.0,2150.0,NaN
2,1,4,22956.0,4.0,4.0,4.0,2.0,3.0,3.0,3.0,...,59.0,2.0,2.0,NaN,6.0,NaN,44.0,102.0,76.0,3257.0
3,2,1,22632.0,3.0,3.0,4.0,3.0,4.0,3.0,3.0,...,49.0,3.0,1.0,1.0,NaN,1.0,46.0,5306.0,3589.0,NaN
4,2,2,20000.0,3.0,3.0,3.0,3.0,4.0,3.0,2.0,...,46.0,2.0,1.0,1.0,NaN,1.0,40.0,5000.0,4200.0,NaN


In [393]:
KNOW_2019_test_mean = round(KNOW_2019_test.groupby(['bq1', 'bq2']).mean().reset_index())

### 1) bq19 결측치

In [394]:
KNOW_2019_test[KNOW_2019_test['bq19']==0]

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,...,bq27,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3
3694,22266,4,4,5,5,5,3,4,4,5,...,0,0,0,0,0,0,0,0,0,0
7090,25663,4,5,5,5,4,4,4,4,4,...,0,0,0,0,0,0,0,0,0,0


In [395]:
for i in range(len(KNOW_2019_test.index)):
    if KNOW_2019_test['bq19'][i] == 0:
        KNOW_2019_test['bq19'][i] = KNOW_2019_test_mean[KNOW_2019_test_mean['bq1']==KNOW_2019_test['bq1'][i]][KNOW_2019_test_mean['bq2']==KNOW_2019_test['bq2'][i]]['bq19']

In [396]:
KNOW_2019_test[KNOW_2019_test['bq19']==0]

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,...,bq27,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3


### 2) bq20(향후 일자리변화) 결측치

In [397]:
KNOW_2019_test[KNOW_2019_test['bq20']==0]

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,...,bq27,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3
3694,22266,4,4,5,5,5,3,4,4,5,...,0,0,0,0,0,0,0,0,0,0
5933,24505,5,5,5,5,5,5,4,4,3,...,0,0,0,0,0,0,0,0,0,0
7090,25663,4,5,5,5,4,4,4,4,4,...,0,0,0,0,0,0,0,0,0,0


In [398]:
for i in range(len(KNOW_2019_test.index)):
    if KNOW_2019_test['bq20'][i] == 0:
        KNOW_2019_test['bq20'][i] = KNOW_2019_test_mean[KNOW_2019_test_mean['bq1']==KNOW_2019_test['bq1'][i]][KNOW_2019_test_mean['bq2']==KNOW_2019_test['bq2'][i]]['bq20']

In [399]:
KNOW_2019_test[KNOW_2019_test['bq20']==0]

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,...,bq27,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3


### 3) bq21 결측치

In [400]:
KNOW_2019_test[KNOW_2019_test['bq21_1']==0]

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,...,bq27,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3
3694,22266,4,4,5,5,5,3,4,4,5,...,0,0,0,0,0,0,0,0,0,0
5933,24505,5,5,5,5,5,5,4,4,3,...,0,0,0,0,0,0,0,0,0,0
7090,25663,4,5,5,5,4,4,4,4,4,...,0,0,0,0,0,0,0,0,0,0


In [401]:
for i in range(len(KNOW_2019_test.index)):
    if KNOW_2019_test['bq21_1'][i] == 0:
        KNOW_2019_test['bq21_1'][i] = KNOW_2019_test_mean[KNOW_2019_test_mean['bq1']==KNOW_2019_test['bq1'][i]][KNOW_2019_test_mean['bq2']==KNOW_2019_test['bq2'][i]]['bq21_1']

In [402]:
KNOW_2019_test[KNOW_2019_test['bq21_1']==0]

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,...,bq27,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3


In [403]:
for i in range(len(KNOW_2019_test.index)):
    if KNOW_2019_test['bq21_2'][i] == 0:
        KNOW_2019_test['bq21_2'][i] = KNOW_2019_test_mean[KNOW_2019_test_mean['bq1']==KNOW_2019_test['bq1'][i]][KNOW_2019_test_mean['bq2']==KNOW_2019_test['bq2'][i]]['bq21_2']

In [404]:
KNOW_2019_test[KNOW_2019_test['bq21_2']==0]

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,...,bq27,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3


In [405]:
for i in range(len(KNOW_2019_test.index)):
    if KNOW_2019_test['bq21_3'][i] == 0:
        KNOW_2019_test['bq21_3'][i] = KNOW_2019_test_mean[KNOW_2019_test_mean['bq1']==KNOW_2019_test['bq1'][i]][KNOW_2019_test_mean['bq2']==KNOW_2019_test['bq2'][i]]['bq21_3']

In [406]:
KNOW_2019_test[KNOW_2019_test['bq21_3']==0]

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,...,bq27,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3


### 4) bq25 결측치

In [407]:
for i in range(len(KNOW_2019_test.index)):
    if KNOW_2019_test['bq25'][i] == 0:
        KNOW_2019_test['bq25'][i] = KNOW_2019_test_mean[KNOW_2019_test_mean['bq1']==KNOW_2019_test['bq1'][i]][KNOW_2019_test_mean['bq2']==KNOW_2019_test['bq2'][i]]['bq25']

### 5) bq26 결측치

In [408]:
for i in range(len(KNOW_2019_test.index)):
    if KNOW_2019_test['bq26'][i] == 0:
        KNOW_2019_test['bq26'][i] = KNOW_2019_test_mean[KNOW_2019_test_mean['bq1']==KNOW_2019_test['bq1'][i]][KNOW_2019_test_mean['bq2']==KNOW_2019_test['bq2'][i]]['bq26']

### 6) bq27 결측치

In [409]:
for i in range(len(KNOW_2019_test.index)):
    if KNOW_2019_test['bq27'][i] == 0:
        KNOW_2019_test['bq27'][i] = KNOW_2019_test_mean[KNOW_2019_test_mean['bq1']==KNOW_2019_test['bq1'][i]][KNOW_2019_test_mean['bq2']==KNOW_2019_test['bq2'][i]]['bq27']

### 7) bq28 결측치

In [410]:
for i in range(len(KNOW_2019_test.index)):
    if KNOW_2019_test['bq28'][i] == 0:
        KNOW_2019_test['bq28'][i] = KNOW_2019_test_mean[KNOW_2019_test_mean['bq1']==KNOW_2019_test['bq1'][i]][KNOW_2019_test_mean['bq2']==KNOW_2019_test['bq2'][i]]['bq28']

### 8) bq29 결측치

In [411]:
for i in range(len(KNOW_2019_test.index)):
    if KNOW_2019_test['bq29'][i] == 0:
        KNOW_2019_test['bq29'][i] = KNOW_2019_test_mean[KNOW_2019_test_mean['bq1']==KNOW_2019_test['bq1'][i]][KNOW_2019_test_mean['bq2']==KNOW_2019_test['bq2'][i]]['bq20']

### 9) bq30(주당 근로시간) 결측치

In [412]:
KNOW_2019_test[KNOW_2019_test['bq30']==0]

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,...,bq27,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3
152,18721,4,3,4,5,4,4,3,3,3,...,4,무역학,2,0,5,2,0,0,0,2400
281,18850,3,4,3,1,3,5,5,4,4,...,3,0,2,0,6,3,0,0,0,2000
442,19011,4,3,4,3,3,3,3,3,4,...,2,0,1,4,0,2,0,4000,0,0
811,19380,4,5,4,5,5,4,5,4,5,...,3,해양전기과,1,4,0,2,0,0,0,0
1288,19857,5,5,4,2,2,4,4,4,4,...,4,수학과,2,0,5,3,0,0,0,1800
1465,20034,5,4,4,5,4,5,4,4,5,...,4,사회복지,1,3,0,1,0,0,2200,0
2092,20661,4,4,4,1,2,2,2,3,3,...,2,전기,2,0,5,2,0,0,0,4000
2231,20800,5,5,5,3,5,3,3,4,5,...,2,문,2,0,5,3,0,0,0,5000
2606,21175,1,1,1,1,2,2,1,1,1,...,4,중어중문학과,1,4,0,2,0,0,0,0
2728,21298,4,5,5,1,5,2,4,5,5,...,2,문과,2,0,5,3,0,0,0,4500


산업군별 평균으로 대치

In [413]:
for i in range(len(KNOW_2019_test.index)):
    if KNOW_2019_test['bq30'][i] == 0:
        KNOW_2019_test['bq30'][i] = KNOW_2019_test_mean[KNOW_2019_test_mean['bq1']==KNOW_2019_test['bq1'][i]][KNOW_2019_test_mean['bq2']==KNOW_2019_test['bq2'][i]]['bq30']

In [414]:
KNOW_2019_test[KNOW_2019_test['bq30']==0]

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,...,bq27,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3


### 10) bq31 결측치

### case1. 임금근로자 연봉 결측치

In [415]:
KNOW_2019_test[KNOW_2019_test['bq28']==1][KNOW_2019_test['bq31_1']==0]

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,...,bq27,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3
89,18658,4,4,4,3,5,2,1,2,2,...,4,안경공학,1,1,0,1,50,0,1900,0
150,18719,5,5,5,5,4,5,5,5,5,...,4,영문,1,1,0,1,40,0,0,0
181,18750,4,4,4,4,4,4,4,4,4,...,5,전자공학,1,1,0,1,52,0,0,0
260,18829,4,4,4,4,4,3,4,4,4,...,5,신문방송학과,1,1,0,1,40,0,7000,0
294,18863,4,4,4,4,4,4,4,4,4,...,5,핀테크 경영MBA,1,3,0,2,52,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8152,26725,3,3,3,4,4,3,3,4,4,...,4,신문방송학,1,1,0,1,40,0,2000,0
8168,26741,3,4,4,4,5,3,4,5,4,...,4,경영,1,3,0,1,40,0,0,0
8357,26930,4,3,3,4,4,3,2,3,3,...,3,멀티미디어 디자인,1,1,0,1,40,0,0,0
8437,27010,5,5,5,5,5,5,4,5,5,...,5,소프트웨어,1,1,0,1,50,0,3000,0


In [416]:
for i in range(len(KNOW_2019_test.index)):
    if KNOW_2019_test['bq28'][i] == 1:
        if KNOW_2019_test['bq31_1'][i] == 0:
            KNOW_2019_test['bq31_1'][i] = KNOW_2019_test_mean[KNOW_2019_test_mean['bq1']==KNOW_2019_test['bq1'][i]][KNOW_2019_test_mean['bq2']==KNOW_2019_test['bq2'][i]]['bq31_1']

In [417]:
KNOW_2019_test[KNOW_2019_test['bq28']==1][KNOW_2019_test['bq31_1']==0]

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,...,bq27,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3
6692,25265,4,4,5,5,5,5,4,4,5,...,5,사회복지학과,1,1,0,1,40,0,3500,0


In [418]:
KNOW_2019_test[KNOW_2019_test['bq28']==1][KNOW_2019_test['bq31_2']==0]

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,...,bq27,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3
150,18719,5,5,5,5,4,5,5,5,5,...,4,영문,1,1,0,1,40,5343,0,0
162,18731,4,5,5,4,5,5,5,5,5,...,5,의학과,1,1,0,1,38,13000,0,0
181,18750,4,4,4,4,4,4,4,4,4,...,5,전자공학,1,1,0,1,52,3461,0,0
247,18816,4,4,3,2,4,4,4,3,4,...,4,치과기공과,1,4,0,2,45,4800,0,0
294,18863,4,4,4,4,4,4,4,4,4,...,5,핀테크 경영MBA,1,3,0,2,52,5343,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8357,26930,4,3,3,4,4,3,2,3,3,...,3,멀티미디어 디자인,1,1,0,1,40,2528,0,0
8432,27005,2,2,2,2,2,2,2,2,2,...,3,안경과학,1,3,0,1,50,3000,0,0
8453,27026,5,5,5,3,5,5,5,5,5,...,4,연극학,1,1,0,1,20,1200,0,0
8464,27037,5,5,5,5,5,5,5,5,5,...,2,0,1,4,0,2,40,986,0,0


In [419]:
for i in range(len(KNOW_2019_test.index)):
    if KNOW_2019_test['bq28'][i] == 1:
        if KNOW_2019_test['bq31_2'][i] == 0:
            KNOW_2019_test['bq31_2'][i] = KNOW_2019_test_mean[KNOW_2019_test_mean['bq1']==KNOW_2019_test['bq1'][i]][KNOW_2019_test_mean['bq2']==KNOW_2019_test['bq2'][i]]['bq31_2']

In [420]:
KNOW_2019_test[KNOW_2019_test['bq28']==1][KNOW_2019_test['bq31_2']==0]

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,...,bq27,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3


### case2. 비임금근로자 연봉 결측치

In [421]:
KNOW_2019_test[KNOW_2019_test['bq28']==2][KNOW_2019_test['bq31_3']==0]

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,...,bq27,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3
140,18709,4,4,4,4,4,4,3,3,4,...,4,0,2,0,5,3,40,0,0,0
439,19008,3,3,2,2,3,3,2,2,3,...,1,0,2,0,7,3,45,0,0,0
510,19079,4,4,4,4,4,4,4,4,3,...,4,미용예술학과,2,0,6,3,16,0,0,0
747,19316,4,4,4,4,4,4,4,4,4,...,4,방송연예학과,2,0,6,2,25,0,0,0
829,19398,5,5,5,3,4,4,4,4,4,...,5,산업공학,2,0,5,3,52,0,0,0
866,19435,5,1,4,3,5,5,3,4,2,...,5,스토리텔링,2,0,7,2,40,0,0,0
1005,19574,5,5,5,5,5,5,5,5,5,...,3,법률,2,0,5,2,50,0,0,0
1092,19661,5,5,5,3,3,2,2,5,5,...,4,경영과,2,0,5,3,52,0,0,0
1188,19757,3,3,4,2,3,3,2,2,3,...,1,0,2,0,7,2,60,0,0,0
1240,19809,2,2,2,1,2,2,3,2,3,...,1,0,2,0,7,2,40,0,0,0


In [422]:
for i in range(len(KNOW_2019_test.index)):
    if KNOW_2019_test['bq28'][i] == 2:
        if KNOW_2019_test['bq31_3'][i] == 0:
            KNOW_2019_test['bq31_3'][i] = KNOW_2019_test_mean[KNOW_2019_test_mean['bq1']==KNOW_2019_test['bq1'][i]][KNOW_2019_test_mean['bq2']==KNOW_2019_test['bq2'][i]]['bq31_3']

In [423]:
KNOW_2019_test[KNOW_2019_test['bq28']==2][KNOW_2019_test['bq31_3']==0]

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,...,bq27,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3


## 2.3.2.7. 컬럼 줄이기

### 1) 불필요한 컬럼 drop

In [424]:
KNOW_2019_test.info(verbose =True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8554 entries, 0 to 8553
Data columns (total 144 columns):
 #    Column  Dtype 
---   ------  ----- 
 0    idx     int64 
 1    sq1     int64 
 2    sq2     int64 
 3    sq3     int64 
 4    sq4     int64 
 5    sq5     int64 
 6    sq6     int64 
 7    sq7     int64 
 8    sq8     int64 
 9    sq9     int64 
 10   sq10    int64 
 11   sq11    int64 
 12   sq12    int64 
 13   sq13    int64 
 14   sq14    int64 
 15   sq15    int64 
 16   sq16    int64 
 17   kq1_1   int64 
 18   kq1_2   int64 
 19   kq2_1   int64 
 20   kq2_2   int64 
 21   kq3_1   int64 
 22   kq3_2   int64 
 23   kq4_1   int64 
 24   kq4_2   int64 
 25   kq5_1   int64 
 26   kq5_2   int64 
 27   kq6_1   int64 
 28   kq6_2   int64 
 29   kq7_1   int64 
 30   kq7_2   int64 
 31   kq8_1   int64 
 32   kq8_2   int64 
 33   kq9_1   int64 
 34   kq9_2   int64 
 35   kq10_1  int64 
 36   kq10_2  int64 
 37   kq11_1  int64 
 38   kq11_2  int64 
 39   kq12_1  int64 
 40   kq12

In [425]:
# 자격증, 학과를 제외한 텍스트 데이터 컬럼 삭제
# KNOW_2019_test = KNOW_2019_test.drop(['bq4_1b','bq4_1c','bq5_2','bq18_10','bq20_1','bq22','bq23','bq24'], axis = 1)


In [426]:
# kq_2 컬럼 삭제
for i in range(1, 34):
    KNOW_2019_test = KNOW_2019_test.drop(['kq'+str(i)+'_2'], axis=1)

In [427]:
KNOW_2019_test.head()

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,...,bq27,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3
0,18569,5,5,5,4,4,5,4,5,5,...,4,경영,1,1,0,1,45,6700,3800,0
1,18570,2,3,3,1,3,2,3,3,3,...,3,화학,1,1,0,1,44,2700,2500,0
2,18571,5,4,5,4,4,4,4,5,4,...,4,건축공학,1,1,0,1,40,6000,3200,0
3,18572,3,3,3,2,3,3,3,3,2,...,2,인문,1,1,0,1,60,900,800,0
4,18573,5,4,5,3,3,3,3,4,5,...,6,기계공학과,1,1,0,1,50,7000,5000,0


### 2) 평균값을 사용해 컬럼 줄이기

In [428]:
# bq8_1, bq8_2, bq8_3 통합하여 bq8 만들기
KNOW_2019_test['bq8_1'] = round((KNOW_2019_test['bq8_1']+KNOW_2019_test['bq8_2']+KNOW_2019_test['bq8_3'])/3)
KNOW_2019_test['bq8_1'] = KNOW_2019_test['bq8_1'].astype(int)
KNOW_2019_test = KNOW_2019_test.rename({'bq8_1':'bq8'}, axis = 1 )
KNOW_2019_test = KNOW_2019_test.drop(['bq8_2','bq8_3'], axis = 1)
KNOW_2019_test.head(3)

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,...,bq27,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3
0,18569,5,5,5,4,4,5,4,5,5,...,4,경영,1,1,0,1,45,6700,3800,0
1,18570,2,3,3,1,3,2,3,3,3,...,3,화학,1,1,0,1,44,2700,2500,0
2,18571,5,4,5,4,4,4,4,5,4,...,4,건축공학,1,1,0,1,40,6000,3200,0


In [429]:
# bq12_5만 사용
KNOW_2019_test = KNOW_2019_test.rename({'bq12_5':'bq12'}, axis = 1 )
KNOW_2019_test = KNOW_2019_test.drop(['bq12_1','bq12_2','bq12_3','bq12_4'], axis = 1)

In [430]:
# bq13_1, bq13_2, bq13_3 통합하여 bq13 만들기
KNOW_2019_test['bq13_1'] = round((KNOW_2019_test['bq13_1']+KNOW_2019_test['bq13_2']+KNOW_2019_test['bq13_3'])/3)
KNOW_2019_test['bq13_1'] = KNOW_2019_test['bq13_1'].astype(int)
KNOW_2019_test = KNOW_2019_test.rename({'bq13_1':'bq13'}, axis = 1 )
KNOW_2019_test = KNOW_2019_test.drop(['bq13_2','bq13_3'], axis = 1)

In [431]:
# bq14_1, bq14_2, bq14_3, bq14_4, bq14_5 통합하여 bq14 만들기
KNOW_2019_test['bq14_1'] = round((KNOW_2019_test['bq14_1']+KNOW_2019_test['bq14_2']+KNOW_2019_test['bq14_3']+KNOW_2019_test['bq14_4']+KNOW_2019_test['bq14_5'])/5)
KNOW_2019_test['bq14_1'] = KNOW_2019_test['bq14_1'].astype(int)
KNOW_2019_test = KNOW_2019_test.rename({'bq14_1':'bq14'}, axis = 1 )
KNOW_2019_test = KNOW_2019_test.drop(['bq14_2','bq14_3','bq14_4','bq14_5'], axis = 1)

In [432]:
# bq16_1, bq16_2, bq16_3, bq16_4, bq16_5 통합하여 bq16 만들기
KNOW_2019_test['bq16_1'] = round((KNOW_2019_test['bq16_1']+KNOW_2019_test['bq16_2']+KNOW_2019_test['bq16_3']+KNOW_2019_test['bq16_4']+KNOW_2019_test['bq16_5'])/5)
KNOW_2019_test['bq16_1'] = KNOW_2019_test['bq16_1'].astype(int)
KNOW_2019_test = KNOW_2019_test.rename({'bq16_1':'bq16'}, axis = 1 )
KNOW_2019_test = KNOW_2019_test.drop(['bq16_2','bq16_3','bq16_4','bq16_5'], axis = 1)

In [433]:
# bq21_1, bq21_2, bq21_3 통합하여 bq21 만들기
KNOW_2019_test['bq21_1'] = round((KNOW_2019_test['bq21_1']+KNOW_2019_test['bq21_2']+KNOW_2019_test['bq21_3'])/3)
KNOW_2019_test['bq21_1'] = KNOW_2019_test['bq21_1'].astype(int)
KNOW_2019_test = KNOW_2019_test.rename({'bq21_1':'bq21'}, axis = 1 )
KNOW_2019_test = KNOW_2019_test.drop(['bq21_2','bq21_3'], axis = 1)

In [434]:
KNOW_2019_test.head()

,idx,sq1,sq2,sq3,sq4,sq5,sq6,sq7,sq8,sq9,...,bq27,bq27_1,bq28,bq28_1,bq28_2,bq29,bq30,bq31_1,bq31_2,bq31_3
0,18569,5,5,5,4,4,5,4,5,5,...,4,경영,1,1,0,1,45,6700,3800,0
1,18570,2,3,3,1,3,2,3,3,3,...,3,화학,1,1,0,1,44,2700,2500,0
2,18571,5,4,5,4,4,4,4,5,4,...,4,건축공학,1,1,0,1,40,6000,3200,0
3,18572,3,3,3,2,3,3,3,3,2,...,2,인문,1,1,0,1,60,900,800,0
4,18573,5,4,5,3,3,3,3,4,5,...,6,기계공학과,1,1,0,1,50,7000,5000,0


In [435]:
# 전공을 제외한 나머지 데이터 유형 인트형으로 변환
for col in range(len(KNOW_2019_test.columns)):
    if (KNOW_2019_test.columns[col] == 'bq27_1')  or (KNOW_2019.columns[col] == 'bq4_1a'):
        pass
    else:
        KNOW_2019_test.iloc[:,col] = KNOW_2019_test.iloc[:,col].astype(int)

In [436]:
processing_test_2019 = KNOW_2019_test

## 2.4. 2020년 전처리

### 2.4.1 2020년 Train 데이터 전처리

In [4]:
df_2020 = know_train[3]

# 데이터의 빈 셀에 None값이 아닌 ' '처럼 공백으로 들어가있기 때문에 is_null등의 함수로 결측치를 찾아낼 수 없어서 결측치 0으로 변환
for col in df_2020.columns:
    df_2020[col].replace(' ',0,inplace = True)

#### 1) saq, vq 문항 전처리: 데이터 타입 확인 및 변환

In [5]:
# saq_1: _1로 끝나는 변수들, saq_2: _2로 끝나는 변수들
saq_2020_1,saq_2020_2 = [],[]
for num in range(1,45):
    column1 = 'saq'+ str(num) + '_1'
    column2 = 'saq'+ str(num) + '_2'
    saq_2020_1.append(column1)
    saq_2020_2.append(column2)
vq_2020 = df_2020.loc[:,'vq1':'vq13'].columns

In [6]:
for col in saq_2020_2:
    df_2020.loc[:,[col]] = df_2020.loc[:,[col]].astype('int64')

df_2020[saq_2020_2].dtypes

saq1_2     int64
saq2_2     int64
saq3_2     int64
saq4_2     int64
saq5_2     int64
saq6_2     int64
saq7_2     int64
saq8_2     int64
saq9_2     int64
saq10_2    int64
saq11_2    int64
saq12_2    int64
saq13_2    int64
saq14_2    int64
saq15_2    int64
saq16_2    int64
saq17_2    int64
saq18_2    int64
saq19_2    int64
saq20_2    int64
saq21_2    int64
saq22_2    int64
saq23_2    int64
saq24_2    int64
saq25_2    int64
saq26_2    int64
saq27_2    int64
saq28_2    int64
saq29_2    int64
saq30_2    int64
saq31_2    int64
saq32_2    int64
saq33_2    int64
saq34_2    int64
saq35_2    int64
saq36_2    int64
saq37_2    int64
saq38_2    int64
saq39_2    int64
saq40_2    int64
saq41_2    int64
saq42_2    int64
saq43_2    int64
saq44_2    int64
dtype: object

##### <업무 수행능력>을 확인하는 column: saq1_1 ~ saq 44_2
- saq**num**_1 : 1~5까지 5점 척도로 응답되어야 함
- saq**num**_2: 1~7까지의 수준으로 응답되어야 함
- 논리구조: saq**num**_1에 중요하지 않다(1)을 선택하는 경우는 saq**num**_2에 응답하지 않도록 되어 있음.

In [7]:
# saqnum_1
df_2020_saq1 = df_2020[saq_2020_1]
df_2020_saq1.isin([1, 2, 3 ,4, 5]).sum(axis=0)

saq1_1     8122
saq2_1     8122
saq3_1     8122
saq4_1     8122
saq5_1     8122
saq6_1     8122
saq7_1     8122
saq8_1     8122
saq9_1     8122
saq10_1    8122
saq11_1    8122
saq12_1    8122
saq13_1    8122
saq14_1    8122
saq15_1    8122
saq16_1    8122
saq17_1    8122
saq18_1    8122
saq19_1    8122
saq20_1    8122
saq21_1    8122
saq22_1    8122
saq23_1    8122
saq24_1    8122
saq25_1    8122
saq26_1    8122
saq27_1    8122
saq28_1    8122
saq29_1    8122
saq30_1    8122
saq31_1    8122
saq32_1    8122
saq33_1    8122
saq34_1    8122
saq35_1    8122
saq36_1    8122
saq37_1    8122
saq38_1    8122
saq39_1    8122
saq40_1    8122
saq41_1    8122
saq42_1    8122
saq43_1    8122
saq44_1    8122
dtype: int64

* df_saq1에는 이상치 및 결측치가 없는 것으로 확인됨.

##### saq : 중요도가 1인데(중요하지 않다) 2번 문항에 답변을 한 경우 로직에 맞게 수정

In [8]:
#중요하지 않다(1)를 선택한 사람이 saq_2에 응답하지 않았는지(0) 확인하는 함수
check_not = []
def check_if(column1,column2):
    if sum(df_2020[column1] == 1 ) != sum(df_2020[column2] == 0):
        print(f'{column1} is not same with {column2}')
        check_not.append((column1,column2))


for num in range(1,45):
    column1 = 'saq'+ str(num) + '_1'
    column2 = 'saq'+ str(num) + '_2'
    check_if(column1,column2)

saq40_1 is not same with saq40_2


In [9]:
# 중요하지 않다고 선택한 사람의 답변 정정 (로직 구조 맞도록)
for col1,col2 in check_not:
    df_2020.loc[(df_2020[col1] == 1) & (df_2020[col2] != 0),col2] = 0

In [10]:
# saqnum_2
df_2020_saq2 = df_2020[saq_2020_2]
df_2020_saq2.isin([0, 1, 2, 3, 4, 5, 6, 7]).sum(axis=0)

saq1_2     8122
saq2_2     8122
saq3_2     8122
saq4_2     8122
saq5_2     8122
saq6_2     8122
saq7_2     8122
saq8_2     8122
saq9_2     8122
saq10_2    8122
saq11_2    8122
saq12_2    8122
saq13_2    8122
saq14_2    8122
saq15_2    8122
saq16_2    8122
saq17_2    8122
saq18_2    8122
saq19_2    8122
saq20_2    8122
saq21_2    8122
saq22_2    8122
saq23_2    8122
saq24_2    8122
saq25_2    8122
saq26_2    8122
saq27_2    8122
saq28_2    8122
saq29_2    8122
saq30_2    8122
saq31_2    8122
saq32_2    8122
saq33_2    8122
saq34_2    8122
saq35_2    8122
saq36_2    8122
saq37_2    8122
saq38_2    8122
saq39_2    8122
saq40_2    8122
saq41_2    8122
saq42_2    8122
saq43_2    8122
saq44_2    8122
dtype: int64

* df_saq2에는 이상치 및 결측치가 없는 것으로 확인됨.

##### saq : 중요도가 1이 아닌데 2번 문항에 답변을 안한 경우 bq_1, bq2별 평균으로 대체

In [11]:
# saq_1가 1이 아닌데 saq_2에 응답했는지( != 0) 확인하는 함수
check_yes = []

def check_null(column1,column2):
    if sum(df_2020[column1] != 1 ) != sum(df_2020[column2] != 0):
        print(f'{column1} is not same with {column2}')
        check_not.append((column1,column2))


for num in range(1,45):
    column1 = 'saq'+ str(num) + '_1'
    column2 = 'saq'+ str(num) + '_2'
    check_null(column1,column2)

In [12]:
# 산업별,업체유형별 각 문항의 평균값을 결측치에 대치
df_2020_mean= df_2020.groupby(['bq1','bq2']).mean().reset_index()
df_2020_mean.head()

,bq1,bq2,idx,saq1_1,saq1_2,saq2_1,saq2_2,saq3_1,saq3_2,saq4_1,...,bq23_1,bq23_2,bq23_3,bq24,bq25,bq26,bq27_1,bq27_2,bq29,knowcode
0,1,1,31119.238806,1.925373,1.746269,2.059701,2.029851,1.791045,1.492537,1.970149,...,4.835821,4.671642,4.731343,1.149254,47.537313,2.582090,1.417910,3.000000,45.044776,836733.850746
1,1,2,27428.000000,4.000000,3.000000,3.000000,4.000000,2.000000,2.000000,3.000000,...,3.000000,3.000000,3.000000,2.000000,24.000000,4.000000,1.000000,3.000000,40.000000,903101.000000
2,1,3,30978.285714,2.000000,2.142857,2.714286,3.285714,1.857143,1.571429,2.571429,...,6.714286,4.571429,2.857143,1.000000,52.428571,2.000000,1.571429,3.285714,40.000000,901729.571429
3,1,4,31057.508772,1.771930,1.561404,2.052632,1.929825,1.596491,1.228070,2.052632,...,5.298246,5.070175,4.701754,1.298246,56.859649,1.947368,1.982456,5.982456,43.263158,902108.017544
4,2,1,31480.250000,2.750000,4.000000,2.916667,4.083333,1.833333,1.833333,2.250000,...,4.666667,4.166667,5.500000,1.000000,44.416667,3.000000,1.000000,1.000000,40.333333,610517.666667


In [13]:
# 중요하지 않다고 선택한 사람의 답변 정정 (로직 구조 맞도록)
col_list = list(df_2020.columns)

for col1,col2 in check_not:
    index = df_2020.loc[(df_2020[col1] != 1) & (df_2020[col2] == 0)].index
    col_idx = col_list.index(col2)
    for i in index:
        value = float(df_2020_mean.loc[(df_2020_mean['bq1'] == df_2020.iloc[i]['bq1']) & 
                                       (df_2020_mean['bq2'] == df_2020.iloc[i]['bq2']),col2])
        df_2020.iloc[i,col_idx] = value

##### saq : 중요도가 1이 아닌데 2번 문항에 답변을 안한 경우는 존재하지 않는 것으로 확인됨.

##### <가치관>을 확인하는 column: vq1 ~ vq13

In [447]:
# vq
df_2020_vq = df_2020[vq_2020]
df_2020_vq.isin([1, 2, 3, 4, 5]).sum(axis=0)

vq1     8122
vq2     8122
vq3     8122
vq4     8122
vq5     8122
vq6     8122
vq7     8122
vq8     8122
vq9     8122
vq10    8122
vq11    8122
vq12    8122
vq13    8122
dtype: int64

In [448]:
df_2020_vq.isin([0]).sum(axis=0)

vq1     0
vq2     0
vq3     0
vq4     0
vq5     0
vq6     0
vq7     0
vq8     0
vq9     0
vq10    0
vq11    0
vq12    0
vq13    0
dtype: int64

* vq에 이상치 및 결측치 없는 것으로 확인됨.

##### 2) bq 문항 전처리

In [449]:
display(df_2020.bq5_1.value_counts())

0    5816
3     611
2     579
1     420
4     328
6     207
5     161
Name: bq5_1, dtype: int64

In [450]:
df_2020['bq5_1'].replace(' ', 0, inplace = True)
df_2020['bq5_1'] = df_2020['bq5_1'].astype(int)

In [451]:
df_2020.index[df_2020['bq28'] == ' '].tolist()

[]

In [452]:
df_2020.loc[df_2020.index[df_2020['bq28'] == ' '].tolist(),'bq28'] = 1
df_2020.bq28 = df_2020.bq28.astype(int)

In [453]:
df_2020.bq30_1.value_counts()

0       1016
3000     483
4000     470
3800     351
3500     346
        ... 
7250       1
3248       1
2920       1
4312       1
2637       1
Name: bq30_1, Length: 329, dtype: int64

In [454]:
df_2020.bq30_1.replace(' ', 0, inplace = True)
df_2020.bq30_1 = df_2020['bq30_1'].astype(int)

In [455]:
df_2020.bq30_2.replace(' ', 0, inplace = True)
df_2020.bq30_2 = df_2020['bq30_2'].astype(int)

In [456]:
df_2020.bq30_3.replace(' ', 0, inplace = True)
df_2020.bq30_3 = df_2020['bq30_3'].astype(int)

In [457]:
list(df_2020.columns).index('bq30_2')

180

In [458]:
df_2020.loc[(df_2020['bq30_1'] != 0) & (df_2020['bq30_2'] == 0) , 'bq30_3'] 

4172    4000
4882    4000
7430    4000
Name: bq30_3, dtype: int64

In [459]:
error_list = df_2020[(df_2020.bq30_1 != 0) & (df_2020.bq30_2 == 0) & (df_2020.bq30_3 != 0)].index.tolist()
value = list(df_2020.loc[(df_2020['bq30_1'] != 0) & (df_2020['bq30_2'] == 0) , 'bq30_3'] )
print(value)
for i,v in zip(error_list,value):
    df_2020.iloc[i,list(df_2020.columns).index('bq30_2')] = v
    df_2020.iloc[i,list(df_2020.columns).index('bq30_3')] = 0

[4000, 4000, 4000]


In [460]:
# check point
display(df_2020[(df_2020.bq30_2 == 0) & (df_2020.bq30_1 != 0)])

display(df_2020[(df_2020.bq30_3 == 0) & (df_2020.bq30_2 == 0)])

,idx,saq1_1,saq1_2,saq2_1,saq2_2,saq3_1,saq3_2,saq4_1,saq4_2,saq5_1,...,bq26,bq26_1,bq27_1,bq27_2,bq28,bq29,bq30_1,bq30_2,bq30_3,knowcode


,idx,saq1_1,saq1_2,saq2_1,saq2_2,saq3_1,saq3_2,saq4_1,saq4_2,saq5_1,...,bq26,bq26_1,bq27_1,bq27_2,bq28,bq29,bq30_1,bq30_2,bq30_3,knowcode


In [461]:
df_2020.bq4_1a.value_counts()

0             3847
전기기능사           53
박사학위            46
정보처리기사          37
전기기사            35
              ... 
보건교사2급           1
잠수기능사 자격증        1
선박교통관제사          1
천장크레인운전기능사       1
사회복지사 1급         1
Name: bq4_1a, Length: 1632, dtype: int64

In [462]:
df_2020.bq18_10.value_counts()

0                                        6693
새로운 장비 도입                                  21
구조 조정                                      12
새로운 기계 도입                                  10
구조조정                                        9
                                         ... 
컴퓨터 OS와 SW변화에 따른 학습 필요                      1
전산 시스템의 변화(수기 작성 일지->전산화 일지 입력 변경)          1
타다 이슈로 인식이 안좋아짐                             1
태블릿 사용 및 매일 매일 변화하는 시장 가격 등으로 판매시 어려움       1
서비스 질 향상과 상품의 다변화                           1
Name: bq18_10, Length: 1314, dtype: int64

In [463]:
# bq18_10의 'N.A' (해당없음)을 ' '로 바꾸기 (의미 같으므로)
df_2020['bq18_10'].replace('N.A', ' ', inplace = True)

In [464]:
df_2020.drop(['bq5_2', 'bq18_10', 'bq20_1'], axis = 1, inplace = True)

In [465]:
df_2020.head()

,idx,saq1_1,saq1_2,saq2_1,saq2_2,saq3_1,saq3_2,saq4_1,saq4_2,saq5_1,...,bq26,bq26_1,bq27_1,bq27_2,bq28,bq29,bq30_1,bq30_2,bq30_3,knowcode
0,27127,4,4,4,4,4,5,4,5,3,...,4,간호학,1,1,1,40,4000,2600,0,304003
1,27128,5,6,5,6,4,5,4,5,4,...,4,기계공학,1,1,1,40,4000,3000,0,221401
2,27129,3,4,3,4,3,4,3,5,3,...,4,기계,1,1,1,48,4300,3000,0,815001
3,27130,4,5,2,3,3,3,4,5,3,...,4,보건학,1,1,1,40,3500,3300,0,304003
4,27131,5,6,4,5,3,5,4,4,4,...,4,건축설비공학,1,1,1,40,5700,2700,0,140201


##### 나머지 처리

- bq4_1b, bq4_1c -> 삭제
- 8_1,2,3 -> 평균 후 인트형으로
- 12 -> 전반적으로만 쓰기
- 13 -> 평균 후 인트형으로
- saq*_2번 다 삭제

In [466]:
#8_1,2,3 -> 평균 후 인트형으로
df_2020['bq8'] = (df_2020['bq8_1'] + df_2020['bq8_2'] + df_2020['bq8_3'])/3

In [467]:
df_2020.bq8 #처리 전

0       4.000000
1       4.000000
2       4.000000
3       3.333333
4       3.666667
          ...   
8117    4.333333
8118    3.000000
8119    3.333333
8120    4.000000
8121    3.000000
Name: bq8, Length: 8122, dtype: float64

In [468]:
df_2020.bq8 = df_2020.bq8.round(0).astype(int) # 처리완료

In [469]:
df_2020.drop(['bq8_1', 'bq8_2', 'bq8_3'], axis=1, inplace=True)

In [470]:
# 12 -> 전반적으로만 쓰기
df_2020.drop(['bq12_' + str(i) for i in range(1,5)], axis = 1, inplace=True)

In [471]:
df_2020.rename(columns={'bq12_5':'bq12'}, inplace = True)

In [472]:
# 13 -> 평균 후 인트형으로
df_2020['bq13'] = (df_2020['bq13_1'] + df_2020['bq13_2'] + df_2020['bq13_3'])/3

In [473]:
df_2020.bq13 = df_2020.bq13.round(0).astype(int)

In [474]:
df_2020.drop(['bq13_1', 'bq13_2', 'bq13_3'], axis=1, inplace=True)

In [475]:
# saq*_2번 다 삭제
df_2020.drop(['saq' + str(i) + '_2' for i in range(1,45)], axis=1, inplace=True)

In [476]:
processing_train_2020 = df_2020

### 2.4.2 2020년 Test 데이터 전처리

In [477]:
df_2020 = know_test[3]


# 데이터의 빈 셀에 None값이 아닌 ' '처럼 공백으로 들어가있기 때문에 is_null등의 함수로 결측치를 찾아낼 수 없어서 결측치 0으로 변환
for col in df_2020.columns:
    df_2020[col].replace(' ',0,inplace = True)

#### saq, vq 문항 전처리: 데이터 타입 확인 및 변환

In [478]:
# saq_1: _1로 끝나는 변수들, saq_2: _2로 끝나는 변수들
saq_2020_1,saq_2020_2 = [],[]
for num in range(1,45):
    column1 = 'saq'+ str(num) + '_1'
    column2 = 'saq'+ str(num) + '_2'
    saq_2020_1.append(column1)
    saq_2020_2.append(column2)
vq_2020 = df_2020.loc[:,'vq1':'vq13'].columns

In [479]:
for col in saq_2020_2:
    df_2020.loc[:,[col]] = df_2020.loc[:,[col]].astype('int64')

df_2020[saq_2020_2].dtypes

saq1_2     int64
saq2_2     int64
saq3_2     int64
saq4_2     int64
saq5_2     int64
saq6_2     int64
saq7_2     int64
saq8_2     int64
saq9_2     int64
saq10_2    int64
saq11_2    int64
saq12_2    int64
saq13_2    int64
saq14_2    int64
saq15_2    int64
saq16_2    int64
saq17_2    int64
saq18_2    int64
saq19_2    int64
saq20_2    int64
saq21_2    int64
saq22_2    int64
saq23_2    int64
saq24_2    int64
saq25_2    int64
saq26_2    int64
saq27_2    int64
saq28_2    int64
saq29_2    int64
saq30_2    int64
saq31_2    int64
saq32_2    int64
saq33_2    int64
saq34_2    int64
saq35_2    int64
saq36_2    int64
saq37_2    int64
saq38_2    int64
saq39_2    int64
saq40_2    int64
saq41_2    int64
saq42_2    int64
saq43_2    int64
saq44_2    int64
dtype: object

##### <업무 수행능력>을 확인하는 column: saq1_1 ~ saq 44_2
- saq**num**_1 : 1~5까지 5점 척도로 응답되어야 함
- saq**num**_2: 1~7까지의 수준으로 응답되어야 함
- 논리구조: saq**num**_1에 중요하지 않다(1)을 선택하는 경우는 saq**num**_2에 응답하지 않도록 되어 있음.

In [480]:
# saqnum_1
df_2020_saq1 = df_2020[saq_2020_1]
df_2020_saq1.isin([1, 2, 3 ,4, 5]).sum(axis=0)

saq1_1     8122
saq2_1     8122
saq3_1     8122
saq4_1     8122
saq5_1     8122
saq6_1     8122
saq7_1     8122
saq8_1     8122
saq9_1     8122
saq10_1    8122
saq11_1    8122
saq12_1    8122
saq13_1    8122
saq14_1    8122
saq15_1    8122
saq16_1    8122
saq17_1    8122
saq18_1    8122
saq19_1    8122
saq20_1    8122
saq21_1    8122
saq22_1    8122
saq23_1    8122
saq24_1    8122
saq25_1    8122
saq26_1    8122
saq27_1    8122
saq28_1    8122
saq29_1    8122
saq30_1    8122
saq31_1    8122
saq32_1    8122
saq33_1    8122
saq34_1    8122
saq35_1    8122
saq36_1    8122
saq37_1    8122
saq38_1    8122
saq39_1    8122
saq40_1    8122
saq41_1    8122
saq42_1    8122
saq43_1    8122
saq44_1    8122
dtype: int64

* df_saq1에는 이상치 및 결측치가 없는 것으로 확인됨.

##### saq : 중요도가 1인데(중요하지 않다) 2번 문항에 답변을 한 경우 로직에 맞게 수정

In [481]:
#중요하지 않다(1)를 선택한 사람이 saq_2에 응답하지 않았는지(0) 확인하는 함수
check_not = []
def check_if(column1,column2):
    if sum(df_2020[column1] == 1 ) != sum(df_2020[column2] == 0):
        print(f'{column1} is not same with {column2}')
        check_not.append((column1,column2))


for num in range(1,45):
    column1 = 'saq'+ str(num) + '_1'
    column2 = 'saq'+ str(num) + '_2'
    check_if(column1,column2)

saq40_1 is not same with saq40_2


In [482]:
# 중요하지 않다고 선택한 사람의 답변 정정 (로직 구조 맞도록)
for col1,col2 in check_not:
    df_2020.loc[(df_2020[col1] == 1) & (df_2020[col2] != 0),col2] = 0

In [483]:
# saqnum_2
df_2020_saq2 = df_2020[saq_2020_2]
df_2020_saq2.isin([0, 1, 2, 3, 4, 5, 6, 7]).sum(axis=0)

saq1_2     8122
saq2_2     8122
saq3_2     8122
saq4_2     8122
saq5_2     8122
saq6_2     8122
saq7_2     8122
saq8_2     8122
saq9_2     8122
saq10_2    8122
saq11_2    8122
saq12_2    8122
saq13_2    8122
saq14_2    8122
saq15_2    8122
saq16_2    8122
saq17_2    8122
saq18_2    8122
saq19_2    8122
saq20_2    8122
saq21_2    8122
saq22_2    8122
saq23_2    8122
saq24_2    8122
saq25_2    8122
saq26_2    8122
saq27_2    8122
saq28_2    8122
saq29_2    8122
saq30_2    8122
saq31_2    8122
saq32_2    8122
saq33_2    8122
saq34_2    8122
saq35_2    8122
saq36_2    8122
saq37_2    8122
saq38_2    8122
saq39_2    8122
saq40_2    8122
saq41_2    8122
saq42_2    8122
saq43_2    8122
saq44_2    8122
dtype: int64

* df_saq2에는 이상치 및 결측치가 없는 것으로 확인됨.

##### saq : 중요도가 1이 아닌데 2번 문항에 답변을 안한 경우 bq_1, bq2별 평균으로 대체

In [484]:
# saq_1가 1이 아닌데 saq_2에 응답했는지( != 0) 확인하는 함수
check_yes = []

def check_null(column1,column2):
    if sum(df_2020[column1] != 1 ) != sum(df_2020[column2] != 0):
        print(f'{column1} is not same with {column2}')
        check_not.append((column1,column2))


for num in range(1,45):
    column1 = 'saq'+ str(num) + '_1'
    column2 = 'saq'+ str(num) + '_2'
    check_null(column1,column2)

saq40_1 is not same with saq40_2


In [485]:
# 앞에서 결측치를 0으로 대치했기 때문에 다시 0을 knowcode의 특성을 반영하기 위해 만든 df_mean를 이용.
# knowcode별 각 문항의 평균값을 결측치에 대치
df_2020_mean= df_2020.groupby(['bq1','bq2']).mean().reset_index()
df_2020_mean.head()

,bq1,bq2,idx,saq1_1,saq1_2,saq2_1,saq2_2,saq3_1,saq3_2,saq4_1,...,bq22_6,bq23_1,bq23_2,bq23_3,bq24,bq25,bq26,bq27_1,bq27_2,bq29
0,1,1,31397.375000,2.062500,2.1250,2.312500,2.390625,1.765625,1.593750,2.171875,...,5.531250,5.625000,5.093750,4.921875,1.078125,48.421875,2.531250,1.43750,2.937500,44.875000
1,1,2,28950.500000,3.500000,5.5000,3.500000,5.000000,2.500000,2.500000,3.000000,...,3.000000,6.000000,6.500000,5.000000,1.500000,51.500000,4.500000,1.00000,1.000000,47.500000
2,1,3,30896.500000,2.250000,2.7500,2.250000,3.250000,1.500000,1.250000,2.500000,...,4.750000,6.750000,5.500000,4.500000,1.000000,48.750000,2.000000,1.50000,3.500000,39.000000
3,1,4,31352.953125,1.765625,1.5625,2.078125,1.984375,1.562500,1.234375,1.984375,...,5.218750,5.593750,4.890625,4.906250,1.359375,56.296875,1.968750,1.96875,5.984375,44.093750
4,2,1,31513.818182,2.727273,3.0000,2.727273,3.818182,1.727273,1.727273,2.727273,...,4.909091,5.363636,4.363636,4.272727,1.000000,42.454545,2.909091,1.00000,1.454545,40.818182


In [486]:
# 중요하지 않다고 선택한 사람의 답변 정정 (로직 구조 맞도록)
col_list = list(df_2020.columns)
for col1,col2 in check_not:
    index = df_2020.loc[(df_2020[col1] != 1) & (df_2020[col2] == 0)].index
    col_idx = col_list.index(col2)
    for i in index:
        value = float(df_2020_mean.loc[(df_2020_mean['bq1'] == df_2020.iloc[i]['bq1']) & (df_2020_mean['bq2'] == df_2020.iloc[i]['bq2']),col2])
        df_2020.iloc[i,col_idx] = value

##### saq : 중요도가 1이 아닌데 2번 문항에 답변을 안한 경우는 존재하지 않는 것으로 확인됨.

##### <가치관>을 확인하는 column: vq1 ~ vq13

In [487]:
# vq
df_2020_vq = df_2020[vq_2020]
df_2020_vq.isin([1, 2, 3, 4, 5]).sum(axis=0)

vq1     8122
vq2     8122
vq3     8122
vq4     8122
vq5     8122
vq6     8122
vq7     8122
vq8     8122
vq9     8122
vq10    8122
vq11    8122
vq12    8122
vq13    8122
dtype: int64

In [488]:
df_2020_vq.isin([0]).sum(axis=0)

vq1     0
vq2     0
vq3     0
vq4     0
vq5     0
vq6     0
vq7     0
vq8     0
vq9     0
vq10    0
vq11    0
vq12    0
vq13    0
dtype: int64

* vq에 이상치 및 결측치 없는 것으로 확인됨.

#### bq 문항 전처리

##### ' ' -> 0으로 변환

- int형으로 변환해주기 위해서 처리해줌

In [489]:
display(df_2020.bq5_1.value_counts())

0    5822
3     595
2     584
1     433
4     307
6     206
5     175
Name: bq5_1, dtype: int64

In [490]:
df_2020['bq5_1'].replace(' ', 0, inplace = True)
df_2020['bq5_1'] = df_2020['bq5_1'].astype(int)

In [491]:
df_2020.index[df_2020['bq28'] == ' '].tolist()

[]

In [492]:
df_2020.loc[df_2020.index[df_2020['bq28'] == ' '].tolist(),'bq28'] = 1
df_2020.bq28 = df_2020.bq28.astype(int)

In [493]:
df_2020.bq30_1.value_counts()

0       1014
3000     519
4000     474
3500     345
3800     327
        ... 
6421       1
3480       1
2898       1
2720       1
4219       1
Name: bq30_1, Length: 345, dtype: int64

In [494]:
df_2020.bq30_1.replace(' ', 0, inplace = True)
df_2020.bq30_1 = df_2020['bq30_1'].astype(int)

In [495]:
df_2020.bq30_2.replace(' ', 0, inplace = True)
df_2020.bq30_2 = df_2020['bq30_2'].astype(int)

In [496]:
df_2020.bq30_3.replace(' ', 0, inplace = True)
df_2020.bq30_3 = df_2020['bq30_3'].astype(int)

In [497]:
list(df_2020.columns).index('bq30_2')

180

In [498]:
df_2020.loc[(df_2020['bq30_1'] != 0) & (df_2020['bq30_2'] == 0) , 'bq30_3'] 

Series([], Name: bq30_3, dtype: int64)

In [499]:
error_list = df_2020[(df_2020.bq30_1 != 0) & (df_2020.bq30_2 == 0) & (df_2020.bq30_3 != 0)].index.tolist()
value = list(df_2020.loc[(df_2020['bq30_1'] != 0) & (df_2020['bq30_2'] == 0) , 'bq30_3'] )
print(value)
for i,v in zip(error_list,value):
    df_2020.iloc[i,list(df_2020.columns).index('bq30_2')] = v
    df_2020.iloc[i,list(df_2020.columns).index('bq30_3')] = 0

[]


In [500]:
# check point
display(df_2020[(df_2020.bq30_2 == 0) & (df_2020.bq30_1 != 0)])

display(df_2020[(df_2020.bq30_3 == 0) & (df_2020.bq30_2 == 0)])

,idx,saq1_1,saq1_2,saq2_1,saq2_2,saq3_1,saq3_2,saq4_1,saq4_2,saq5_1,...,bq25,bq26,bq26_1,bq27_1,bq27_2,bq28,bq29,bq30_1,bq30_2,bq30_3


,idx,saq1_1,saq1_2,saq2_1,saq2_2,saq3_1,saq3_2,saq4_1,saq4_2,saq5_1,...,bq25,bq26,bq26_1,bq27_1,bq27_2,bq28,bq29,bq30_1,bq30_2,bq30_3


In [501]:
df_2020.bq4_1a.value_counts()

0                 3881
산업기사                49
전기기사                47
전기기능사               44
박사학위                42
                  ... 
간호사  자격증             1
화공안전기술기사             1
건설산업안전기사, 산업기사       1
산업계측제어기사             1
엔지니어링 기술자            1
Name: bq4_1a, Length: 1535, dtype: int64

In [502]:
df_2020.bq18_10.value_counts()

0                            6742
새로운 장비 도입                      19
새로운 프로그램 도입                    10
새로운 기계 도입                      10
구조조정                            9
                             ... 
새로운 디자인 프로그램 개발                 1
기계화로 인한 업무 감소                   1
자동으로 고장부분 확인 및 수리 시스템           1
새로운 조직을 구성해서 업무를 이끌고 가야해서       1
새로운 장비 도입으로 업무효율성 극대화           1
Name: bq18_10, Length: 1258, dtype: int64

In [503]:
df_2020.drop(['bq5_2', 'bq18_10', 'bq20_1'], axis = 1, inplace = True)

##### 나머지 처리

- bq4_1b, bq4_1c -> 삭제
- 8_1,2,3 -> 평균 후 인트형으로
- 12 -> 전반적으로만 쓰기
- 13 -> 평균 후 인트형으로
- saq*_2번 다 삭제

In [504]:
#8_1,2,3 -> 평균 후 인트형으로
df_2020['bq8'] = (df_2020['bq8_1'] + df_2020['bq8_2'] + df_2020['bq8_3'])/3

In [505]:
df_2020.bq8 #처리 전

0       4.333333
1       4.333333
2       4.000000
3       4.000000
4       3.333333
          ...   
8117    2.666667
8118    2.333333
8119    3.666667
8120    4.000000
8121    3.333333
Name: bq8, Length: 8122, dtype: float64

In [506]:
df_2020.bq8 = df_2020.bq8.round(0).astype(int) # 처리완료

In [507]:
df_2020.drop(['bq8_1', 'bq8_2', 'bq8_3'], axis=1, inplace=True)

In [508]:
# 12 -> 전반적으로만 쓰기
df_2020.drop(['bq12_' + str(i) for i in range(1,5)], axis = 1, inplace=True)

In [509]:
df_2020.rename(columns={'bq12_5':'bq12'}, inplace = True)

In [510]:
# 13 -> 평균 후 인트형으로
df_2020['bq13'] = (df_2020['bq13_1'] + df_2020['bq13_2'] + df_2020['bq13_3'])/3

In [511]:
df_2020.bq13 = df_2020.bq13.round(0).astype(int)

In [512]:
df_2020.drop(['bq13_1', 'bq13_2', 'bq13_3'], axis=1, inplace=True)

In [513]:
# saq*_2번 다 삭제
df_2020.drop(['saq' + str(i) + '_2' for i in range(1,45)], axis=1, inplace=True)

In [514]:
processing_test_2020 = df_2020

---


## 2.5 자격증 데이터 전처리

In [515]:
processing_train_allyear = [processing_train_2017,
                           processing_train_2018, 
                            processing_train_2019,
                            processing_train_2020
                          ]
processing_test_allyear = [processing_test_2017,
                          processing_test_2018, 
                           processing_test_2019,
                           processing_test_2020
                         ]

### 자격증목록 통합(외부데이터 사용, 한국산업인력공단)

In [516]:
npc = pd.read_csv('./used_file/국가전문자격목록.csv', encoding='cp949')
pq = pd.read_excel('./used_file/민간자격목록(수정).xlsx')

In [517]:
npc = npc[['자격명']]

In [518]:
pq = pq['자격명'].unique()

In [519]:
pq = pd.DataFrame(pq)

In [520]:
pq.columns = ['자격명']

In [521]:
npc = npc['자격명'].unique()

In [522]:
npc = pd.DataFrame(npc)

In [523]:
npc.columns = ['자격명']

In [524]:
ntc_total = pd.DataFrame({'자격명':[0]})

In [525]:
for i in range(0,26):
    ntc_total = pd.concat([ntc_total, pd.read_excel('./used_file/국가기술자격목록.xls', sheet_name = i).iloc[:,[-1]]])

In [526]:
ntc_total

,자격명,비고,개정'20.09.08.,개정 '19.06.11.
0,0.0,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
...,...,...,...,...
36,NaN,NaN,에너지관리산업기사,NaN
37,NaN,NaN,에너지관리기능사,NaN
38,NaN,NaN,신재생에너지발전설비기사(태양광),NaN
39,NaN,NaN,신재생에너지발전설비산업기사(태양광),NaN


In [527]:
ntc_total[ntc_total.iloc[:,-1].notnull() == True]

# 확인해보니 세탁기능사  --> 다른 항목으로 들어가있으므로 null값 그대로 처리

,자격명,비고,개정'20.09.08.,개정 '19.06.11.
0,NaN,NaN,NaN,고용노동부령 제255호
3,NaN,NaN,NaN,[19 섬유∙의복 분야로 이관]


In [528]:
ntc_total.isin(['세탁기능사']).sum().sum()

1

In [529]:
ntc_total = ntc_total.iloc[:,[2]]

In [530]:
ntc_total.columns = ['자격명']

In [531]:
ntc_total.dropna(inplace = True)

In [532]:
ntc_total = ntc_total.replace({'고용노동부령 제293호':np.NaN})

In [533]:
ntc_total = ntc_total.dropna()

In [534]:
ntc_total['자격명'].unique()

array(['사회조사분석사1급', '사회조사분석사2급', '소비자전문상담사1급', '소비자전문상담사2급', '컨벤션기획사1급',
       '컨벤션기획사2급', '전산회계운용사1급', '전산회계운용사2급', '전산회계운용사3급', '비서1급', '비서2급',
       '비서3급', '워드프로세서', '컴퓨터활용능력1급', '컴퓨터활용능력2급', '한글속기1급', '한글속기2급',
       '한글속기3급', '공장관리기술사', '포장기술사', '포장기사', '포장산업기사', '품질경영기사',
       '품질경영산업기사', '품질관리기술사', '임상심리사1급', '임상심리사2급', '국제의료관광코디네이터',
       '직업상담사1급', '직업상담사2급', '서비스∙경험디자인기사', '시각디자인기사', '시각디자인산업기사',
       '웹디자인기능사', '제품디자인기술사', '제품디자인기사', '제품디자인산업기사', '제품응용모델링기능사',
       '컬러리스트기사', '컬러리스트산업기사', '컴퓨터그래픽스운용기능사', '영사산업기사', '영사기능사',
       '농기계운전기능사', '철도운송산업기사', '전자상거래관리사1급', '전자상거래관리사2급', '전자상거래운용사',
       '텔레마케팅관리사', '이용장', '이용사', '미용장', '미용사(일반)', '미용사(피부)', '미용사(네일)',
       '미용사(메이크업)', '스포츠경영관리사', '조리기능장', '한식조리산업기사', '한식조리기능사',
       '중식조리산업기사', '중식조리기능사', '양식조리산업기사', '양식조리기능사', '일식조리산업기사',
       '일식조리기능사', '복어조리산업기사', '복어조리기능사', '조주기능사', '거푸집기능사', '건축구조기술사',
       '건축기계설비기술사', '건축기사', '건축산업기사', '건축도장기능사', '건축목공산업기사', '건축목공기능사',
       '건축목재시공기능장', '건축설비기사', '건

In [535]:
ntc_total = ntc_total.reset_index().drop('index', axis = 1)

In [536]:
ntc_total['자격명수정'] = '-'

In [537]:
ntc_total['자격명수정'] = ntc_total['자격명']

In [538]:
for i in range(len(ntc_total.index)):
    if ntc_total['자격명'].str.startswith('(신)')[i] == True :
        ntc_total['자격명수정'][i] = ntc_total['자격명'][i][3:]

In [539]:
for i in range(len(ntc_total.index)):
    for j in range(len(ntc_total['자격명'][i])):
        if (ntc_total['자격명'][i][j] == '(' ) == True :
            ntc_total['자격명수정'][i] = ntc_total['자격명'][i][:j]

In [540]:
ntc_total[ntc_total['자격명수정']=='<폐 지>']

,자격명,자격명수정
328,<폐 지>,<폐 지>
514,<폐 지>,<폐 지>


In [541]:
ntc_total.drop([328, 514], inplace = True)

In [542]:
ntc = ntc_total['자격명수정'].unique()

In [543]:
ntc = pd.DataFrame(ntc)

In [544]:
ntc.columns = ['자격명']

In [545]:
ntc['자격구분'] = '국가기술자격'
npc['자격구분'] = '국가전문자격'
pq['자격구분'] = '민간자격'

In [546]:
npc

,자격명,자격구분
0,가축인공수정사,국가전문자격
1,농산물검사관,국가전문자격
2,농산물품질관리사,국가전문자격
3,말조련사,국가전문자격
4,장제사,국가전문자격
...,...,...
173,국방사업관리사,국가전문자격
174,버스운전자격,국가전문자격
175,어린이집원장,국가전문자격
176,철도교통 관제자격증명,국가전문자격


In [547]:
cert_list = pd.merge(npc, pq, on = '자격명', how = 'outer')

In [548]:
cert_list = pd.merge(cert_list, ntc,  on = '자격명', how = 'outer')

In [549]:
cert_list = cert_list.fillna(0)

In [550]:
for i in range(len(cert_list.index)):
    if (cert_list['자격구분'][i] == 0)==True & (cert_list['자격구분_y'][i] == 0)== True:
        cert_list['자격구분'][i] = cert_list['자격구분_x'][i]
    elif (cert_list['자격구분'][i] == 0)==True & (cert_list['자격구분_x'][i] == 0)== True:
        cert_list['자격구분'][i] = cert_list['자격구분_y'][i]

In [551]:
cert_list = cert_list.iloc[:,[0,3]]

### 자격증 목록에 라벨링 작업

In [552]:
cert_list = cert_list.astype(str)

In [553]:
cert_list['수정자격명'] = cert_list['자격명']

In [554]:
# 괄호 삭제
for i in range(len(cert_list.index)):
    for j in range(len(cert_list['자격명'][i])):
        if cert_list['자격명'][i][j] =='(':
            cert_list['수정자격명'][i] = cert_list['자격명'][i][:j]

In [555]:
# 산업기사 삭제
p = re.compile(".+(?=산업기사)")
for i in range (len(cert_list.index)):
    m = p.search(cert_list['수정자격명'][i])
    
    if m:
        cert_list['수정자격명'][i] = m.group()
    else :
        pass

In [556]:
# 기사 삭제
p = re.compile(".+(?=기사)")
for i in range (len(cert_list.index)):
    m = p.search(cert_list['수정자격명'][i])
    
    if m:
        cert_list['수정자격명'][i] = m.group()
    else :
        pass

In [557]:
# 기능사 삭제
p = re.compile(".+(?=기능사)")
for i in range (len(cert_list.index)):
    m = p.search(cert_list['수정자격명'][i])
    
    if m:
        cert_list['수정자격명'][i] = m.group()
    else :
        pass

In [558]:
# 기술사 삭제
p = re.compile(".+(?=기술사)")
for i in range (len(cert_list.index)):
    m = p.search(cert_list['수정자격명'][i])
    
    if m:
        cert_list['수정자격명'][i] = m.group()
    else :
        pass

In [559]:
# 기능장 삭제
p = re.compile(".+(?=기능장)")
for i in range (len(cert_list.index)):
    m = p.search(cert_list['수정자격명'][i])
    
    if m:
        cert_list['수정자격명'][i] = m.group()
    else :
        pass

In [560]:
# 면허 삭제
p = re.compile(".+(?=면허)")
for i in range (len(cert_list.index)):
    m = p.search(cert_list['수정자격명'][i])
    
    if m:
        cert_list['수정자격명'][i] = m.group()
    else :
        pass

In [561]:
# 자격 삭제
p = re.compile(".+(?=자격)")
for i in range (len(cert_list.index)):
    m = p.search(cert_list['수정자격명'][i])
    
    if m:
        cert_list['수정자격명'][i] = m.group()
    else :
        pass

In [562]:
# 자격증의 수준은 고려하지 않기로 함. 자격증목록의 고유값으로 데이터프레임 생성
mod_dic = pd.DataFrame(cert_list['수정자격명'].unique())
mod_dic.columns = ['자격명']

In [563]:
mod_dic = mod_dic.sort_values('자격명').reset_index().drop('index', axis = 1)

In [564]:
# 라벨을 위한 컬럼 생성
mod_dic['id'] = mod_dic.index

In [565]:
# 자격증목록과 일치 하지 않는 값들에 대해선 기타 처리 예정
etc = pd.DataFrame({'자격명' : ['기타'], 'id' : [615]})

In [566]:
mod_dic = mod_dic.append(etc, ignore_index = True)

In [567]:
mod_dic = mod_dic.drop(0)

In [568]:
# 자격증목록 전처리 끝
mod_dic['자격명'] = mod_dic['자격명'].astype(str)

### 자격증 처리 함수

In [569]:
def cert_processing(cert_df):
    cert = cert_df
    cert.columns = ['자격명']
    cert['수정자격명'] = cert['자격명']
    
    # 괄호 삭제
    for i in range(len(cert.index)):
        for j in range(len(cert['자격명'][i])):            
            
            if cert['자격명'][i][j] =='(':
                cert['수정자격명'][i] = cert['자격명'][i][:j]
                
    p = re.compile(".+(?=산업기사)")
    for i in range (len(cert.index)):
        m = p.search(cert['수정자격명'][i])

        if m:
            cert['수정자격명'][i] = m.group()
        else :
            pass
        
    p = re.compile(".+(?=기사)")
    for i in range (len(cert.index)):
        m = p.search(cert['수정자격명'][i])

        if m:
            cert['수정자격명'][i] = m.group()
        else :
            pass
        
    p = re.compile(".+(?=기능사)")
    for i in range (len(cert.index)):
        m = p.search(cert['수정자격명'][i])

        if m:
            cert['수정자격명'][i] = m.group()
        else :
            pass
        
    p = re.compile(".+(?=기술사)")
    for i in range (len(cert.index)):
        m = p.search(cert['수정자격명'][i])

        if m:
            cert['수정자격명'][i] = m.group()
        else :
            pass
        
    p = re.compile(".+(?=기능장)")
    for i in range (len(cert.index)):
        m = p.search(cert['수정자격명'][i])

        if m:
            cert['수정자격명'][i] = m.group()
        else :
            pass
        
    p = re.compile(".+(?=자격)")
    for i in range (len(cert.index)):
        m = p.search(cert['수정자격명'][i])

        if m:
            cert['수정자격명'][i] = m.group()
        else :
            pass
        
    p = re.compile(".+(?=면허)")
    for i in range (len(cert.index)):
        m = p.search(cert['수정자격명'][i])

        if m:
            cert['수정자격명'][i] = m.group()
        else :
            pass
        
    cert = cert.iloc[:,[1]]
    cert.columns = ['자격명']
    
    cert = pd.merge(cert, mod_dic, on='자격명', how = 'left')
    
    cert = cert.fillna(999)
    
    ## '기타'로 포함된 자격증 중 발생빈도가 높은 자젹증 재할당

    # 자동차운전 (384) 로 변환 
    cert['id'] = cert.apply(lambda row :  384 if row['자격명'] == '운전'else row['id'], axis = 1 )
    # 한식조리 (559) 으로 변환 
    cert['id'] = cert.apply(lambda row : 559 if row['자격명'] == '한식조리사' else row['id'], axis = 1)
    # 양식조리 (315) 으로 변환 
    cert['id'] = cert.apply(lambda row : 315 if row['자격명'] == '양식조리사' else row['id'], axis = 1)
    # 중식조리 (470) 으로 변환 
    cert['id'] = cert.apply(lambda row : 470 if row['자격명'] == '중식조리사' else row['id'], axis = 1)
    # 일식조리 (373) 으로 변환 
    cert['id'] = cert.apply(lambda row : 373 if row['자격명'] == '일식조리사' else row['id'], axis = 1)

    # 1종대형운전 (617) 으로 통일 
    cert['id'] = cert.apply(lambda row : 617 if row['자격명'] == '대형운전' else row['id'], axis = 1)
    cert['id'] = cert.apply(lambda row : 617 if row['자격명'] == '1종대형' else row['id'], axis = 1)
    cert['id'] = cert.apply(lambda row : 617 if row['자격명'] == '대형1종' else row['id'], axis = 1)
    cert['id'] = cert.apply(lambda row : 617 if row['자격명'] == '대형' else row['id'], axis = 1)
    cert['id'] = cert.apply(lambda row : 617 if row['자격명'] == '1종운전' else row['id'], axis = 1)
    cert['id'] = cert.apply(lambda row : 617 if row['자격명'] == '1종' else row['id'], axis = 1)

    # 컴퓨터활용능력2급 (519) 으로 변환
    cert['id'] = cert.apply(lambda row : 519 if row['자격명'] == '컴퓨터활용능력' else row['id'], axis = 1)
    cert['id'] = cert.apply(lambda row : 519 if row['자격명'] == '컴퓨터활용' else row['id'], axis = 1)
    cert['id'] = cert.apply(lambda row : 519 if row['자격명'] == '컴퓨터' else row['id'], axis = 1)
    cert['id'] = cert.apply(lambda row : 519 if row['자격명'] == '컴활2급' else row['id'], axis = 1)

    # 워드프로세서 (341) 으로 변환
    cert['id'] = cert.apply(lambda row : 341 if row['자격명'] == '워드' else row['id'], axis = 1)
    cert['id'] = cert.apply(lambda row : 341 if row['자격명'] == '워드프로세스' else row['id'], axis = 1)
    cert['id'] = cert.apply(lambda row : 341 if row['자격명'] == '워드프로세서1급' else row['id'], axis = 1)


    # 데이터에 입력
    cert_result = cert['id']
    
    return cert_result

### 자격증 처리 함수 적용 

In [570]:
# 자격증 처리 함수 적용 - 4개년도 train
for train_year in processing_train_allyear:
    train_year['bq4_1a'] = cert_processing(train_year[['bq4_1a']])

In [571]:
# 자격증 처리 함수 적용 - 4개년도  test
for test_year in processing_test_allyear:
    test_year['bq4_1a'] = cert_processing(test_year[['bq4_1a']])

## 2.6 전공 데이터 전처리

### 전공 라벨링 함수

In [572]:
def major_processing(major_df):
    major = major_df
    major.columns = ['전공']
    major['수정전공'] = major['전공'].astype(str)
    

    for i in range (len(major.index)):
        major['수정전공'][i] = re.sub('계$','',major['수정전공'][i])
    for i in range (len(major.index)):
        major['수정전공'][i] = re.sub('과$','',major['수정전공'][i])
    for i in range (len(major.index)):
        major['수정전공'][i] = re.sub('학$','',major['수정전공'][i])
    for i in range (len(major.index)):
        major['수정전공'][i] = re.sub('공$','',major['수정전공'][i])
    
    for i in range(len(major.index)):

        # 인문계, 인문 통합
        if '인문' in major['수정전공'][i]:
            major['수정전공'][i] = '인문'

        # 기계, 기 통합
        elif '기계' in major['수정전공'][i]:
            major['수정전공'][i] = '기'

        # 화학, 화 통합
        elif '화학' in major['수정전공'][i]:
            major['수정전공'][i] = '화'

        # 영어, 영문, 영어영문 통합
        elif '영어' in major['수정전공'][i]:
            major['수정전공'][i] = '영어영문'
        elif '영문' in major['수정전공'][i]:
            major['수정전공'][i] = '영어영문'

        # 국어, 국문, 국어국문 통합
        elif '국어' in major['수정전공'][i]:
            if major['수정전공'][i] != '국어교육':
                major['수정전공'][i] = '국어국문'

        elif '국문' in major['수정전공'][i]:
            major['수정전공'][i] = '국어국문'


        # 일본, 일본어, 일어일문 통합
        elif '일본' in major['수정전공'][i]:
            major['수정전공'][i] = '일어일문'

        # 중국, 중국어, 중어중문 통합
        elif '중국' in major['수정전공'][i]:
            major['수정전공'][i] = '중어중문'

        # 회계, 세무회계 통합
        elif '회계' in major['수정전공'][i]:
            major['수정전공'][i] = '회계'


        elif '경영' in major['수정전공'][i]:
            major['수정전공'][i] = '경영'

        elif '전기' in major['수정전공'][i]:
            if major['수정전공'][i] != '전기전자':
                major['수정전공'][i] = '전기'

        elif '전자' in major['수정전공'][i]:
            if major['수정전공'][i] != '전기전자' or major['수정전공'][i] != '유전자':
                major['수정전공'][i] = '전자'

        elif '건축' in major['수정전공'][i]:
            major['수정전공'][i] = '건축'

        elif '컴퓨터' in major['수정전공'][i]:
            major['수정전공'][i] = '컴퓨터'

        elif '헹정' in major['수정전공'][i]:
            major['수정전공'][i] = '행정'

        elif '법' in major['수정전공'][i]:
            major['수정전공'][i] = '법'

        elif '의예' in major['수정전공'][i]:
            major['수정전공'][i] = '의'

        elif '토목' in major['수정전공'][i]:
            major['수정전공'][i] = '토목'

        elif '경제' in major['수정전공'][i]:
            major['수정전공'][i] = '경제'

        elif '사회복지' in major['수정전공'][i]:
            major['수정전공'][i] = '사회복지'

        elif '환경' in major['수정전공'][i]:
            major['수정전공'][i] = '환경'

        elif '통신' in major['수정전공'][i]:
            major['수정전공'][i] = '정보통신'

        elif '간호' in major['수정전공'][i]:
            major['수정전공'][i] = '간호'

        elif '유아' in major['수정전공'][i]:
            major['수정전공'][i] = '유아교육'

        elif '식품' in major['수정전공'][i]:
            major['수정전공'][i] = '식품영양'

        elif '아동' in major['수정전공'][i]:
            major['수정전공'][i] = '식품영양'

        elif '공고' in major['수정전공'][i]:
            major['수정전공'][i] = '공고'

        elif '자동차' in major['수정전공'][i]:
            major['수정전공'][i] = '자동차'

        elif '조리' in major['수정전공'][i]:
            major['수정전공'][i] = '자동차'

        elif '음악' in major['수정전공'][i]:
            major['수정전공'][i] = '음악'

        elif '미용' in major['수정전공'][i]:
            major['수정전공'][i] = '미용'

        elif '뷰티' in major['수정전공'][i]:
            major['수정전공'][i] = '뷰티'
            
        bq27_1 = major['수정전공'].value_counts()    
        bq27_one = bq27_1[major['수정전공'].value_counts() == 1].index
        major.loc[major['수정전공'].isin(list(bq27_one)),'수정전공'] = '기타'
        

        # 데이터에 입력
        major_result = major['수정전공']

        return major_result

### 전공 라밸링 함수 적용

In [573]:
processing_train_2017['bq38_1'] = major_processing( processing_train_2017[['bq38_1']])
processing_train_2018['bq37_1'] = major_processing( processing_train_2018[['bq37_1']])
processing_train_2019['bq27_1'] = major_processing( processing_train_2019[['bq27_1']])
processing_train_2020['bq26_1'] = major_processing( processing_train_2020[['bq26_1']])

In [574]:
processing_test_2017['bq38_1'] = major_processing( processing_test_2017[['bq38_1']])
processing_test_2018['bq37_1'] = major_processing( processing_test_2018[['bq37_1']])
processing_test_2019['bq27_1'] = major_processing( processing_test_2019[['bq27_1']])
processing_test_2020['bq26_1'] = major_processing( processing_test_2020[['bq26_1']])

## 2.7 나이/연봉 그룹화 

- 연도별 나이 칼럼 리스트 

In [575]:
age_cols_allyear = [
                    'bq37', # 2017
                    'bq36', # 2018 
                    'bq26', # 2019
                    'bq25'  # 2020 
                    ]

- 연도별 임금근로자 근로소득(연봉), 임금근로자 초임임금(연봉), 비임금근로자 순수입(연봉) 리스트

In [576]:
salary_cols_allyear = [ 
                        ['bq41_1','bq41_2','bq41_3'], # 2017
                        ['bq41_1','bq41_2','bq41_3'], # 2018
                        ['bq31_1','bq31_2','bq31_3'], # 2019 
                        ['bq30_1','bq30_2','bq30_3']  # 2020
                      ]

- 연봉 그룹핑 결과 저장 칼럼 리스트 

In [577]:
salary_result_cols = [
                    'mod_bq2',
                    'mod_bq3',
                    'mod_bq4'
                    ]

### 2.7.1. 나이 그룹화

#### 1) 나이 그룹화 함수 

In [578]:
def age_grouping(input_age):    
    if input_age <30:
        grouped_result = 1
    elif input_age < 40:
        grouped_result = 2
    elif input_age < 50:
         grouped_result= 3
    elif input_age < 60:
         grouped_result = 4
    else:
         grouped_result = 5
            
    return  grouped_result

#### 2) 나이 그룹화 함수 적용

In [579]:
# 나이 그룹화 함수적용 - 4개년도 train
for train_year, age_col in zip(processing_train_allyear, age_cols_allyear):
    train_year['mod_bq1'] = '-'
    train_year['mod_bq1'] = train_year[age_col].apply(lambda x : age_grouping(x))

In [580]:
# 나이 그룹화 함수적용 - 4개년도 test
for test_year, age_col in zip(processing_test_allyear, age_cols_allyear):
    test_year['mod_bq1'] = '-'
    test_year['mod_bq1'] = test_year[age_col].apply(lambda x : age_grouping(x))

### 2.7.2 연봉 그룹화

#### 1) 연봉 그룹화 함수 

In [581]:
def salary_grouping(input_salary):
    if input_salary ==0:
        grouped_result = 0
    elif input_salary <2000:
        grouped_result= 1
    elif input_salary < 3000:
        grouped_result = 2
    elif input_salary< 4000:
        grouped_result= 3
    elif input_salary < 5000:
        grouped_result= 4
    elif input_salary < 6000:
        grouped_result = 5
    elif input_salary< 7000:
        grouped_result = 6
    elif input_salary < 8000:
        grouped_result = 7
    elif input_salary< 9000:
        grouped_result= 8
    elif input_salary < 10000:
        grouped_result = 9
    else:
        grouped_result= 10
        
    return grouped_result

#### 2) 연봉 그룹화 함수 적용 

In [582]:
# 나이 그룹화 함수적용 - 4개년도 train
for train_year, salary_cols_year in zip(processing_train_allyear, salary_cols_allyear):
    
    train_year['mod_bq2']='-'
    train_year['mod_bq3']='-'
    train_year['mod_bq4']='-'
    
    for salary_col, result_col in zip(salary_cols_year, salary_result_cols):  # 연도마다 3개의 연봉관련 칼럼 

        train_year[result_col] = train_year[salary_col].apply(lambda x : salary_grouping(x))

In [583]:
# 나이 그룹화 함수적용 - 4개년도 test
for test_year, salary_cols_year in zip(processing_test_allyear, salary_cols_allyear):
    
    test_year['mod_bq2']='-'
    test_year['mod_bq3']='-'
    test_year['mod_bq4']='-'
    
    for salary_col, result_col in zip(salary_cols_year, salary_result_cols):  # 연도마다 3개의 연봉관련 칼럼 

        test_year[result_col] = test_year[salary_col].apply(lambda x : salary_grouping(x))

In [ ]:
for train_year, age_col in zip(processing_train_allyear, age_cols_allyear):
    train_year['mod_bq1'] = train_year[age_col].apply(lambda x : age_grouping(x))

for train_year, salary_cols_year in zip(processing_train_allyear, salary_cols_allyear):
    for salary_col, result_col in zip(salary_cols_year, salary_result_cols):  
        train_year[result_col] = train_year[salary_col].apply(lambda x : salary_grouping(x))

### 2.7.3. 그룹화 이후 필요없는 칼럼 정리 

In [584]:
processing_train_2017 = processing_train_2017.drop(['bq37', 'bq41_1', 'bq41_2', 'bq41_3'], axis = 1)
processing_test_2017 = processing_test_2017.drop(['bq37', 'bq41_1', 'bq41_2', 'bq41_3'], axis = 1)

In [585]:
processing_train_2018 = processing_train_2018.drop(['bq36', 'bq41_1', 'bq41_2', 'bq41_3'], axis = 1)
processing_test_2018 = processing_test_2018.drop(['bq36', 'bq41_1', 'bq41_2', 'bq41_3'], axis = 1)

In [586]:
processing_train_2019 = processing_train_2019.drop(['bq26', 'bq31_1', 'bq31_2', 'bq31_3'], axis = 1)
processing_test_2019 = processing_test_2019.drop(['bq26', 'bq31_1', 'bq31_2', 'bq31_3'], axis = 1)

In [587]:
processing_train_2020 = processing_train_2020.drop(['bq25', 'bq30_1', 'bq30_2', 'bq30_3'], axis = 1)
processing_test_2020 = processing_test_2020.drop(['bq25', 'bq30_1', 'bq30_2', 'bq30_3'], axis = 1)

### 2.7.4 전공 & 자격증 컬럼을 제외한 나머지 컬럼 데이터 형태 int형으로 변환

- 연도별 전공 칼럼 리스트 

In [588]:
major_cols_allyear = [
                            'bq38_1',  # 2017
                            'bq37_1',  # 2018
                            'bq27_1',  # 2019
                            'bq26_1'   # 2020 
]

- 정수화 함수 

In [589]:
def to_integer(input_df_year, input_major_col):
    for col in range(len(input_df_year.columns)):
        if (input_df_year.columns[col] == input_major_col) or (input_df_year.columns[col] == 'bq4_1a'):
            pass
        else:
            input_df_year.iloc[:,col] = input_df_year.iloc[:,col].astype(int)

In [590]:
# 정수화 함수적용 - 4개년도 train
for train_year, major_col in zip(processing_train_allyear, major_cols_allyear):
    to_integer(train_year, major_col)

In [591]:
# 정수화 함수적용 - 4개년도 test
for test_year, major_col in zip(processing_test_allyear, major_cols_allyear):
    to_integer(test_year, major_col)

## 2.8 파생변수 

##### 1) hs : 산업군 & 근무업체 유형을 결합  

- train

In [27]:
processing_train_allyear

NameError: name 'processing_train_allyear' is not defined

In [592]:
# hs 파생변수 생성 함수 적용 - 4개년도 train
for train_year in processing_train_allyear:
    train_year['hs'] = train_year.apply(lambda row : row['bq1']*10 + row['bq2'], axis =1)

- test

In [593]:
# hs 파생변수 생성 함수 적용 - 4개년도 test
for test_year in processing_test_allyear:
    test_year['hs'] = test_year.apply(lambda row : row['bq1']*10 + row['bq2'], axis =1)

### 2017년 단답형 응답 재결합

In [594]:
know_train[0].head()

,idx,aq1_1,aq1_2,aq2_1,aq2_2,aq3_1,aq3_2,aq4_1,aq4_2,aq5_1,...,bq37,bq38,bq38_1,bq39_1,bq39_2,bq40,bq41_1,bq41_2,bq41_3,knowcode
0,0,3,3,3,3,3,3,4,4,3,...,52,2,실업,1,1,1,4000,0,2200,825101
1,1,4,5,4,5,3,4,3,4,3,...,38,4,건축공학,1,1,1,0,0,2400,140204
2,2,3,4,3,4,3,4,5,6,4,...,50,4,건축공학,1,1,1,4000,0,2400,140204
3,3,3,3,3,3,3,5,4,5,4,...,42,4,환경학과,1,1,1,7000,0,3500,140601
4,4,4,5,3,4,3,4,4,5,3,...,51,4,건축공학,1,1,1,4000,0,2500,140204


In [595]:
processing_train_2017['bq5_2'] = know_train[0]['bq5_2']
processing_train_2017['bq30'] = know_train[0]['bq30']
processing_train_2017['bq31'] = know_train[0]['bq31']
processing_train_2017['bq32'] = know_train[0]['bq32']
processing_train_2017['bq33'] = know_train[0]['bq33']
processing_train_2017['bq34'] = know_train[0]['bq34']

In [596]:
processing_test_2017['bq5_2'] = know_test[0]['bq5_2']
processing_test_2017['bq30'] = know_test[0]['bq30']
processing_test_2017['bq31'] = know_test[0]['bq31']
processing_test_2017['bq32'] = know_test[0]['bq32']
processing_test_2017['bq33'] = know_test[0]['bq33']
processing_test_2017['bq34'] = know_test[0]['bq34']

In [597]:
processing_train_2017.replace({'없다':'0', '무':'0', '없음':'0', ' ':'0'}, inplace = True)
processing_test_2017.replace({'없다':'0', '무':'0', '없음':'0', ' ':'0'}, inplace = True)

In [598]:
prep_train = [processing_train_2017,
                           processing_train_2018, 
                            processing_train_2019,
                            processing_train_2020
             ]

prep_test = [processing_test_2017,
                          processing_test_2018, 
                           processing_test_2019,
                           processing_test_2020
            ]

In [599]:
processing_train_2017.to_csv('./prep_file/train/prep_train_2017.csv', encoding='utf-8', index=False)
processing_train_2018.to_csv('./prep_file/train/prep_train_2018.csv', encoding='utf-8', index=False)
processing_train_2019.to_csv('./prep_file/train/prep_train_2019.csv', encoding='utf-8', index=False)
processing_train_2020.to_csv('./prep_file/train/prep_train_2020.csv', encoding='utf-8', index=False)

In [600]:
processing_test_2017.to_csv('./prep_file/test/prep_test_2017.csv', encoding='utf-8', index=False)
processing_test_2018.to_csv('./prep_file/test/prep_test_2018.csv', encoding='utf-8', index=False)
processing_test_2019.to_csv('./prep_file/test/prep_test_2019.csv', encoding='utf-8', index=False)
processing_test_2020.to_csv('./prep_file/test/prep_test_2020.csv', encoding='utf-8', index=False)

## 모델링

## 라벨인코딩

### 트레인데이터 전처리

In [601]:
years = ['2017', '2018', '2019', '2020']

year_encoder = {}

for year, df in zip(years, prep_train):
    print(year)
    encoders = {}
    
    for col in df.columns:
        if col == 'ID':
            continue
        
        try:
            df[col] = df[col].map(int) # 숫자 데이터들은 모두 인트형으로 매핑될 것임, 아닌 것들은 에러가 난다. 그 에러나는 컬럼들은 except문 실행됨!
        except:
            encoder = LabelEncoder()
            df[col] = df[col].map(str) # obj -> str로 매핑한 후
            df[col] = encoder.fit_transform(df[col]) # 인코더 적용함. good
            encoders[col] = encoder
            
            
    year_encoder[year] = encoders

2017
2018
2019
2020


In [602]:
# check point
print(year_encoder['2017'], year_encoder['2019'], year_encoder['2018'], year_encoder['2020']) # 학과만 인코딩 잘 되었는지 확인

{'bq38_1': LabelEncoder(), 'bq5_2': LabelEncoder(), 'bq30': LabelEncoder(), 'bq31': LabelEncoder(), 'bq32': LabelEncoder(), 'bq33': LabelEncoder(), 'bq34': LabelEncoder()} {'bq27_1': LabelEncoder()} {'bq37_1': LabelEncoder()} {'bq26_1': LabelEncoder()}


### 테스트데이터 전처리

In [603]:
years = ['2017', '2018', '2019', '2020']

for year, df in zip(years, prep_test):
    print(year)
    #print(df)
    encoders = {}
    
    for col in df.columns:
        
        try:
            df[col] = df[col].map(int)
        except:
            encoder = year_encoder[year][col]
            df[col] = df[col].map(str)
            category_map = {category: idx for idx, category in enumerate(encoder.classes_)}
            df[col] = df[col].apply(lambda x: category_map[x] if x in category_map else -1) # train set에서 보지못한 카테고리변수 -1(UNK) 처리

2017
2018
2019
2020


## train/test data X, y 분할 후 딕셔너리 저장 

train_data/test data에 저장함

In [604]:
train_data = {}
for year, df in zip(years, prep_train):
    train_data[year] = {'X': df.drop(['knowcode','idx'], axis=1), # idx제외
                        'y': df['knowcode']}

In [605]:
test_data = {}
for year, df in zip(years, prep_test):
    test_data[year] =  {'X': df.iloc[:,1:]}

# Modeling

연도마다 개별적으로 배깅(ensemble of ensembles) 구현

- 추가 아이디어 : 모델을 리스트로 구현하여 각 라운드마다 다른 모델 파라미터 써보기, 배깅 라운드 바꿔서 시도해보기

In [606]:
## 리스트 최빈값 뽑는 함수 ##

from collections import Counter

def modefinder(numbers): #리스트 or 튜플형태의 input
    c = Counter(numbers)
    order = c.most_common()
    maximum = order[0][1]
    
    modes = []
    for num in order:
        if num[1] == maximum:
            modes.append(num[0])
    return modes
# 최빈값이 두개 있다면 그것들 리스트로 만든 후 첫번째 인덱스 값을 가져옴(이게 최선)

## 2017년

최종 예측값 bagging_predict_2017에 리스트로 저장됨

train_x -> train_data['2017']['X'] 

train_y -> train_data['2017']['y']

valid_x => test_data['2017']['X']

서술형 문항을 제외한 텍스트 데이터를 포함 

In [607]:
rf_model = RandomForestClassifier(n_estimators=2100,class_weight='balanced', n_jobs=-1, random_state=2800) # 모델 구축

In [608]:
bagging_predict_result_2017 = [] # 빈 리스트 생성
bootstrapped_data_2017 = []
for _ in tqdm(range(100)):
    data_index = [data_index for data_index in range(train_data['2017']['X'].shape[0])] # 학습 데이터의 인덱스를 리스트로 변환
    random_data_index = np.random.choice(data_index, train_data['2017']['X'].shape[0]) # 데이터의 크기만큼 랜덤 샘플링, // 는 소수점을 무시하기 위함
    bootstrapped_data_2017.append(len(set(random_data_index)))
    rf_train_x = train_data['2017']['X'].iloc[random_data_index, ] # 랜덤 인덱스에 해당되는 학습 데이터 중 설명 변수
    rf_train_y =  train_data['2017']['y'].iloc[random_data_index, ] # 랜덤 인덱스에 해당되는 학습 데이터 중 종속 변수
    
    fitted_sm_model = rf_model.fit(rf_train_x, rf_train_y) # 학습 진행
    
    sm_model_predict = fitted_sm_model.predict(test_data['2017']['X']) # 테스트 데이터 예측
    bagging_predict_result_2017.append(sm_model_predict) # 반복문이 실행되기 전 빈 리스트에 결과 값 저장
print(f"각 라운드마다 쓰인 데이터 개수 : {bootstrapped_data_2017}")

100%|█████████████████████████████████████████| 100/100 [00:19<00:00,  5.07it/s]

각 라운드마다 쓰인 데이터 개수 : [6013, 6024, 5988, 6047, 6014, 6017, 5971, 6002, 6018, 5993, 6011, 5977, 6014, 5997, 5979, 6007, 5956, 5956, 5965, 5998, 6014, 6053, 5980, 5981, 6039, 5947, 5990, 6020, 6048, 6010, 6033, 5995, 6023, 5994, 5937, 5980, 5971, 5989, 5964, 5984, 6000, 5985, 6002, 6073, 5973, 6009, 5979, 6040, 5988, 5988, 6013, 5925, 5993, 6020, 5979, 6017, 6010, 5982, 5957, 5994, 6004, 5966, 6001, 6027, 5990, 5944, 5994, 5967, 6055, 6058, 5986, 6006, 5983, 6030, 6013, 6057, 5964, 5944, 5993, 6000, 6005, 6006, 5974, 6009, 5963, 5959, 5969, 5949, 5983, 5975, 5999, 6004, 6022, 6009, 6080, 6026, 6051, 5998, 6017, 5969]


예측한 데이터 bagging_predict_result_2017에 저장됨 -> 최빈값 뽑기

In [609]:
# check point
bagging_predict_result_2017[0]

array([ 23403,  31501,  12301, ..., 704001,  12201, 131101])

In [610]:
# Bagging을 바탕으로 예측한 결과값에 대한 최빈값을 계산
bagging_predict_2017 = [] # 빈 리스트 생성
for lst2_index in range(test_data['2017']['X'].shape[0]): # 테스트 데이터 개수만큼의 반복
    temp_predict = [] # 임시 빈 리스트 생성 (반복문 내 결과값 저장)
    for lst_index in range(len(bagging_predict_result_2017)): # Bagging 결과 리스트 반복 10번씩 반복
        temp_predict.append(bagging_predict_result_2017[lst_index][lst2_index]) # 각 Bagging 결과 예측한 값 중 같은 인덱스를 리스트에 저장
        #print(temp_predict)
    bagging_predict_2017.append(modefinder(temp_predict)[0]) # 해당 인덱스의 30개의 결과값에서 최빈값을 최종 리스트에 추가

In [611]:
# check point
print(len(bagging_predict_2017), len(test_data['2017']['X']))

9486 9486


## 2018년

최종 예측값 bagging_predict_2018에 리스트로 저장됨

In [612]:
bagging_predict_result_2018 = [] # 빈 리스트 생성
bootstrapped_data_2018 = []
for _ in tqdm(range(100)):
    data_index = [data_index for data_index in range(train_data['2018']['X'].shape[0])] # 학습 데이터의 인덱스를 리스트로 변환
    random_data_index = np.random.choice(data_index, train_data['2018']['X'].shape[0]) # 데이터의 1/10 크기만큼 랜덤 샘플링, // 는 소수점을 무시하기 위함
    bootstrapped_data_2018.append(len(set(random_data_index)))
    rf_train_x = train_data['2018']['X'].iloc[random_data_index, ] # 랜덤 인덱스에 해당되는 학습 데이터 중 설명 변수
    rf_train_y =  train_data['2018']['y'].iloc[random_data_index, ] # 랜덤 인덱스에 해당되는 학습 데이터 중 종속 변수
    
    fitted_sm_model = rf_model.fit(rf_train_x, rf_train_y) # 학습 진행
    
    sm_model_predict = fitted_sm_model.predict(test_data['2018']['X']) # 테스트 데이터 예측
    bagging_predict_result_2018.append(sm_model_predict) # 반복문이 실행되기 전 빈 리스트에 결과 값 저장
print(f"각 라운드마다 쓰인 데이터 개수 : {bootstrapped_data_2018}")

100%|█████████████████████████████████████████| 100/100 [00:16<00:00,  6.23it/s]

각 라운드마다 쓰인 데이터 개수 : [5750, 5715, 5758, 5775, 5744, 5787, 5723, 5723, 5766, 5729, 5741, 5751, 5719, 5784, 5709, 5711, 5740, 5720, 5761, 5724, 5749, 5696, 5727, 5723, 5743, 5740, 5709, 5690, 5772, 5723, 5738, 5766, 5760, 5767, 5783, 5733, 5729, 5752, 5754, 5748, 5743, 5702, 5712, 5775, 5788, 5719, 5729, 5774, 5747, 5732, 5754, 5736, 5722, 5696, 5761, 5716, 5692, 5738, 5759, 5741, 5762, 5758, 5758, 5766, 5720, 5797, 5717, 5727, 5739, 5728, 5717, 5709, 5735, 5727, 5768, 5748, 5758, 5748, 5704, 5721, 5744, 5724, 5688, 5710, 5742, 5783, 5774, 5715, 5667, 5736, 5746, 5722, 5757, 5735, 5780, 5771, 5748, 5745, 5765, 5719]


In [613]:
# Bagging을 바탕으로 예측한 결과값에 대한 최빈값을 계산
bagging_predict_2018 = [] # 빈 리스트 생성
for lst2_index in range(test_data['2018']['X'].shape[0]): # 테스트 데이터 개수만큼의 반복
    temp_predict = [] # 임시 빈 리스트 생성 (반복문 내 결과값 저장)
    for lst_index in range(len(bagging_predict_result_2018)): # Bagging 결과 리스트 반복 10번씩 반복
        temp_predict.append(bagging_predict_result_2018[lst_index][lst2_index]) # 각 Bagging 결과 예측한 값 중 같은 인덱스를 리스트에 저장
        #print(temp_predict)
    bagging_predict_2018.append(modefinder(temp_predict)[0]) # 해당 인덱스의 30개의 결과값에서 최빈값을 최종 리스트에 추가

In [614]:
# check point
print(len(bagging_predict_2018), len(test_data['2018']['X']))

9069 9069


## 2019년

최종 예측값 bagging_predict_2019에 리스트로 저장됨

In [615]:
bagging_predict_result_2019 = [] # 빈 리스트 생성
bootstrapped_data_2019 = []
for _ in tqdm(range(100)):
    data_index = [data_index for data_index in range(train_data['2019']['X'].shape[0])] # 학습 데이터의 인덱스를 리스트로 변환
    random_data_index = np.random.choice(data_index, train_data['2019']['X'].shape[0]) # 데이터의 1/10 크기만큼 랜덤 샘플링, // 는 소수점을 무시하기 위함
    bootstrapped_data_2019.append(len(set(random_data_index)))
    rf_train_x = train_data['2019']['X'].iloc[random_data_index, ] # 랜덤 인덱스에 해당되는 학습 데이터 중 설명 변수
    rf_train_y =  train_data['2019']['y'].iloc[random_data_index, ] # 랜덤 인덱스에 해당되는 학습 데이터 중 종속 변수
    
    fitted_sm_model = rf_model.fit(rf_train_x, rf_train_y) # 학습 진행
    
    sm_model_predict = fitted_sm_model.predict(test_data['2019']['X']) # 테스트 데이터 예측
    bagging_predict_result_2019.append(sm_model_predict) # 반복문이 실행되기 전 빈 리스트에 결과 값 저장
print(f"각 라운드마다 쓰인 데이터 개수 : {bootstrapped_data_2019}")

100%|█████████████████████████████████████████| 100/100 [00:15<00:00,  6.29it/s]

각 라운드마다 쓰인 데이터 개수 : [5398, 5362, 5401, 5386, 5391, 5364, 5390, 5416, 5403, 5399, 5380, 5385, 5375, 5432, 5395, 5421, 5408, 5371, 5425, 5386, 5413, 5381, 5424, 5444, 5416, 5410, 5408, 5365, 5418, 5421, 5417, 5436, 5450, 5448, 5362, 5348, 5403, 5398, 5347, 5445, 5429, 5390, 5403, 5423, 5411, 5427, 5434, 5421, 5464, 5393, 5449, 5372, 5405, 5417, 5412, 5342, 5418, 5378, 5432, 5401, 5340, 5372, 5447, 5377, 5470, 5385, 5427, 5423, 5414, 5415, 5398, 5420, 5431, 5412, 5434, 5426, 5416, 5375, 5441, 5379, 5395, 5436, 5403, 5401, 5413, 5403, 5399, 5357, 5413, 5405, 5399, 5393, 5387, 5418, 5408, 5384, 5406, 5412, 5472, 5359]


In [616]:
# Bagging을 바탕으로 예측한 결과값에 대한 최빈값을 계산
bagging_predict_2019 = [] # 빈 리스트 생성
for lst2_index in range(test_data['2019']['X'].shape[0]): # 테스트 데이터 개수만큼의 반복
    temp_predict = [] # 임시 빈 리스트 생성 (반복문 내 결과값 저장)
    for lst_index in range(len(bagging_predict_result_2019)): # Bagging 결과 리스트 반복 10번씩 반복
        temp_predict.append(bagging_predict_result_2019[lst_index][lst2_index]) # 각 Bagging 결과 예측한 값 중 같은 인덱스를 리스트에 저장
    bagging_predict_2019.append(modefinder(temp_predict)[0]) # 해당 인덱스의 30개의 결과값에서 최빈값을 최종 리스트에 추가

In [617]:
# check point
print(len(bagging_predict_2019), len(test_data['2019']['X']))

8554 8554


## 2020년

최종 예측값 bagging_predict_2020에 리스트로 저장됨

In [618]:
bagging_predict_result_2020 = [] # 빈 리스트 생성
bootstrapped_data_2020 = []
for _ in tqdm(range(100)):
    data_index = [data_index for data_index in range(train_data['2020']['X'].shape[0])] # 학습 데이터의 인덱스를 리스트로 변환
    random_data_index = np.random.choice(data_index, train_data['2020']['X'].shape[0]) # 데이터의 1/10 크기만큼 랜덤 샘플링, // 는 소수점을 무시하기 위함
    bootstrapped_data_2020.append(len(set(random_data_index)))
    rf_train_x = train_data['2020']['X'].iloc[random_data_index, ] # 랜덤 인덱스에 해당되는 학습 데이터 중 설명 변수
    rf_train_y =  train_data['2020']['y'].iloc[random_data_index, ] # 랜덤 인덱스에 해당되는 학습 데이터 중 종속 변수
    
    fitted_sm_model = rf_model.fit(rf_train_x, rf_train_y) # 학습 진행
    
    sm_model_predict = fitted_sm_model.predict(test_data['2020']['X']) # 테스트 데이터 예측
    bagging_predict_result_2020.append(sm_model_predict) # 반복문이 실행되기 전 빈 리스트에 결과 값 저장
print(f"각 라운드마다 쓰인 데이터 개수 : {bootstrapped_data_2020}")

100%|█████████████████████████████████████████| 100/100 [00:17<00:00,  5.63it/s]

각 라운드마다 쓰인 데이터 개수 : [5170, 5119, 5121, 5079, 5108, 5172, 5153, 5172, 5109, 5108, 5147, 5180, 5084, 5174, 5094, 5137, 5200, 5137, 5139, 5101, 5132, 5143, 5099, 5115, 5195, 5115, 5149, 5150, 5132, 5145, 5167, 5088, 5152, 5153, 5144, 5131, 5171, 5159, 5124, 5113, 5145, 5117, 5136, 5166, 5125, 5132, 5150, 5149, 5188, 5095, 5167, 5144, 5132, 5120, 5115, 5124, 5084, 5104, 5129, 5176, 5111, 5139, 5145, 5115, 5145, 5125, 5189, 5141, 5146, 5132, 5140, 5146, 5118, 5155, 5156, 5156, 5176, 5090, 5109, 5084, 5147, 5132, 5109, 5147, 5157, 5148, 5128, 5149, 5133, 5150, 5134, 5125, 5139, 5163, 5115, 5124, 5135, 5134, 5094, 5138]


In [619]:
# Bagging을 바탕으로 예측한 결과값에 대한 최빈값을 계산
bagging_predict_2020 = [] # 빈 리스트 생성
for lst2_index in range(test_data['2020']['X'].shape[0]): # 테스트 데이터 개수만큼의 반복
    temp_predict = [] # 임시 빈 리스트 생성 (반복문 내 결과값 저장)
    for lst_index in range(len(bagging_predict_result_2020)): # Bagging 결과 리스트 반복 10번씩 반복
        temp_predict.append(bagging_predict_result_2020[lst_index][lst2_index]) # 각 Bagging 결과 예측한 값 중 같은 인덱스를 리스트에 저장
    bagging_predict_2020.append(modefinder(temp_predict)[0]) # 해당 인덱스의 30개의 결과값에서 최빈값을 최종 리스트에 추가

In [620]:
# check point
print(len(bagging_predict_2020), len(test_data['2020']['X']))

8122 8122


# 저장 및 제출

## 리스트 통합

In [621]:
EOE_pred = bagging_predict_2017 + bagging_predict_2018 + bagging_predict_2019 + bagging_predict_2020

In [622]:
len(EOE_pred)

35231

## 제출

In [623]:
submission = pd.read_csv('./used_file/sample_submission.csv') # sample submission 불러오기

In [624]:
submission['knowcode'] = EOE_pred

submission.to_csv('./final.csv', index=False) # 제출파일 생성